In [1]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [2]:
pip install beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ExpectedCond
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [4]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #-------------------------------------------------------------- ok
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(3)

    #--------------------------------------------------------------- ok
        
    #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()
    sleep(2)
    driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
    sleep(5)
    

    #--------------------------------------------------------------- ok

    lista_actores_peli_i=[]
    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,12):

        #accedemos a la informacion del actor/actriz:
        try:
            driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
            sleep(5)                    
            #-----------------------------------------------------------------ok

            #seleccionamos MENU desplegable, para acceder a biografia y premios:
            driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
            sleep(2)
            #acceso a la biografia
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
            sleep(2)
            #----------------------------------------------------------------ok
            
            # obtenemos el NOMBRE del actor
            nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
            sleep(5)
            #---------------------------------------------------------------- ok
            #obtenemos AÑO
            try:
                año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
                año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
            except:
                año='NULL'
            sleep(3)

            #---------------------------------------------------------------- ok

            #obtenemos el texto para filtrar informacion 'CONOCIDO POR':
            try:
                # scroll hasta el texto que necesitamos
                xpath_d='//*[@id="mini_bio"]'
                pyhton_d= driver.find_element(By.XPATH, xpath_d)
                driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
                sleep(6)
                #Obtenemos el texto:
                driver.find_element("xpath", '//*[@id="mini_bio"]').click()
                texto_actor = driver.find_element("xpath",'//*[@id="mini_bio_0"]/div/ul/div/div').text
                
                sleep(6)
                conocido_por=[]
                if re.search('conocido por|conocida por', texto_actor) == None:
                        conocido_por='NULL'
                else:
                    patron_a='\.'
                    patron_b='conocido por|conocida por' # siempre me va a dar un len()=2, por que en la cadena solo hay un 'conocido por' y este siempre estara en [1]
                    patron_c='\(\d{4}'
                    patron_d="\), |\) y |\)"
                    lista_sep=re.split(patron_b, texto_actor)
                    lista_sep_2=re.split(patron_a, lista_sep[1])
                    # la estring que nos interesa termina con un '.' simpre va a estar en [0] por que es consecutivo al 'conocido por'
                    lista_sep_3=re.split(patron_c, lista_sep_2[0])
                        
                    for y in lista_sep_3:
                        lista_sep_3_sin_espacios=(re.sub(patron_d,'', y)).strip() #quitamos espacios y parentesis
                        if lista_sep_3_sin_espacios != '': #quitamos de la lista posibles espacios
                            conocido_por.append(lista_sep_3_sin_espacios)
            except:
                conocido_por='NULL'
            #---------------------------------------------------------------- ok
            sleep(2)
            driver.back() # al acceder a 'mi biografía' hacemos click. tenemos que volver atrás. SEGUIMOS DENTRO DE BIOGRAFIA ACTOR
            sleep(2)
            driver.back() # VOLVEMOS A PAGINA PRINCIPAL ACTOR.
            sleep(10)

            #--------------------------------------------------------------- ok
            #acceso a los PREMIOS, desde el desplegable.
            try:
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
            
            except:
                #si se sale de la pagina, (si no hay texto, conocido por) volvemos a ella:
                driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click() #acceso actor.
                sleep(2) 
                driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a los premios
                sleep(2)

    
            driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la premios.
            sleep(6)
            #creamos ciclo for para sacar las los premios:
            
            try:
                premios=[]
                for i in range(1,6): #solo queremos 5 pelis
                    premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                    premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                    premios.append(premio_limpio)
                sleep(6)
            except:
                premios='NULL'
            
            

            driver.back()
            sleep(3)

            #--------------------------------------------------------------- ok
            #extraemos informacion que hace.

            que_hace=[]
            try:
                try:
                    for z in range(1,4):
                        peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                        que_hace.append(peli_hace)
                        sleep(2)
                except:
                        peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                        que_hace.append(peli_hace)
                        sleep(2)
            except:
                que_hace='NULL'
            
            #--------------------------------------------------------------- ok

            #sacamos tupla de cada actor
            lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
            tupla_x=tuple(lista_infor_actor)
            lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula
        
            sleep(6)
            driver.back()
            #volvemos a la página principal 
        except:
            pass

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [5]:
datos_csv =pd.read_csv(r"C:\Users\Mabel\Desktop\Resto_Adalab\Proyecto_2_Interstellar\Files\sublista_selenium.csv") 

In [6]:
datos_csv


,ID_Pelicula
0,tt1800301
1,tt1847719
2,tt10270648
3,tt14991492
4,tt2363363
...,...
995,tt10106152
996,tt10363182
997,tt3857098
998,tt10095498


In [7]:
id_ = datos_csv["ID_Pelicula"]

In [8]:
lista_id = id_.to_list()
print(id_)


0       tt1800301
1       tt1847719
2      tt10270648
3      tt14991492
4       tt2363363
          ...    
995    tt10106152
996    tt10363182
997     tt3857098
998    tt10095498
999     tt1067107
Name: ID_Pelicula, Length: 1000, dtype: object


In [9]:
#parte 1
lista_id_parte_1= lista_id[701:801]

In [10]:
dic_peli={'id_peli':[], 'actores_10':[]}
no_peli=[]
for id_peli in tqdm(lista_id_parte_1):
    try:
        tupla_una_peli=tuplas_actores(id_peli)
        print(tupla_una_peli)
        dic_peli['actores_10'].append(tupla_una_peli)
        dic_peli['id_peli'].append(id_peli)
    except:
        no_peli.append(id_peli)
        print('fallo peli:--------->', id_peli)
        continue

    
print(dic_peli)

  1%|          | 1/100 [00:27<45:10, 27.38s/it]

[]
fallo peli:---------> tt31037572


  3%|▎         | 3/100 [12:51<9:17:24, 344.79s/it]

[('Nicola Anselmi', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en Nico&Nico', 'Gli sbancati 2', 'Gli sbancati 1944'], 'NULL'), ('Daniele Borghi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Fate vobis', 'Cibo', 'La luce della notte'], 'NULL'), ('Simone Diotallevi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Profondo', 'Gli sbancati 2', 'Gli sbancati 1944'], 'NULL'), ('Giorgio Falcioni', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Daniele Gaudenzi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Irene Guidi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Fate vobis', 'The Hidden Reflection'], 'NULL'), ('Luca Longarini', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Maria Grazia Mea', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobi

  4%|▍         | 4/100 [27:03<14:32:26, 545.27s/it]

...

[('Robert Follin', '1971', ['Especialistas', 'Reparto', 'Equipos adicionales'], ['Vikingos', 'El último rey', 'Viking Destiny'], 'NULL'), ('Kenneth Wolf Andersen Haugen', '1989', ['Reparto', 'Especialistas', 'Producción'], ['El último superviviente', 'De Gjenvendte', 'Kuppel 16'], 'NULL'), ('Arben Bala', 'NULL', ['Reparto', 'Reparto'], ['su papel en Uno tras otro', 'La noche del lobo', 'Reprise'], 'NULL'), ('Veera Seppälä', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en El último superviviente', 'Koskinen', 'Piiritys'], 'NULL'), ('Mikael Aron', 'NULL', ['Especialistas', 'Reparto', 'Producción'], ['su papel en Christmas Cookies', 'El último superviviente', 'Våre Små Liv'], 'NULL'), ('Silje Landrø Botten', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en Trol', "Brother's Revenge", 'El abismo de Kiruna'], 'NULL'), ('Stian Buland Lie', 'NULL', ['Reparto', 'Especialistas', 'Efectos especiales'], ['su papel en El último superviviente', "Brother's Re

 99%|█████████▉| 99/100 [11:45:15<10:12, 612.81s/it]

[('Calyx Passailaigue', 'NULL', ['Dirección', 'Producción', 'Guion'], ['su papel en Zeen?', 'Speak', 'Scarlet'], ['Alternative Film Festival 2018 Nominado Best Comedy Short Film Scarlet Compartido con: Delaney Novinka', 'Couch Film Festival 2023 Nominado Best Long Short Film 20:01 to 45:00 Minutes The Injustices 2022 Nominado Best Comedy 20:01 to 45:00 minutes Zeen? Compartido con: Richard Walters 2022 Nominado Best Actor 20:01 - 45:00 Minutes Zeen? 2022 Nominado Best of the Festival Best of The Festival (Short Film) Men at War', 'RUFF: Ryerson University Film Festival 2017 Nominado RUFF Award Best Fiction Film Scarlet Compartido con: Delaney Novinka', 'Toronto Arthouse Film Festival 2020 Ganador Best Toronto Based Filmmaker Zeen?', 'Trinidad and Tobago Film Festival 2020 Ganador Best Film Best Medium Length Narrative Film Zeen? Compartido con: Richard Walters']), ('Andon Campbell', 'NULL', ['Reparto', 'Efectos visuales', 'Equipos adicionales'], ['su papel en Descension Part II', 'Desc

100%|██████████| 100/100 [11:58:56<00:00, 431.37s/it]

[('Cynthia Rothrock', '1957', ['Reparto', 'Producción', 'Dirección'], ['The Martial Arts Kid', 'In Search of the Last Action Heroes', 'Diary of a Lunatic'], 'NULL'), ('Costas Mandylor', '1965', ['Reparto', 'Reparto'], ['Saw V', 'Saw VI', 'Saw IV'], ['MTV Movie + TV Awards 2009 Nominado MTV Movie Award Best Fight Saw V Compartido con: Scott Patterson', 'Screen Actors Guild Awards 1996 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Picket Fences Compartido con: Amy Aquino · Kathy Baker · Don Cheadle · Kelly Connell · Fyvush Finkel · Lauren Holly · Marlee Matlin · · · · 1995 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Picket Fences Compartido con: Kathy Baker · Don Cheadle · Holly Marie Combs · Kelly Connell · Robert Cornthwaite · Fyvush Finkel · · · · · ·', 'Maverick Movie Awards 2009 Nominado Maverick Movie Award Best Actor: Feature Emma Blue', "Buffalo Dreams Fantastic Film Festival 2020 Nominado Dreamer Award Outstanding Actor in a 

In [1]:
dic_peli={'id_peli': ['tt30997271', 'tt10089828', 'tt2937898', 'tt10294668', 'tt3264412', 'tt10661470', 'tt10612842', 'tt17499324', 'tt11633066', 'tt13416954', 'tt1074638', 'tt31450469', 'tt14913250', 'tt12837054', 'tt1800672', 'tt1071875', 'tt10553722', 'tt10280778', 'tt11306136', 'tt28520532', 'tt10500534', 'tt10010836', 'tt10028626', 'tt10106838', 'tt3846972', 'tt12080810', 'tt10365674', 'tt11296440', 'tt0464891', 'tt12831622', 'tt10258992', 'tt3775202', 'tt12976372', 'tt1582584', 'tt3286680', 'tt12677492', 'tt3149366', 'tt2034800', 'tt10510250', 'tt2431478', 'tt3717252', 'tt11808288', 'tt13904268', 'tt3129364', 'tt10027848', 'tt26448850', 'tt10253820', 'tt10908226', 'tt13086098', 'tt10217030', 'tt2065997', 'tt10077576', 'tt11120680', 'tt12985342', 'tt1213590', 'tt1194424', 'tt2369154', 'tt2267968', 'tt10986214', 'tt14451444', 'tt10274250', 'tt23024254', 'tt2336284', 'tt10097350', 'tt10327026', 'tt10611546', 'tt3179448', 'tt10109948', 'tt3402594', 'tt10043054', 'tt10056994', 'tt10832044', 'tt10365828', 'tt1401636', 'tt32119109', 'tt2302966', 'tt10009148', 'tt12402412', 'tt10588964', 'tt11867620', 'tt10479784', 'tt11362678', 'tt10123682', 'tt17074300'], 'actores_10': [[], [('Nicola Anselmi', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en Nico&Nico', 'Gli sbancati 2', 'Gli sbancati 1944'], 'NULL'), ('Daniele Borghi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Fate vobis', 'Cibo', 'La luce della notte'], 'NULL'), ('Simone Diotallevi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Profondo', 'Gli sbancati 2', 'Gli sbancati 1944'], 'NULL'), ('Giorgio Falcioni', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Daniele Gaudenzi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Irene Guidi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Fate vobis', 'The Hidden Reflection'], 'NULL'), ('Luca Longarini', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Maria Grazia Mea', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Florindo Piccini', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL'), ('Filippo Tranquilli', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gli sbancati 2', 'Gli sbancati 1944', 'Fate vobis'], 'NULL')], [('Oscar Isaac', '1979', ['Reparto', 'Producción', 'Banda sonora'], ['Ex_Machina', 'A propósito de Llewyn Davis', 'El año más violento'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2022 Ganador Saturn Award Best Actor in a Streaming Series Caballero Luna 2014 Nominado Saturn Award Best Actor A propósito de Llewyn Davis', 'ALMA Awards 2012 Nominado ALMA Award Favorite Movie Actor: Supporting Role Cristiada', 'Film Critics Circle of Australia Awards 2010 Nominado FCCA Award Best Supporting Actor - Male Balibo', 'Australian Film Institute 2009 Ganador AFI Award Best Supporting Actor Balibo', 'Critics Choice Awards 2014 Nominado Critics Choice Award Best Song A propósito de Llewyn Davis Song: "Please Mr. Kennedy" Compartido con: Justin Timberlake · Adam Driver']), ('Elyes Gabel', '1983', ['Reparto', 'Dirección', 'Producción'], ['Guerra mundial Z', 'El año más violento', 'Scorpion'], ['National Television Awards, UK 2005 Nominado National Television Award Most Popular Newcomer Casualty', 'Screen Actors Guild Awards 2012 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Juego de tronos Compartido con: Amrita Acharia · Mark Addy · Alfie Allen · Josef Altin · Sean Bean · Susan Brown · Emilia Clarke · · · · · · · · · · · · · · · · · · · · · · · ·', 'Idyllwild International Festival of Cinema 2019 Nominado Grand Jury Award Best Director - Featurette Grimshaw', 'CinEuphoria Awards 2020 Ganador CinEuphoria Merit - Honorary Award Juego de tronos Compartido con: David Benioff · D.B. Weiss · George R.R. Martin · Ramin Djawadi · Michele Clapton · Amrita Acharia · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Top Shorts Film Festival 2018 Ganador Jury Prize Best Narrative Film Grimshaw Compartido con: Cottrell Guidry']), ('Jessica Chastain', '1977', ['Reparto', 'Producción', 'Banda sonora'], ['La noche más oscura', "Molly's Game", 'El año más violento'], ['Academy Awards, USA 2022 Ganador Oscar Best Performance by an Actress in a Leading Role Los ojos de Tammy Faye 2013 Nominado Oscar Best Performance by an Actress in a Leading Role La noche más oscura 2012 Nominado Oscar Best Performance by an Actress in a Supporting Role Criadas y señoras', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2016 Nominado Saturn Award Best Actress Marte (The Martian) 2016 Ganador Saturn Award Best Supporting Actress La cumbre escarlata 2015 Nominado Saturn Award Best Supporting Actress Interstellar 2013 Nominado Saturn Award Best Actress La noche más oscura 2012 Nominado Saturn Award Best Actress Take Shelter', 'BAFTA Awards 2013 Nominado BAFTA Film Award Best Leading Actress La noche más oscura 2012 Nominado BAFTA Film Award Best Supporting Actress Criadas y señoras', "Critics Choice Awards 2022 Ganador Critics Choice Award Best Actress Los ojos de Tammy Faye 2018 Nominado Critics Choice Award Best Actress Molly's Game 2015 Nominado Critics Choice Award Best Supporting Actress El año más violento 2015 Ganador Critics' Choice MVP Award El año más violentoInterstellarLa señorita JuliaLa desaparición de Eleanor RigbyLa desaparición de Eleanor Rigby: EllaLa desaparición de Eleanor Rigby: Él 2013 Ganador Critics Choice Award Best Actress La noche más oscura 1 más", 'Chicago Film Critics Association Awards 2021 Nominado CFCA Award Best Actress Los ojos de Tammy Faye 2014 Nominado CFCA Award Best Supporting Actress El año más violento 2012 Ganador CFCA Award Best Actress La noche más oscura 2011 Ganador CFCA Award Best Supporting Actress El árbol de la vida']), ('Lorna Pruce', 'NULL', ['Reparto', 'Vestuario', 'Especialistas'], ['su papel en Tan fuerte, tan cerca', 'El año más violento', 'Noé'], 'NULL'), ('Christopher Abbott', '1986', ['Reparto', 'Producción', 'Banda sonora'], ['Llega de noche', 'Possessor Uncut', 'Pobres criaturas'], ['Critics Choice Awards 2020 Nominado Critics Choice Award Best Actor in a Limited Series or Movie Made for Television Trampa 22', 'Chicago Film Critics Association Awards 2015 Nominado CFCA Award Best Actor James White 2015 Nominado CFCA Award Most Promising Performer James White', 'Golden Globes, USA 2020 Nominado Golden Globe Best Performance by an Actor in a Limited Series or a Motion Picture Made for Television Trampa 22', 'Gotham Awards 2015 Nominado Gotham Independent Film Award Best Actor James White 2011 Nominado Gotham Independent Film Award Best Ensemble Performance Martha Marcy May Marlene Compartido con: Elizabeth Olsen · Brady Corbet · Hugh Dancy · Maria Dizzia · Julia Garner · John Hawkes · Louisa Krause ·', 'Hamptons International Film Festival 2015 Ganador Breakthrough Performer James White']), ('Matthew Maher', '1971', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Adiós pequeña, adiós', 'Capitana Marvel', 'Una historia casi divertida'], 'NULL'), ('Albert Brooks', '1947', ['Reparto', 'Guion', 'Música'], ['Drive', 'Al filo de la noticia', 'El cielo'], ['Academy Awards, USA 1988 Nominado Oscar Best Actor in a Supporting Role Al filo de la noticia', 'Academy of Science Fiction, Fantasy & Horror Films, USA 1992 Nominado Saturn Award Best Writing El cielo... próximamente', 'American Comedy Awards, USA 1997 Nominado American Comedy Award Funniest Actor in a Motion Picture (Leading Role) Las manías de mamá 1992 Nominado American Comedy Award Funniest Actor in a Motion Picture (Leading Role) El cielo... próximamente 1988 Ganador American Comedy Award Funniest Supporting Male Performer - Motion Picture or TV Al filo de la noticia', 'Boston Society of Film Critics Awards 2011 Ganador BSFC Award Best Supporting Actor Drive 1988 Ganador BSFC Award Best Actor Al filo de la noticia', 'Critics Choice Awards 2012 Nominado Critics Choice Award Best Supporting Actor Drive']), ('Jerry Adler', '1929', ['Equipos adicionales', 'Reparto', 'Dirección'], ['Misterioso asesinato en Manhattan', 'En sus zapatos', 'Secretos compartidos (Prime'], 'NULL'), ('Quinn Meyers', 'NULL', ['Reparto', 'Producción', 'Equipos adicionales'], ['su papel en Cuando todo está perdido', 'El año más violento', 'Margin Call'], 'NULL'), ('Chester Jones III', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Hey Diddle Diddle', 'El año más violento', 'Hey Diddle Diddle'], 'NULL')], [('Jennings Brower', 'NULL', ['Reparto', 'Reparto'], ['su papel en Netflix Dreams', 'Kickstart My Heart', 'Cruel and Unusual Therapy Too'], 'NULL'), ('Gina Marie Rodriguez', 'NULL', ['Reparto', 'Producción', 'Responsable'], ['su papel en Cruel and Unusual Therapy', 'Frame of Reference', 'Happy Hour'], ['WorldFest Houston 2017 Ganador Bronze Remi Award Romantic Comedy Short Frame of Reference Compartido con: Ian J. Keeney · Steven Hayet', 'Hang Onto Your Shorts Film Festival, NJ 2022 Nominado Best Actress in a Comedy Best Actress in a Comedy Cruel and Unusual Therapy Too 2022 Nominado Best Director - Comedy Best Director, Comedy Cruel and Unusual Therapy Too 2017 Ganador Festival Award Best Actress in a Comedy Frame of Reference', 'Golden Door International Film Festival of Jersey City 2017 Nominado Festival Award Best Local Film Frame of Reference Compartido con: Ian J. Keeney · Steven Hayet', 'Shorty Awards 2021 Nominado Shorty Award Best in Social Activism Riding with Sunshine Compartido con: Kristian Comer · J.T. Maguire · Rahiem Wilson Morgan · Scott Churchson · Rick Levi · Peter Simpson · · · · · · · · · · · · · · · · · ·', 'Los Angeles Cinefest 2017 Nominado Jury Award Best Short Film Frame of Reference Compartido con: Ian J. Keeney · Steven Hayet'])], [], [('Wyatt Daane', 'NULL', ['Guion', 'Reparto', 'Producción'], ['su papel en The Rifter', 'Stool: a love story', 'Return to Dog Valley'], 'NULL'), ('Zach Louis', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Bella y los Bulldogs', 'Marshall the Miracle Dog', 'Boys in Blue'], 'NULL'), ('Katie McDonald', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Skinny-Webz', 'The End', 'The Pre-Game'], 'NULL'), ('Steven Portoles', 'NULL', ['Dirección', 'Guion', 'Reparto'], ["su papel en Compassion, Skinny-Webz: Honey I'm Home", 'Buried'], 'NULL'), ('Kenneth Ryan', 'NULL', ['Dirección', 'Producción', 'Edición'], ['su papel en Blight', 'PSA y Misery Is the River of the World'], 'NULL'), ('Alex J. Dean', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Nerdtabulous', 'Lost Tapes', 'El medallón perdido: Las aventuras de Billy Stone'], 'NULL')], [('Julia Jordan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dinner Time', 'Eleanor', 'Dear America: A Film by Generation Z'], 'NULL'), ('Oliver Prose', 'NULL', ['Reparto', 'Reparto'], ['su papel en Citizen Khan y Eleanor'], 'NULL')], [('Ben Johnson', '1979', ['Reparto', 'Reparto'], ['Zero Hour', 'Prey', 'Jekyll and Hyde: The True Story'], 'NULL'), ('Sean Ramey', 'NULL', ['Reparto', 'Especialistas', 'Reparto'], ['su papel en A Game of Pool', 'A Place Called Home', 'Chicago P'], 'NULL'), ('Muskan Verma', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Prey', 'A Conscious Is a Terrible Thing to Waste', 'I Know What You Need'], 'NULL'), ('Megan Salisbury', 'NULL', ['Reparto', 'Guion y continuidad', 'Equipos adicionales'], ['su papel en The Business of Christmas', 'Big Freaking Rat', 'The Amityville Harvest'], 'NULL'), ('Mason Hawkes', 'NULL', ['Reparto', 'Guion', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en Prey'], 'NULL'), ('Dylan Kraekel', 'NULL', ['Reparto', 'Reparto'], ['su papel en Prey'], 'NULL'), ('Nathaniel Dombek', 'NULL', ['Reparto', 'Dirección de arte', 'Producción'], ['su papel en Ash Woman', 'Patrol', 'Prey'], 'NULL'), ('August Thornton', 'NULL', ['Reparto', 'Reparto'], ['su papel en Suffocation', 'Chapman Speaks', 'The United States of Horror: Chapter 1'], 'NULL'), ('Paul Mattor', 'NULL', ['Reparto', 'Reparto'], ['su papel en Prey'], 'NULL')], [('Nina E. Jordan', 'NULL', ['Reparto', 'Reparto'], ['su papel en El club del odio', 'Desaparecido a medianoche', 'Thirsty'], 'NULL'), ('Megumi Smisson', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Fare', 'Ghost of a Chance', 'Scuba Love'], 'NULL'), ('Millie Jean Warren', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Manufactured Beauty', 'Meet the Team', "It's for Exposure"], 'NULL'), ('Diogo Hausen', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en La vida sexual de las universitarias', 'Henry Danger', 'Solve'], ['The 48 Hour Film Project, Los Angeles 2020 Nominado Judges Award Best Production Design The Debate', 'LA Underground Film Forum 2019 Ganador Best Comedy Micro Short Film Best Comedy Film Russell & Cornell: The Case of the Award Show Haircut', "Flicks Film Festival 2021 Ganador July Award Best Web-Series Bastet's Test", "Venice Short Film Festival 2021 Ganador Festival Award Best Web Series/TV Pilot Bastet's Test", 'Austin Micro Short Film Festival 2020 Ganador Festival Award Most Original Concept Book Clubbed']), ('AnnieScott Rogers', 'NULL', ['Reparto', 'Reparto'], ['su papel en Saltwater', 'Breakdowns', 'Sonata'], 'NULL'), ('JJ Herz', 'NULL', ['Reparto', 'Guion', 'Dirección'], 'NULL', 'NULL'), ('George Higgins', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ghost of a Chance'], 'NULL'), ('Corinne Meadors', 'NULL', ['Reparto', 'Reparto'], ['su papel en Neighbor', 'The Great Controversy Ended', 'Fire on the Ridge'], ['WorldFest Houston 2017 Ganador Gold Award Best Short Film The Great Controversy Ended', 'Accolade Competition 2020 Nominado Award of Excellence Actress: Leading Fire on the Ridge Compartido con: Tressa Ohler · Sandra Dee Richardson · Irene Klarich · Debra Smith · Sally Rudloff 2017 Ganador Award Of Recognition Leading Actress The Great Controversy Ended', "Sacramento International Film Festival 2020 Nominado People's Choice Outstanding Regional Actress Best Actress Two Kisses", 'Best Shorts Competition 2017 Ganador Award of Recognition Short Film The Great Controversy Ended Compartido con: Nancy Hamilton · Desiree DeAnda · Sandra Dee Richardson · Rick Richardson', "IndieFEST Film Awards 2020 Ganador Award of Recognition Film Feature Fire on the Ridge Compartido con: Nancy Hamilton · Desiree DeAnda · Sandra Dee Richardson · Irene Klarich · Debra Smith · Sally Rudloff · · · 2020 Ganador Award of Recognition Women Filmmakers Fire on the Ridge Compartido con: Nancy Hamilton · Desiree DeAnda · Sandra Dee Richardson · Tressa Ohler · Bobby O'Reiley · Leah Koon Goymer · · · 2020 Ganador Award of Recognition Actress: Leading Fire on the Ridge 2020 Ganador Award of Recognition Lead Actress Fire on the Ridge Compartido con: Sandra Dee Richardson"]), ('Duncan Heath', 'NULL', ['Reparto', 'Reparto'], ['su papel en Pick a Side', 'City Council: Parrhesia in the Commons', 'Dope Queens'], 'NULL'), ('Diane Snowder', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ghost of a Chance', 'Jolted 2', 'Rest Stop'], 'NULL')], [], [('Daniel Craig', '1968', ['Reparto', 'Producción', 'Banda sonora'], ['Casino Royale', 'Sin tiempo para morir', 'Skyfall'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2021 Nominado Saturn Award Best Actor Puñales por la espalda (Knives Out) 2013 Nominado Saturn Award Best Actor Skyfall 2007 Nominado Saturn Award Best Actor Casino Royale', 'Berlin International Film Festival 2000 Ganador EFP Shooting Star United Kingdom.', 'BAFTA Awards 2007 Nominado BAFTA Film Award Best Actor in a Leading Role Casino Royale', 'British Independent Film Awards 2007 Ganador Variety Award 2004 Nominado British Independent Film Award Best Actor El intruso (Enduring Love) 2000 Ganador British Independent Film Award Best Actor Some Voices 1999 Nominado British Independent Film Award Best Actor La Trinchera', 'Critics Choice Awards 2016 Nominado Critics Choice Award Best Actor in an Action Movie Spectre 2013 Ganador Critics Choice Award Best Actor in an Action Movie Skyfall']), ('Judi Dench', '1934', ['Reparto', 'Música', 'Dirección'], ['Skyfall', 'Diario de un escándalo', 'Shakespeare in Love (Shakespeare enamorado'], ['Academy Awards, USA 2022 Nominado Oscar Best Performance by an Actress in a Supporting Role Belfast 2014 Nominado Oscar Best Performance by an Actress in a Leading Role Philomena 2007 Nominado Oscar Best Performance by an Actress in a Leading Role Diario de un escándalo 2006 Nominado Oscar Best Performance by an Actress in a Leading Role Mrs. Henderson presenta 2002 Nominado Oscar Best Actress in a Leading Role Iris 3 más', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2013 Nominado Saturn Award Best Supporting Actress Skyfall 2009 Nominado Saturn Award Best Supporting Actress Quantum of Solace 2007 Nominado Saturn Award Best Actress Diario de un escándalo', 'American Comedy Awards, USA 2001 Nominado American Comedy Award Funniest Female Performer in a TV Special (Leading or Supporting) Network, Cable or Syndication Las últimas rubias explosivas', 'Boston Society of Film Critics Awards 2013 Nominado BSFC Award Best Actress Philomena 2006 Nominado BSFC Award Best Actress Diario de un escándalo', 'BAFTA Awards 2014 Nominado BAFTA Film Award Best Leading Actress Philomena 2013 Nominado BAFTA Film Award Best Supporting Actress Skyfall 2012 Nominado BAFTA Film Award Best Supporting Actress Mi semana con Marilyn 2008 Nominado BAFTA TV Award Best Actress Cranford 2007 Nominado BAFTA Film Award Best Actress in a Leading Role Diario de un escándalo 22 más']), ('Javier Bardem', '1969', ['Reparto', 'Producción', 'Música'], ['Skyfall', 'No es país para viejos', 'Mar adentro'], ['Ariel Awards, Mexico 2011 Nominado Silver Ariel Best Actor (Mejor Actuación Masculina) Biutiful', 'Academy Awards, USA 2022 Nominado Oscar Best Performance by an Actor in a Leading Role Ser los Ricardo 2011 Nominado Oscar Best Performance by an Actor in a Leading Role Biutiful 2008 Ganador Oscar Best Performance by an Actor in a Supporting Role No es país para viejos 2001 Nominado Oscar Best Actor in a Leading Role Antes que anochezca', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2013 Nominado Saturn Award Best Supporting Actor Skyfall 2008 Ganador Saturn Award Best Supporting Actor No es país para viejos', 'ALMA Awards 2009 Nominado ALMA Award Actor in Film Vicky Cristina Barcelona', 'Boston Society of Film Critics Awards 2007 Ganador BSFC Award Best Supporting Actor No es país para viejos 2000 Nominado BSFC Award Best Actor Antes que anochezca'])], [], [('Blanche Gardin', '1977', ['Reparto', 'Guion', 'Dirección'], ['La meilleure version de moi-même', 'Borrar el historial', 'Problemos'], ['Lisbon & Estoril Film Festival 2020 Ganador Jury Special Prize Artistic Contribution Borrar el historial', "L'Association des Critiques de Séries 2022 Nominado A.C.S. Award Best Actress La meilleure version de moi-même", "Molière Award 2019 Ganador Molière Award Molière de l'Humour Bonne nuit Blanche 2018 Ganador Molière Award Molière de l'Humour Je parle toute seule", 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Laurent Lafitte', '1973', ['Reparto', 'Guion', 'Dirección'], ['Elle', 'El principito', 'No se lo digas a nadie'], ['BAFTA Awards 2024 Ganador BAFTA TV Award International Bernard Tapie Compartido con: Bruno Nahon · Tristan Séguéla · Olivier Demangel', 'César Awards, France 2018 Nominado César Best Supporting Actor (Meilleur acteur dans un second rôle) Nos vemos allá arriba 2017 Nominado César Best Supporting Actor (Meilleur acteur dans un second rôle) Elle', 'CinEuphoria Awards 2017 Nominado CinEuphoria Best Supporting Actor - International Competition Elle', 'Order of Arts and Letters, France 2013 Ganador Knight of the Order of Arts and Letters', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Maxence Tual', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rodin', 'Il est des nôtres', "Victor Hugo, ennemi d'État"], 'NULL'), ('Nuno Lopes', '1978', ['Reparto', 'Reparto'], ['São Jorge', 'Las líneas de Wellington', 'Goodnight Irene'], ['Berlin International Film Festival 2006 Ganador EFP Shooting Star Portugal.', 'Golden Globes, Portugal 2023 Nominado Golden Globe Best Actor Restos do Vento 2022 Nominado Golden Globe Best Actor - Television Princípio, Meio e Fim 2021 Nominado Golden Globe Best Actor - Television White Lines 2018 Ganador Golden Globe Best Actor São Jorge 2017 Ganador Golden Globe Best Actor Posto Avançado do Progresso 5 más', 'Monte-Carlo TV Festival 2009 Nominado Golden Nymph Outstanding Actor - Comedy Series Os Contemporâneos', 'Venice Film Festival 2016 Ganador Venice Horizons Award Best Actor São Jorge', 'Coimbra Caminhos do Cinema Português 2017 Ganador Film Award Best Actor São Jorge 2010 Ganador Film Award Best Supporting Actor Efeitos Secundários']), ('Marthe Keller', '1945', ['Reparto', 'Equipos adicionales', 'Banda sonora'], ['Marathon Man', 'Fedora', 'Más allá de la vida'], ['Bambi Awards 1977 Ganador Bambi Best Actress - National Marathon Man', 'Biarritz International Festival of Audiovisual Programming 2003 Ganador Golden FIPA Fiction: Actress Par amour', 'David di Donatello Awards 1988 Nominado David Best Supporting Actress (Migliore Attrice non Protagonista) Ojos negros Ex aequo with Vivian Wu for El último emperador (1987).', 'Genie Awards 1982 Nominado Genie Best Performance by a Foreign Actress Servicios secretos paralelos', 'German Film Awards 1967 Nominado Film Award in Gold Best Performance by an Actress in a Supporting Role (Beste darstellerische Leistung - Weibliche Nebenrolle) Wilder Reiter GmbH']), ('Lisa Mirey', 'NULL', ['Reparto', 'Reparto'], ['su papel en Everybody Loves Jeanne'], 'NULL'), ('Andrew Sanko Logan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Everybody Loves Jeanne'], 'NULL'), ('Samira Sedira', 'NULL', ['Reparto', 'Reparto'], ['su papel en Malavita', 'El asalto', 'El despertar de María'], 'NULL'), ('Patty Hannock', 'NULL', ['Reparto', 'Equipos adicionales', 'Casting'], ['su papel en 3 días para matar', 'Hitch', '360 - Juego de destinos'], 'NULL'), ('Pedro Lacerda', '1971', ['Reparto', 'Guion', 'Reparto'], ['Terra Nova - O Filme', 'Os Maias: Cenas da Vida Romântica', 'Lua Vermelha'], ['Golden Globes, Portugal 2023 Nominado Golden Globe Best Actor Um Filme em Forma de Assim 2022 Nominado Golden Globe Best Actor Terra Nova - O Filme', 'SoHo International Film Festival 2020 Ganador Festival Prize Best Performance by an Ensemble - World Feature Terra Nova - O Filme Compartido con: Vítor Andrade · Miguel Borges · Virgílio Castelo · João Catarré · João Craveiro · Ricardo de Sá · · · · · · ·', 'CinEuphoria Awards 2022 Nominado CinEuphoria Best Actor - National Competition Terra Nova - O Filme 2022 Nominado CinEuphoria Best Ensemble - National Competition Terra Nova - O Filme Compartido con: Vítor Andrade · Miguel Borges · Virgílio Castelo · João Catarré · João Craveiro · Paulo Manso · · · · · · · 2021 Nominado CinEuphoria Best Actor - National Competition A Arte de Morrer Longe 2017 Nominado CinEuphoria Best Actor - National Competition Axilas 2015 Ganador CinEuphoria Best Ensemble - National Competition Os Maias: Cenas da Vida Romântica Compartido con: Rita Blanco · Nuno Casanovas · Graciano Dias · Maria Flor · Pedro Inês · Adriano Luz · Hugo Mestre Amaro · · · · · · · 2 más', 'Portuguese Film Academy Sophia Awards 2022 Nominado Sophia Best Actor (Melhor Actor) Terra Nova - O Filme', 'YMotion Festival de Cinema Jovem 2022 Ganador Film Award Best Cast Encoberto Compartido con: Rafael Morais · João Pedro Mamede · Manuel Mozos · Simon Frankel · Rafael Gomes'])], [('Bianca Bin', 'NULL', ['Reparto', 'Reparto'], ['su papel en Boogie Oogie', 'O Outro Lado do Paraíso', 'Canastra Suja'], ['Prêmio Contigo, Brazil 2015 Nominado Contigo Best Actress (Melhor Atriz) Boogie Oogie', 'Extra Television Awards, Brazil 2019 Nominado Extra Award Best Actress (Melhor Atriz) O Outro Lado do Paraíso', 'Prêmio Guarani 2019 Nominado Premio Guarani Best Female Newcomer Canastra Suja', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Sergio Guizé', '1980', ['Reparto', 'Reparto'], ['Êta Mundo Bom!', 'A Dona do Pedaço', 'Saramandaia'], ['Prêmio Contigo, Brazil 2020 Nominado Contigo Best Romantic Couple (Melhor Par Romântico) A Dona do Pedaço Compartido con: Paolla Oliveira 2020 Nominado Contigo Best Supporting Actor (Melhor Ator Coadjuvante) A Dona do Pedaço', 'Extra Television Awards, Brazil 2016 Nominado Extra Award Best Actor (Melhor Ator) Êta Mundo Bom! 2013 Nominado Extra Award Most Promising Actor (Melhor Ator Revelação) Saramandaia', 'Troféu Imprensa, Brazil 2017 Ganador Troféu Imprensa Best Actor (Melhor Ator) Êta Mundo Bom! 2017 Ganador Troféu Internet Best Actor (Melhor Ator) Êta Mundo Bom!', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Paulo Miklos', '1959', ['Reparto', 'Música', 'Composición'], ['El Invasor', 'É Proibido Fumar', 'O Homem Cordial'], ['Brazilia Festival of Brazilian Cinema 2009 Ganador Candango Trophy Best Actor (Melhor Ator) É Proibido Fumar 2001 Ganador Candango Trophy Best Score (Melhor Trilha Sonora) El Invasor Compartido con: Rica Amabis · Daniel Ganjaman · Sabotage · Tejo Damasceno 2001 Ganador Special Jury Award El Invasor For the acting.', 'Cinema Brazil Grand Prize 2009 Nominado Cinema Brazil Grand Prize Best Supporting Actor (Melhor Ator Coadjuvante) Estômago 2003 Ganador Cinema Brazil Grand Prize Best Supporting Actor (Melhor Ator Coadjuvante) El Invasor', 'Gramado Film Festival 2022 Nominado Golden Kikito Best Supporting Actor (Melhor Ator Coadjuvante) O Clube dos Anjos Compartido con: André Abujamra · Ângelo Antônio · António Capelo · Augusto Madeira · César Mello · Marco Ricca 2019 Ganador Golden Kikito Best Actor (Melhor Ator) O Homem Cordial', 'Miami Brazilian Film Festival 2002 Ganador Crystal Lens Best Supporting Actor El Invasor 2002 Ganador Crystal Lens Best Music El Invasor', 'Prêmio Contigo Cinema, Brazil 2010 Nominado Jury Award Best Actor (Melhor Ator) É Proibido Fumar 2010 Ganador Jury Award Best Music (Melhor Trilha Sonora) É Proibido Fumar Compartido con: Marcio Nigro']), ('Neusa Borges', '1941', ['Reparto', 'Banda sonora', 'Reparto'], ['O Clone', 'A Indomada', 'De Corpo e Alma'], ['São Paulo Association of Art Critics Awards 1993 Ganador APCA Trophy Television: Best Supporting Actress (Televisão: Melhor Atriz Coadjuvante) De Corpo e Alma', 'Prêmio Contigo, Brazil 1998 Ganador Contigo Best Supporting Actress (Melhor Atriz Coadjuvante) A Indomada', 'Prêmio Qualidade, Brazil 2008 Nominado Prêmio Qualidade Best Supporting Actress (Melhor Atriz Coadjuvante) Polaróides Urbanas', "Pena de Prata 2022 Nominado Pena de Prata Best Ensemble in a Comedy Series Encantado's Compartido con: Augusto Madeira · Dandara Mariana · Dhonata Augusto · Dhu Moraes · Digão Ribeiro · Evelyn Castro · · · · · · · · ·", 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Cássio Scapin', '1964', ['Reparto', 'Reparto'], ['Caminhos do Coração', 'Um Só Coração', 'Os Mutantes: Caminhos do Coração'], 'NULL'), ('Gregory Pena', 'NULL', ['Reparto', 'Reparto'], ['su papel en Escape from Brazil', 'O Time do Povo'], 'NULL'), ('Oscar Filho', '1978', ['Reparto', 'Guion', 'Reparto'], ['Jogos Clandestinos, O Time do Povo', 'Escola de Quebrada'], 'NULL'), ('Luna Milano', 'NULL', ['Reparto', 'Reparto'], ['su papel en O Time do Povo'], 'NULL'), ('Toquinho', '1946', ['Composición', 'Música', 'Reparto'], ['O Bem-Amado', 'Woman on Top', 'Fogo Sobre Terra'], 'NULL'), ('Liam Haym', 'NULL', ['Reparto', 'Reparto'], ['su papel en O Time do Povo'], 'NULL')], [], [('Nicolas Cage', '1964', ['Reparto', 'Producción', 'Dirección'], ['Cara a cara (Face/Off', 'Leaving Las Vegas', 'La Roca'], ['Academy Awards, USA 2003 Nominado Oscar Best Actor in a Leading Role Adaptation (El ladrón de orquídeas) 1996 Ganador Oscar Best Actor in a Leading Role Leaving Las Vegas', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2024 Ganador Saturn Award Best Supporting Actor in a Film Renfield 2019 Nominado Saturn Award Best Actor Mandy 1998 Nominado Saturn Award Best Actor Cara a cara (Face/Off)', 'American Comedy Awards, USA 1993 Nominado American Comedy Award Funniest Actor in a Motion Picture (Leading Role) Luna de miel para tres 1988 Nominado American Comedy Award Funniest Actor in a Motion Picture (Leading Role) Arizona Baby', 'Annie Awards 2021 Nominado Annie Outstanding Achievement for Voice Acting in an Animated Feature Los Croods: Una nueva era', 'Blockbuster Entertainment Awards 2001 Ganador Blockbuster Entertainment Award Favorite Actor - Comedy/Romance Family Man 2001 Nominado Blockbuster Entertainment Award Favorite Actor - Action 60 segundos 1999 Ganador Blockbuster Entertainment Award Favorite Actor - Suspense Snake Eyes (Ojos de serpiente) 1999 Ganador Blockbuster Entertainment Award Favorite Actor - Drama/Romance City of Angels 1998 Ganador Blockbuster Entertainment Award Favorite Actor - Action/Adventure Cara a cara (Face/Off)Con Air (Convictos en el aire) 1 más']), ('Violante Placido', '1976', ['Reparto', 'Composición', 'Reparto'], ['El americano', 'Ghost Rider: Espíritu de venganza', 'Lezioni di cioccolato'], ['David di Donatello Awards 2004 Nominado David Best Actress (Migliore Attrice Protagonista) Che ne sarà di noi', 'Italian National Syndicate of Film Journalists 2013 Nominado Silver Ribbon Best Song (Migliore Canzone) Evil Things For the song "Hey Sister". 2004 Nominado Silver Ribbon Best Actress (Migliore Attrice Protagonista) Ora o mai più 2003 Nominado Silver Ribbon Best Actress (Migliore Attrice Protagonista) L\'anima gemella Compartido con: Valentina Cervi', "Venice Film Festival 2002 Ganador Wella Prize L'anima gemella Compartido con: Valentina Cervi", 'Monte-Carlo Comedy Film Festival 2007 Ganador Jury Prize Best Actress Lezioni di cioccolato', 'Golden Globes, Italy 2007 Nominado Golden Globe Best Actress (Migliore Attrice) La cena per farli conoscere']), ('Ciarán Hinds', '1953', ['Reparto', 'Banda sonora', 'Reparto'], ['Belfast', 'La mujer de negro', 'El topo'], ['Academy Awards, USA 2022 Nominado Oscar Best Performance by an Actor in a Supporting Role Belfast', 'BAFTA Awards 2022 Nominado BAFTA Film Award Best Supporting Actor Belfast', "British Independent Film Awards 2022 Nominado British Independent Film Award Best Ensemble Performance El prodigio Compartido con: Kíla Lord Cassidy · Florence Pugh · Tom Burke · Toby Jones · Niamh Algar · Elaine Cassidy · Brían F. O'Byrne · 2021 Nominado British Independent Film Award Best Supporting Actor Belfast", 'Critics Choice Awards 2022 Nominado Critics Choice Award Best Supporting Actor Belfast', 'Dallas-Fort Worth Film Critics Association Awards 2021 Nominado DFWFCA Award Best Supporting Actor Belfast 3rd place']), ('Idris Elba', '1972', ['Reparto', 'Producción', 'Guion'], ['Beasts of No Nation', 'Los perdedores', 'Thor: Ragnarok'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2022 Nominado Saturn Award Best Actor El escuadrón suicida', 'Acapulco Black Film Festival 2014 Nominado Hollywood Award Artist of the Year Mandela: Del mito al hombrePacific RimThor: El mundo oscuro 2014 Nominado Hollywood Award Best Actor Mandela: Del mito al hombre', 'Berlin International Film Festival 2018 Nominado C.I.C.A.E. Award Panorama Yardie', 'BET Awards 2024 Nominado BET Award Best Actor 2016 Nominado BET Award Best Actor ZootrópolisBeasts of No NationLuther 2015 Nominado BET Award Best Actor LutherSin escrúpulos 2014 Nominado BET Award Best Actor Mandela: Del mito al hombreThor: El mundo oscuro 2012 Nominado BET Award Best Actor ThorGhost Rider: Espíritu de venganzaLuther 5 más', 'BAFTA Awards 2016 Nominado BAFTA Film Award Best Supporting Actor Beasts of No Nation 2016 Nominado BAFTA TV Award Best Leading Actor Luther']), ('Johnny Whitworth', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['Sin límites', 'Empire Records', 'Legítima defensa, de John Grisham'], 'NULL'), ('Fergus Riordan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ghost Rider: Espíritu de venganza', 'No crezcas o morirás', 'Frágiles'], 'NULL'), ('Spencer Wilding', '1972', ['Reparto', 'Especialistas', 'Producción'], ['Rogue One: Una historia de Star Wars', 'Victor Frankenstein', 'Hércules: El origen de la leyenda'], 'NULL'), ('Sorin Tofan', '1969', ['Reparto', 'Guion', 'Reparto'], ['Ghost Rider: Espíritu de venganza', 'Baieti buni', 'Hackerville'], 'NULL'), ('Jacek Koman', '1956', ['Reparto', 'Banda sonora', 'Reparto'], ['Moulin Rouge', 'Hijos de los hombres', 'Resistencia'], 'NULL'), ('Anthony Head', '1954', ['Reparto', 'Banda sonora', 'Reparto'], ['Buffy, cazavampiros', 'Repo! The Genetic Opera', 'Rosas Rojas'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2001 Nominado Saturn Award Best Supporting Actor on Television Buffy, cazavampiros', 'Monte-Carlo TV Festival 2007 Nominado Golden Nymph Television Films - Best Performance by an Actor Persuasión', 'Screen Actors Guild Awards 2024 Nominado Actor Outstanding Performance by an Ensemble in a Comedy Series Ted Lasso Compartido con: Katy Wix · Bronson Webb · Hannah Waddingham · Juno Temple · Jeremy Swift · Jason Sudeikis · · · · · · · · · · · · ·', 'Online Film & Television Association 2021 Ganador OFTA Television Award Best Guest Actor in a Comedy Series Ted Lasso', 'International Online Cinema Awards (INOCA) 2021 Ganador INOCA TV Best Guest Actor in a Comedy Series Ted Lasso'])], [('Demetries Coad', 'NULL', ['Reparto', 'Reparto'], ['su papel en Buddy Cops; The Stakeout'], 'NULL'), ('Adam L. Flowers', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en Heat', 'The Kingdom y B'], 'NULL'), ('Emil Ghanem', 'NULL', ['Reparto', 'Reparto'], ['su papel en Buddy Cops; The Stakeout', 'The Cleanse, a Story of Lies'], 'NULL'), ('Evan Griffiths', 'NULL', ['Reparto', 'Reparto'], ['su papel en Buddy Cops; The Stakeout'], 'NULL'), ('James Hearron', 'NULL', ['Dirección de fotografía y cámara', 'Reparto', 'Efectos visuales'], ["su papel en Time's Up", 'Anthrophobia', 'Mind Cloud'], 'NULL'), ('Hemlok Knight', 'NULL', ['Reparto', 'Reparto'], ['su papel en Top Secret Crush, Bury the Hatchet y Buddy Cops; The Stakeout'], 'NULL'), ('Jordan Lundy', 'NULL', ['Cámara y electricidad', 'Dirección de fotografía y cámara', 'Dirección'], ['su papel en Apparitional', "Wrastlin'", 'Small Pond'], 'NULL'), ('Tris Marie', 'NULL', ['Reparto', 'Equipos adicionales', 'Maquillaje'], ['su papel en MEAD', 'El negocio del dolor', 'In Search of'], 'NULL'), ('Tyler Meyer', 'NULL', ['Reparto', 'Reparto'], ['su papel en Buddy Cops; The Stakeout'], 'NULL'), ('Raymond Roberson', 'NULL', ['Reparto', 'Guion', 'Producción'], ["su papel en Vincent's Vow", 'Buddy Cops; The Stakeout', 'Smile'], ["Sunny Side Up Film Festival 2021 Nominado Sunny Side Up Film Festival Award Best Supporting Actor (Feature) Vincent's Vow", "Flicks Film Festival 2020 Ganador December Award Best Supporting Actor / Feature Film Vincent's Vow", "Branson International Film Festival 2021 Ganador Festival Award Best Supporting Actor / Feature Film Vincent's Vow", 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona'])], [('Jordyn Curet', '2008', ['Reparto', 'Equipos adicionales', 'Equipo ejecutivo'], ['Star Wars: Visions', 'Economía doméstica', '9-1-1'], 'NULL'), ('Chelsea Easter', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Love Just Is'], 'NULL'), ('Nadine Ellis', 'NULL', ['Reparto', 'Reparto'], ['Hairspray', 'La jungla 4'], 'NULL'), ('Asia Johnson', 'NULL', ['Guion', 'Reparto', 'Música'], ['su papel en Love Just Is', 'Exemplary', 'Caged'], 'NULL'), ('Billy Jones', 'NULL', ['Reparto', 'Guion', 'Música'], ['su papel en A Reckless Phase', 'Children of the Sun', "A Midsummer Day's Dream"], 'NULL')], [('Brandon M. Weber', 'NULL', ['Reparto', 'Reparto'], ['su papel en Zeroes', 'The 80s', 'Sweet Tooth (II'], 'NULL'), ('Kristen Hardimon', 'NULL', ['Reparto', 'Reparto'], ['su papel en Two Lane Blacktop, Protect and Serve y Perfect'], 'NULL'), ('Joshua Norris', 'NULL', ['Reparto', 'Reparto'], ['su papel en Abiding', 'The E-Listers', 'Something Thirty'], 'NULL'), ('Drea Joana', 'NULL', ['Reparto', 'Guion y continuidad', 'Equipos adicionales'], 'NULL', 'NULL'), ('Jay Potts', 'NULL', ['Reparto', 'Reparto'], ['su papel en I Am that I Am', 'The 80s'], 'NULL'), ('James B. Potts', 'NULL', ['Reparto', 'Equipos adicionales', 'Dirección'], ['su papel en Outlaws of Sherwood', 'Megalópolis', 'Shadows of the Ancient'], 'NULL')], [('Akshita Agnihotri', 'NULL', ['Reparto', 'Reparto'], 'NULL', 'NULL'), ('Prakash Dhotre', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sundara Manamadhe Bharli', 'Shiva', 'GAAYAM'], 'NULL'), ('Moses Fernandes', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en Baadshah', 'Daraar', 'Mawaali'], 'NULL'), ('Himmat Mav', 'NULL', ['Reparto', 'Reparto'], ['su papel en Bombay, Mawaali y 3 Storeys'], 'NULL'), ('Mehtab Mirza', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mawaali'], 'NULL'), ('Deepshikha Nagpal', '1977', ['Reparto', 'Guion', 'Producción'], ['Yeh Dooriyan', 'Red Swastik', 'Baadshah'], 'NULL'), ('Vaibhav Pagare', 'NULL', ['Reparto', 'Reparto'], ['su papel en San 1981', 'Mawaali y Bombay'], 'NULL'), ('Ganesh Pai', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mawaali, Bombay y Chicken Curry Law'], 'NULL'), ('Ashish Warang', 'NULL', ['Reparto', 'Reparto'], ['su papel en Drishyam', 'Sooryavanshi', 'Ek Villain Returns'], 'NULL')], [('Irma von Platen', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en El Círculo', 'Inland', 'Turpa Kiinni Minun Haters'], 'NULL'), ('Eva Melander', '1974', ['Reparto', 'Reparto'], ['Border', 'Flocken', 'Inland'], ['European Film Awards 2018 Nominado European Film Award European Actress Border', 'Guldbagge Awards 2021 Nominado Guldbagge Best Supporting Actress (Bästa kvinnliga biroll) Inland 2019 Ganador Guldbagge Best Actress (Bästa kvinnliga huvudroll) Border 2016 Ganador Guldbagge Best Supporting Actress (Bästa kvinnliga biroll) Flocken', 'Hamptons International Film Festival 2018 Ganador Golden Starfish Award - Honorable Mention Narrative Feature Border -Acting. Compartido con: Eero Milonoff', 'Chlotrudis Awards 2019 Nominado Chlotrudis Award Best Actress Border', 'Indiana Film Journalists Association, US 2018 Nominado IFJA Award Best Actress Border']), ('Albin Grenholm', 'NULL', ['Reparto', 'Reparto'], ['su papel en Midnattssol', 'Hijos de la nieve', 'Blinded'], 'NULL'), ('Ann Petrén', '1954', ['Reparto', 'Reparto'], ['Border', 'Om jag vänder mig om', 'Happy End'], ['Berlin International Film Festival 2004 Ganador Silver Berlin Bear Outstanding Artistic Achievement Om jag vänder mig om To the ensemble cast. Compartido con: Leif Andrée · Pernilla August · Jan Coster · Jakob Eklund · Ingvar Hirdwall · Magnus Krepper · · · · · · · ·', 'Festróia - Tróia International Film Festival 2004 Ganador Silver Dolphin Best Actress (Melhor Interpretação Feminia) Om jag vänder mig om', 'Guldbagge Awards 2012 Ganador Guldbagge Best Actress (Bästa kvinnliga huvudroll) Happy End 2005 Nominado Guldbagge Best Supporting Actress (Bästa kvinnliga biroll) Masjävlar 2004 Ganador Guldbagge Best Actress (Bästa kvinnliga huvudroll) Om jag vänder mig om', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Paloma Perez', 'NULL', ['Reparto', 'Reparto'], ['su papel en Inland'], 'NULL'), ('Tage Hervén', 'NULL', ['Reparto', 'Arte', 'Dirección'], ['su papel en Ägget', 'Kung Fury', 'Charter'], 'NULL'), ('Rasmus Winblad', 'NULL', ['Reparto', 'Reparto'], ['su papel en Flugparken', 'Inland'], 'NULL'), ('Simon Lindgren', '1999', ['Cámara y electricidad', 'Reparto', 'Cámara y electricidad'], ['Glaciär', 'Bullets', 'Rift'], 'NULL'), ('Lena P. Cederlund', 'NULL', ['Arte', 'Reparto', 'Dirección de arte'], ['su papel en Déjame entrar', 'Un cuento gamberro de Navidad', 'Inland'], 'NULL'), ('Daniel Eneris Sandström', 'NULL', ['Reparto', 'Reparto'], ['su papel en Inland', 'Den längsta dagen'], 'NULL')], [('Kent Allen', 'NULL', ['Reparto', 'Reparto'], ['su papel en How Father Lawrence Briefly Considered a Career in Landscaping', 'Out of the Can'], 'NULL')], [('Sharayu Mahale', 'NULL', ['Reparto', 'Producción', 'Arte'], ['su papel en Attaway General', 'Unmothered', 'Severance'], ['Jaipur International Film Festival 2023 Nominado Best Web Series Web Series Strings Attached Compartido con: Andy Wolf · Jessica Hutchinson · Marie Förster · Lucinda Bruce · Jack Armstrong · Julia Kemp · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'The Short Film Awards 2022 Nominado Best Web Series Best Web Series Strings Attached Compartido con: Andy Wolf · Jessica Hutchinson · Marie Förster · Lucinda Bruce · Jack Armstrong · Julia Kemp · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Seoul Web Fest 2022 Ganador Best Web Series Best Web Series Strings Attached Compartido con: Andy Wolf · Jessica Hutchinson · Marie Förster · Lucinda Bruce · Jack Armstrong · Julia Kemp · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Cult Critic Movie Awards 2020 Ganador Outstanding Achievement Award Film on Women Fi aur Me Compartido con: Ria Patel · Chris Leary', 'Indie Short Fest 2022 Nominado June Award Best Web and New Media Strings Attached Compartido con: Andy Wolf · Jessica Hutchinson · Marie Förster · Lucinda Bruce · Jack Armstrong · Julia Kemp · · · · · · · · · · · · · · · · · · · · · · · · · ·']), ('Braxton Michael', 'NULL', ['Reparto', 'Dirección', 'Reparto'], ['su papel en Swamp', 'The Comedy News Hour', 'Oh Brother Felch Films'], 'NULL')], [('Jakub Jablonský', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Male Gaze: Hide and Seek', 'Kdyz draka boli hlava', 'If Only You Were Mine'], 'NULL'), ('Peter Martincek', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Male Gaze: Hide and Seek', 'Lajna', 'If Only You Were Mine'], 'NULL'), ('Judita Hansman', '1967', ['Reparto', 'Reparto'], ['Tajne zivoty', 'La profesora', 'We Have Never Been Modern'], 'NULL'), ('Claudia Pittner Konecna', 'NULL', ['Equipos adicionales', 'Dirección de la segunda unidad o asistencia a dirección', 'Reparto'], ['su papel en If Only You Were Mine', 'El informe Auschwitz', 'Mama na prenájom'], 'NULL')], [('Tadanobu Asano', '1973', ['Reparto', 'Dirección', 'Composición'], ['Mongol', 'Battleship', 'Thor: Ragnarok'], ['Hawaii International Film Festival 2015 Ganador Maverick Award', 'Hochi Film Awards 2000 Ganador Hochi Film Award Best Supporting Actor Gojô reisenki: GojoeGohatto', 'Awards of the Japanese Academy 2016 Nominado Award of the Japanese Academy Best Supporting Actor Nagasaki, recuerdos de mi hijo 2010 Nominado Award of the Japanese Academy Best Actor Tsurugidake: Ten no ki 2010 Nominado Award of the Japanese Academy Best Actor Viyon no tsuma 2009 Nominado Award of the Japanese Academy Best Supporting Actor Kâbê 2004 Nominado Award of the Japanese Academy Best Supporting Actor Zatoichi 1 más', 'Japanese Professional Movie Awards 1997 Ganador Japanese Professional Movie Award Best Actor HelplessFocus', 'Moscow International Film Festival 2014 Ganador Silver St. George Best Actor Watashi no otoko']), ('Kumiko Asô', '1978', ['Reparto', 'Banda sonora', 'Reparto'], ['Pulse (Kairo', 'Dr'], ['Hochi Film Awards 2007 Ganador Hochi Film Award Best Actress Yûnagi no machi sakura no kuni 1998 Ganador Hochi Film Award Best Supporting Actress Dr. Akagi', 'Awards of the Japanese Academy 2012 Nominado Award of the Japanese Academy Best Supporting Actress Moteki 1999 Ganador Award of the Japanese Academy Best Supporting Actress Dr. Akagi 1999 Ganador Newcomer of the Year Dr. Akagi', 'Japanese Professional Movie Awards 2002 Ganador Japanese Professional Movie Award Best Actress Pulse (Kairo)Zeitaku na hone', 'Mainichi Film Concours 2008 Ganador Mainichi Film Concours Best Actress Yûnagi no machi sakura no kuni', 'Blue Ribbon Awards 2008 Ganador Blue Ribbon Award Best Actress Yûnagi no machi sakura no kuni']), ('Toyotaka Hanazawa', '1964', ['Reparto', 'Reparto'], ['Shin Godzilla', '22-nenme no kokuhaku: Watashi ga satsujinhan desu', 'Gurasuhoppâ'], 'NULL'), ('Haru', 'NULL', ['Reparto', 'Reparto'], ['su papel en Asa ga kita', 'Kôhî ga samenai uchi ni', 'Riaru onigokko'], 'NULL'), ('Tôma Ikuta', '1984', ['Reparto', 'Reparto'], ['Ningen shikkaku', 'Hanamizuki', 'Nô Otoko'], 'NULL'), ('Renji Ishibashi', '1941', ['Reparto', 'Banda sonora', 'Reparto'], ['Rônin-gai', 'Audition', 'Ware ni utsu yoi ari'], ['Hochi Film Awards 1990 Ganador Hochi Film Award Best Supporting Actor Rônin-gaiWare ni utsu yoi ari', "Awards of the Japanese Academy 2011 Nominado Award of the Japanese Academy Best Supporting Actor Kondo wa aisaika 1991 Ganador Award of the Japanese Academy Best Supporting Actor Rônin-gaiWare ni utsu yoi ari 1982 Nominado Award of the Japanese Academy Best Supporting Actor Masho no natsu - 'Yotsuya kaidan' yoriKemono-tachi no atsui nemuri", 'Kinema Junpo Awards 1991 Ganador Kinema Junpo Award Best Supporting Actor Rônin-gaiWare ni utsu yoi ari', 'Mainichi Film Concours 1991 Ganador Mainichi Film Concours Best Supporting Actor Rônin-gaiWare ni utsu yoi ari', 'Yokohama Film Festival 1982 Ganador Festival Prize Best Supporting Actor Kemono-tachi no atsui nemuri']), ('Norihito Kaneko', '1978', ['Reparto', 'Reparto'], ['Taiho shichauzo', 'Gurasuhoppâ', 'Urutoraman Zetto'], 'NULL'), ('Jun Murakami', '1973', ['Reparto', 'Dirección', 'Guion'], ['Futei no kisetsu', 'Nabbie no koi', 'Kyûtî Hanî'], 'NULL'), ('Ozuno Nakamura', 'NULL', ['Reparto', 'Reparto'], ['su papel en Godzilla Minus One', 'Toshokan sensô', 'Eien no 0'], 'NULL'), ('Nanao', '1988', ['Reparto', 'Reparto'], ['Gintama', 'Gurasuhoppâ', 'Wotaku ni koi wa muzukashii'], 'NULL')], [('Tegan Farrelly', 'NULL', ['Reparto', 'Banda sonora', 'Reparto'], ['su papel en La diplomática', 'Wine and Roses (and Wine'], 'NULL'), ('Craig Andrew Robertson', 'NULL', ['Edición', 'Dirección', 'Guion'], ['su papel en The Thistle in the Kiss, Last House on CCTV', 'The Acolyte'], 'NULL'), ('Abbie McIntosh', 'NULL', ['Reparto', 'Reparto'], ['su papel en This Is My Blood', 'ST HOPE', 'Wine and Roses (and Wine'], 'NULL')], [('Johan M. Craft', 'NULL', ['Reparto', 'Reparto'], ['su papel en Monster School: Minecraft Animation'], 'NULL'), ('Pham Viet Dung', 'NULL', ['Reparto', 'Reparto'], ['su papel en Drama', 'Drama', 'Drama'], 'NULL'), ('Viet Pham', 'NULL', ['Equipos adicionales', 'Equipos adicionales'], ['su papel en Kinect Adventures!', 'Drama', 'Deuramaui Jewang'], 'NULL'), ('Eben Ezer Purba', 'NULL', ['Dirección', 'Reparto', 'Guion'], ['su papel en Monster School: Minecraft Animation'], 'NULL'), ('Randall Sanchez', 'NULL', ['Reparto', 'Reparto'], ['su papel en Monster School: Minecraft Animation', 'Alex and the Monsters', 'Alex and the Monsters'], 'NULL'), ('Teddy Squid', 'NULL', ['Reparto', 'Reparto'], ['su papel en Monster School: Minecraft Animation', 'Alex and the Monsters', 'Alex and the Monsters'], 'NULL'), ('Kiara Alexa T. Tadic', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Alex and the Monsters', 'Alex and the Monsters', 'Monster School: Minecraft Animation'], 'NULL'), ('Dung Pham Viet', 'NULL', ['Reparto', 'Reparto'], ['su papel en Drama Queen', 'The Coop', 'Drama'], 'NULL')], [], [('Michael Muhney', '1975', ['Reparto', 'Guion', 'Producción'], ['Veronica Mars', 'The Young and the Restless', 'Sidepiece'], ['Daytime Emmy Awards 2013 Nominado Daytime Emmy Outstanding Lead Actor in a Drama Series The Young and the Restless For playing "Adam Newman".', 'AltFF Alternative Film Festival 2020 Ganador Festival Prize Best Actor Sidepiece', 'X World Short Film Festival 2019 Nominado Best Actor Comedy Sidepiece 2019 Ganador Best Actor Best Comedy Actor Sidepiece', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Eddie Jones', '1934', ['Reparto', 'Reparto'], ['La terminal', 'Seabiscuit, más allá de la leyenda', 'Sneakers (Los fisgones'], 'NULL'), ('Pat Morita', '1932', ['Reparto', 'Guion', 'Producción'], ['Karate Kid (El momento de la verdad', 'Karate Kid III: El desafío final', 'Karate Kid II'], ['Academy Awards, USA 1985 Nominado Oscar Best Actor in a Supporting Role Karate Kid (El momento de la verdad)', 'Primetime Emmy Awards 1986 Nominado Primetime Emmy Outstanding Supporting Actor in a Miniseries or a Special Quizás en primavera For playing "Tommy Tanaka".', 'Golden Globes, USA 1986 Nominado Golden Globe Best Performance by an Actor in a Supporting Role in a Series, Miniseries or Motion Picture Made for Television Quizás en primavera 1985 Nominado Golden Globe Best Performance by an Actor in a Supporting Role in a Motion Picture Karate Kid (El momento de la verdad)', 'Razzie Awards 1990 Nominado Razzie Award Worst Supporting Actor Karate Kid III: El desafío final', 'Walk of Fame 1994 Ganador Star on the Walk of Fame Motion Picture On 4 August 1994. At 6633 Hollywood Blvd.']), ('Adrienne Frantz', '1978', ['Reparto', 'Sonido', 'Banda sonora'], ['Belleza y poder', 'The Young and the Restless', 'Justified: La ley de Raylan'], ['Daytime Emmy Awards 2003 Nominado Daytime Emmy Outstanding Younger Actress in a Drama Series Belleza y poder 2001 Ganador Daytime Emmy Outstanding Younger Actress in a Drama Series Belleza y poder 2000 Nominado Daytime Emmy Outstanding Younger Actress in a Drama Series Belleza y poder For playing "Amber Forrester".', 'Soap Opera Digest Awards 2001 Nominado Soap Opera Digest Award Outstanding Younger Lead Actress Belleza y poder 2000 Nominado Soap Opera Digest Award Outstanding Younger Lead Actress Belleza y poder', 'Young Artist Awards 2000 Nominado Young Artist Award Best Performance in a Soap Opera - Young Actress Belleza y poder 1999 Ganador Young Artist Award Best Performance in a Day Time Serial - Young Performer Belleza y poder', 'Online Film & Television Association 2001 Nominado OFTA Television Award Best Supporting Actress in a Daytime Serial Belleza y poder', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Frank Nicotero', '1969', ['Equipos adicionales', 'Reparto', 'Producción'], ['Tiny Commando', 'Tribute', 'Beyond Twisted'], 'NULL'), ('Amber Benson', '1977', ['Reparto', 'Guion', 'Dirección'], ['Chance', 'Veneno en la piel', 'Latter Days'], ['Sidewalk Film Festival 2002 Ganador Audience Choice Award Best Feature Chance', 'L.A. Outfest 2005 Ganador Grand Jury Award Outstanding Actress in a Feature Film Race You to the Bottom', 'Young Artist Awards 1994 Nominado Young Artist Award Best Youth Actress Co-Starring in a Motion Picture Drama El rey de la colina', 'Oldenburg Film Festival 2010 Nominado German Independence Award - Audience Award Drones Compartido con: Adam Busch', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Max Gail', '1943', ['Reparto', 'Dirección', 'Guion'], ['42: La verdadera historia de una leyenda del deporte', 'Vida y milagros del Capitán Miller', 'Hospital General'], ['Daytime Emmy Awards 2021 Ganador Daytime Emmy Outstanding Performance by a Supporting Actor in a Drama Series Hospital General 2019 Ganador Daytime Emmy Outstanding Supporting Actor in a Drama Series Hospital General For playing: "Mike Corbin".', 'Primetime Emmy Awards 1980 Nominado Primetime Emmy Outstanding Supporting Actor in a Comedy or Variety or Music Series Vida y milagros del Capitán Miller For playing "Stan "Wojo" Wojciehowicz". 1979 Nominado Primetime Emmy Outstanding Supporting Actor in a Comedy or Comedy-Variety or Music Series Vida y milagros del Capitán Miller For playing: "Stan Wojciehowicz".', 'Massachusetts Independent Film Festival 2014 Ganador Festival Prize Best Actor Feature The Frontier', 'Los Angeles Independent Film Festival Awards 2015 Nominado LAIFF October Award Best Actor In Memory', 'Soap Hub Award 2021 Nominado 2nd Annual Soap Hub Award Favorite General Hospital Actor Hospital General 2020 Nominado 1st Annual Soap Hub Award Favorite General Hospital Actor Hospital General']), ('Sierra Peters', 'NULL', ['Reparto', 'Reparto'], ['su papel en Disposable', 'Act Your Age', 'Crab Orchard'], 'NULL'), ('Morris Berenbaum', 'NULL', ['Reparto', 'Reparto'], ['su papel en Act Your Age'], 'NULL')], [('Saydee Dickinson', 'NULL', ['Reparto', 'Dirección de casting', 'Reparto'], ['su papel en Desde cero', 'Nate el grande', 'Fauces de la noche'], ['Paris Film Festival 2021 Nominado Best Actress The Carducci Talent Show', 'Couch Film Festival 2021 Nominado Best Actress 30:01 to 60:00 Minutes The Carducci Talent Show', 'Horrorhaus Film Festival 2021 Nominado Best Supporting Actress Best Supporting Actress The Carducci Talent Show', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Johnno Wilson', 'NULL', ['Reparto', 'Guion', 'Producción'], ['Me alegro por ti', 'Larry David (Curb Your Enthusiasm', 'El método Williams'], 'NULL'), ('Sara Fletcher', 'NULL', ['Reparto', 'Producción', 'Guion'], 'NULL', ['Savannah Film Festival 2015 Nominado Best Historical Short Historical Shorts The Girl in the Green Dress Compartido con: Johanna Saldaña · Leah McKendrick', 'River Bend Film Festival, US 2014 Ganador Best Short The Secret Keeper Compartido con: Bradley Fletcher · Bears Rebecca Fonté', 'Idyllwild International Festival of Cinema 2017 Ganador IIFC Award Best Supporting Actress - Feature Bad People', 'LA Web Fest 2013 Ganador Outstanding Achievement Award Best ensemble cast for horror/sci-fi/fantasy 8.13 Compartido con: Beau Ryan · Chris Dorman · Craig Frank · Traycee King · Flood Reed · Marisha Ray · Rebecca Denise · · ·', 'Horrorhaus Film Festival 2021 Ganador Best Supporting Actress Best Supporting Actress The Carducci Talent Show']), ('Anthony Fanelli', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Reasonable Doubt', 'S'], ['Key West Film Festival 2021 Nominado Jury Prize Best Comedy Short Film Last Chance Compartido con: Gina Comparetto · Lyndsi LaRose', 'Shockfest Film Festival, US 2021 Ganador Festival Award Best Short Film The Carducci Talent Show', 'Couch Film Festival 2021 Nominado Best Drama 30:01-60:00 Minutes The Carducci Talent Show 2021 Nominado Best Writer 30:01 to 60:00 Minutes The Carducci Talent Show 2021 Nominado Best Thriller 10:01-20:00 Minutes Juncture', 'LA Indie Film Fest 2019 Nominado Audience Choice Award Best Short Film Juncture 2019 Nominado Screenplay Competition Feature Screenplay Skokie Stakeout', 'Mile High Horror Film Festival 2021 Ganador Audience Award Audience Award for Best Short Film Last Chance']), ('Maya Nalli', '1991', ['Reparto', 'Guion', 'Producción'], ['La familia Upshaw', 'Dollface', 'Black-ish'], 'NULL'), ('Michael Strassner', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Young Rock', 'Modern Family', 'A Spy Movie'], 'NULL'), ('Arielle Fanelli', '1985', ['Reparto', 'Producción', 'Reparto'], ['Agente Carter', 'The Carducci Talent Show', 'Juncture'], 'NULL'), ('Andrew Searles', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Ghosts', 'Big Break', 'The Carducci Talent Show'], 'NULL'), ("Miranda O'Hare", 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Coven', 'Valorant', 'XCOM: Chimera Squad'], ['Los Angeles Film Awards 2018 Ganador LAFA January Award Best Duo Indigo Lake Compartido con: Andrew Cutcliffe', 'Oniros Film Awards 2018 Ganador February Monthly Edition Best Actress Indigo Lake', 'Hollywood Blood Horror Festival 2023 Ganador Jury Prize Best Feature Clown Fear Compartido con: Minh Collins · Asif Akbar · Sadie Katz · Augie Duke · Tiffani Fest · Nikki Kris · Courtney Lynn · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Lucy Blehar', 'NULL', ['Reparto', 'Reparto'], ['su papel en Monster Mash', 'Mary Tyler, Millennial', 'Chicago P'], 'NULL')], [('Kelly Hauenstein', 'NULL', ['Reparto', 'Reparto'], ['su papel en Bad Penny'], 'NULL'), ('Molly Tage', 'NULL', ['Reparto', 'Reparto'], ['su papel en Bad Penny'], 'NULL'), ('Paul Villabrille', 'NULL', ['Reparto', 'Reparto'], ['su papel en 28 Hours Later: The Zombie Movie', 'Lifeline', 'Bad Penny'], 'NULL')], [('Yô Ôizumi', '1973', ['Reparto', 'Guion', 'Dirección'], ['El recuerdo de Marnie', 'El viaje de Chihiro', 'El castillo ambulante'], ['Awards of the Japanese Academy 2024 Nominado Award of the Japanese Academy Best Supporting Actor Mom, Is That You?! 2023 Nominado Award of the Japanese Academy Best Actor Tsuki no michikake 2018 Nominado Award of the Japanese Academy Best Actor Tantei wa bar ni iru 3 2016 Nominado Award of the Japanese Academy Best Actor Kakekomi onna to kakedashi otoko 2012 Nominado Award of the Japanese Academy Best Actor Tantei wa bar ni iru', 'Blue Ribbon Awards 2016 Ganador Blue Ribbon Award Best Actor Kakekomi onna to kakedashi otoko', 'Toronto After Dark Film Festival 2018 Ganador Special Award Best Leading Actor I Am a Hero', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Kasumi Arimura', '1993', ['Reparto', 'Guion', 'Dirección'], ['Birigyaru', 'El recuerdo de Marnie', 'Sutorobo ejji'], ['Awards of the Japanese Academy 2023 Nominado Award of the Japanese Academy Best Supporting Actress Tsuki no michikake 2022 Ganador Award of the Japanese Academy Best Actress Hanatabamitai na koi o shita 2016 Nominado Award of the Japanese Academy Best Actress Birigyaru 2016 Ganador Award of the Japanese Academy Newcomer of Year Birigyaru', 'Blue Ribbon Awards 2016 Ganador Blue Ribbon Award Best Actress BirigyaruSutorobo ejji', 'Huading Award 2022 Nominado Huading Award Best Global Actress in a Motion Picture Hanatabamitai na koi o shita', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Masami Nagasawa', '1987', ['Reparto', 'Reparto'], ['Gintama', 'Mazâ', 'Sekai no chûshin de, ai o sakebu'], ["Hochi Film Awards 2020 Nominado Hochi Film Award Best Actress The Confidence Man JP: PrincessMazâ 2020 Ganador Readers' Choice Award Best Actress The Confidence Man JP: PrincessMazâ 2019 Ganador Hochi Film Award Best Actress Masukarêdo hoteru 2004 Ganador Hochi Film Award Best Supporting Actress Sekai no chûshin de, ai o sakebu", 'Awards of the Japanese Academy 2021 Ganador Award of the Japanese Academy Best Actress Mazâ 2021 Nominado Award of the Japanese Academy Best Actress The Confidence Man JP: Princess 2020 Ganador Award of the Japanese Academy Best Supporting Actress Kingudamu 2018 Nominado Award of the Japanese Academy Best Actress Sanpo suru shinryakusha 2016 Nominado Award of the Japanese Academy Best Supporting Actress Nuestra hermana pequeña 5 más', 'Mainichi Film Concours 2021 Nominado Mainichi Film Concours Best Actress MazâThe Confidence Man JP: Princess 2018 Ganador Mainichi Film Concours Best Actress Sanpo suru shinryakusha 2016 Ganador Mainichi Film Concours Best Supporting Actress Nuestra hermana pequeña', 'Blue Ribbon Awards 2021 Ganador Blue Ribbon Award Best Actress MazâThe Confidence Man JP: Princess 2020 Ganador Blue Ribbon Award Best Actress The Confidence Man JP: The Movie 2020 Nominado Blue Ribbon Award Best Supporting Actress KingudamuMasukarêdo hoteru 2012 Nominado Blue Ribbon Award Best Actress Gaku 2012 Ganador Blue Ribbon Award Best Supporting Actress Moteki 1 más', 'Yokohama Film Festival 2004 Ganador Festival Prize Best New Talent Ashura no gotokuRobokon']), ('Hisashi Yoshizawa', '1978', ['Reparto', 'Reparto'], ['Êsu o nerae!', 'Burû Hâtsu ga kikoeru', 'Watashi-tachi wa Doka Shiteiru'], 'NULL'), ('Yoshinori Okada', '1977', ['Reparto', 'Reparto'], ['Gendai ninkyoden', 'Oto-na-ri', 'Nonchan noriben'], 'NULL'), ('Nana Katase', '1981', ['Reparto', 'Banda sonora', 'Reparto'], ['Death Note: El último nombre', '20th century boys: Cap'], 'NULL'), ('Jin Katagiri', '1973', ['Reparto', 'Reparto'], ['Kamen raidâ Daburu', 'The Quiz Show', 'Kubo Ibuki'], 'NULL'), ('Makita Sports', '1970', ['Reparto', 'Reparto'], ['Outrage', 'Iribito - Ihôjin -', 'Sankaku Mado no Sotogawa wa Yoru'], 'NULL'), ('Muga Tsukaji', '1971', ['Reparto', 'Reparto'], ['Mamiya kyodai', 'Gekijôban Kamen Raidâ Doraibu: Sapuraizu fyûchâ', 'Ousama ni Sasagu Kusuriyubi'], ['Awards of the Japanese Academy 2007 Ganador Popularity Award Most Popular Actor Mamiya kyodai', 'Kinema Junpo Awards 2007 Ganador Kinema Junpo Award Best New Actor Mamiya kyodai', 'Mainichi Film Concours 2007 Ganador Sponichi Grand Prize New Talent Award Mamiya kyodai', 'Blue Ribbon Awards 2007 Ganador Blue Ribbon Award Best New Actor Mamiya kyodai', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Yû Tokui', '1959', ['Reparto', 'Reparto'], ['Dark Water', 'Carta de amor', 'Shall We Dance?'], 'NULL')], [('Evan Carrington', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Until the End', 'The Wages of Sin', 'The Nurse That Saw the Baby on the Highway'], 'NULL'), ('Davis Haymes', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rainy Carolina', 'A Lonely Conversation', 'Diner 57'], 'NULL'), ('Brady Lokey', 'NULL', ['Reparto', 'Reparto'], ['su papel en Boothe', 'A Lonely Conversation', 'Mare'], 'NULL'), ('Bethany Paulsen', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en The Space We Fill, Heroes of Edelstein: Grip of Darkness y Liminal Vortices'], 'NULL'), ('Calvin Pozsonyi', 'NULL', ['Reparto', 'Reparto'], ['su papel en A Lonely Conversation'], 'NULL'), ('Austin Smith', 'NULL', ['Edición', 'Efectos visuales', 'Dirección'], ['su papel en Cold Mind', 'Worm Radio', 'Breathe'], ['Top Indie Film Awards 2022 Ganador Jury Prize Best FX Launch 2022 Ganador Jury Prize Best Music Launch 2020 Ganador Summer Award Best Student Film Cold Mind Compartido con: Harrison Dove-Green · Rachel Lebak', 'Short to the Point International Film Festival 2020 Ganador July Award Best Drama Cold Mind Compartido con: Harrison Dove-Green · Rachel Lebak · Kent Corbin · Vince Williams', 'Eurasia International Monthly Film Festival 2020 Ganador August Award Best Film of the Month Cold Mind Compartido con: Harrison Dove-Green · Rachel Lebak · Kent Corbin · Vince Williams', 'South Film and Arts Academy Festival 2022 Ganador Short Film Award Best VFX Launch', 'Canadian Cinematography Awards 2020 Ganador August Award Best Student Director Cold Mind Compartido con: Harrison Dove-Green']), ('Dayton Willison', 'NULL', ['Reparto', 'Maquillaje', 'Reparto'], ['su papel en Serial', 'Clouds', 'Because It Pleases Me'], ['Oniros Film Awards 2021 Ganador July Monthly Edition Best Acting Duo Serial Compartido con: Robert T. Engle', "Indie Short Fest 2021 Nominado July Award Outstanding Achievement Award (Acting Ensemble) Serial Compartido con: Robert T. Engle · Terry Jernigan · Sam Huggin · Dave Hobbs · Kera O'Bryon", "IndieX Film Festival 2021 Ganador Best Supporting Actor Best Supporting Actor Serial 2021 Ganador Best Acting Ensemble Best Acting Ensemble Serial Compartido con: Robert T. Engle · Terry Jernigan · Sam Huggin · Dave Hobbs · Kera O'Bryon", 'Los Angeles Cinematography AWARDS (LACA) 2021 Ganador May Award Best Supporting Actor Serial', "Best Actor & Director Awards 2021 Ganador Platinum Award Best Duo Serial Compartido con: Robert T. Engle 2021 Ganador Platinum Award Best Ensemble Serial Compartido con: Robert T. Engle · Terry Jernigan · Sam Huggin · Dave Hobbs · Kera O'Bryon"])], [('Ram Pothineni', '1988', ['Reparto', 'Música', 'Reparto'], ['iSmart Shankar', 'Devadasu', 'Kandireega'], 'NULL'), ('Priya Anand', '1986', ['Reparto', 'Reparto'], ['English Vinglish', 'Leo', '180'], 'NULL'), ('Bindhu Madhavi', '1986', ['Reparto', 'Música', 'Banda sonora'], ['Tamiluku En Ondrai Aluthavum', 'Pakka', 'Veppam'], 'NULL'), ('Arjun Sarja', '1962', ['Reparto', 'Dirección', 'Guion'], ['Irumbu Thirai', 'Mudhalvan', 'Gentleman'], ['Filmfare Awards South 2000 Nominado Filmfare Award - Tamil Film Industry Best Actor Mudhalvan', 'South Indian International Movie Awards 2019 Nominado SIIMA - Tamil Best Performance by an Actor in a Negative Role Irumbu Thirai 2018 Nominado SIIMA - Telugu Best Performance by an Actor in a Negative Role Lie 2014 Nominado SIIMA - Tamil Best Performance by an Actor in a Negative Role Kadal 2014 Nominado SIIMA - Kannada Best Actor in a Supporting Role Attahaasa', 'TSR-TV9 National Film Awards 2017 Nominado TSR-TV9 National Film Awards Kannada Best Hero Nibunan For the Kannada Version "Vismaya (2017)"', 'Tamil Nadu State Film Awards, India 1999 Ganador Creative Award Best Actor Mudhalvan 1993 Ganador Creative Award Best Actor Gentleman', 'Norway Tamil Film Festival 2020 Ganador Tamilar Award Best Supporting Actor Hero']), ('Gracy Singh', '1980', ['Reparto', 'Música', 'Reparto'], ['Lagaan: Érase una vez en la India', 'Munna Bhai M'], ['Awards of the International Indian Film Academy 2002 Nominado Popular Award Best Actress in a Leading Role Lagaan: Érase una vez en la India 2002 Ganador Special Award Face of the Year 2002 Ganador Popular Award Best Debut Actress Lagaan: Érase una vez en la India', 'Zee Cine Awards 2002 Ganador Popular Award Best Debut - Female Lagaan: Érase una vez en la India', 'Bollywood Awards 2002 Nominado Bollywood Movie Award Best Debutant - Female Lagaan: Érase una vez en la India', "Sansui Viewer's Choice Awards 2002 Ganador Sansui Viewer's Choice Award Most Promising Debut Actress Lagaan: Érase una vez en la India", "People's Choice Awards, India 2001 Ganador People's Choice Award Best Female Debut Lagaan: Érase una vez en la India"]), ('Banerjee', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rangasthalam', 'Bharat Ane Nenu', 'Yaadon Ki Baaraat'], 'NULL'), ('Brahmaji', '1965', ['Reparto', 'Reparto'], ['Pushpa: The Rise - Part 1', 'Rangasthalam', 'Salaar'], 'NULL'), ('Brahmanandam', '1956', ['Reparto', 'Música', 'Banda sonora'], ['Dookudu', 'Race Gurram', 'Sarrainodu'], ['Filmfare Awards South 2013 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Dhenikaina Ready 2011 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Adhurs 2009 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Ready 2006 Nominado Filmfare Award - Telugu Film Industry Best Comedian Athadu', 'South Indian International Movie Awards 2017 Nominado SIIMA - Telugu Best Comic Actor Sarrainodu 2016 Nominado SIIMA - Telugu Best Comic Actor Bruce Lee: The Fighter 2015 Ganador SIIMA - Telugu Best Comic Actor Race Gurram 2014 Ganador SIIMA - Telugu Best Comic Actor Baadshah 2013 Nominado SIIMA - Telugu Best Comic Actor Julayi 1 más', 'Times of India Film Awards 2012 Ganador Hyderabad Times Film Awards Best Actor in a Comic Role Dookudu', 'CineMAA Awards 2012 Ganador CineMAA Award Best Comedian Dookudu 2011 Ganador CineMAA Award Best Comedian Adhurs 2010 Ganador CineMAA Award Best Comedian Konchem Ishtam Konchem Kashtam 2008 Ganador CineMAA Award Best Comedian Dhee 2003 Ganador CineMAA Award Best Comedian Manmadhudu', 'Guinness World Record Award 2010 Ganador Guinness World Record Most screen credits for a living actor']), ('Vajja Venkata Giridhar', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rangasthalam', 'Athadu', 'Nannaku Prematho'], 'NULL')], [('Heidi Lewandowski', 'NULL', ['Reparto', 'Guion', 'Producción'], ['Galveston', 'Plan de escape 3', 'Mi hermano ciego'], 'NULL'), ('Richard Cannon', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cleanland'], 'NULL'), ('Raphael Robinson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cleanland', 'Three Man Monte', 'Crime Stoppers Case Files: Chicago'], 'NULL'), ('Al LaFleur', 'NULL', ['Reparto', 'Producción', 'Guion'], ['The Night Passenger', 'With This Ring', 'Lisen'], 'NULL'), ('Denny Castiglione', 'NULL', ['Reparto', 'Producción', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en Decisión final', 'Cleanland', 'Project E'], 'NULL'), ('Iran Black', 'NULL', ['Reparto', 'Especialistas', 'Reparto'], ['su papel en Cleanland', 'Lamia: The Zombie Slayer'], 'NULL'), ('Kevin Wells', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cleanland', 'Underdogs'], 'NULL'), ('Kyle Adam', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cleanland', 'After Life', 'Amateurs'], 'NULL')], [('Dugan Sherbondy', 'NULL', ['Sonido', 'Reparto', 'Sonido'], ['su papel en Jake Muller Adventures', 'The Star of Christmas', 'An Easter Carol'], 'NULL'), ('Sky Ziskin', 'NULL', ['Reparto', 'Reparto'], ['su papel en La bestia', 'Jake Muller Adventures'], 'NULL'), ('Elisa Hatten', 'NULL', ['Reparto', 'Reparto'], ['su papel en Jake Muller Adventures'], 'NULL'), ('Micah Touchet', '1980', ['Sonido', 'Reparto', 'Producción'], ['Jake Muller Adventures', 'My 7 Grandmas y The Watch'], 'NULL'), ('Josh Bultman', 'NULL', ['Reparto', 'Reparto'], ['su papel en Jake Muller Adventures', 'The Schoolhouse Rocked Podcast'], 'NULL'), ('Christian Gill', 'NULL', ['Reparto', 'Especialistas', 'Reparto'], ['su papel en Alias', 'Jake Muller Adventures', 'The Unit'], 'NULL'), ('Randy Streu', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Jake Muller Adventures', 'Homicide at Heavensgate', 'The Watch'], 'NULL'), ('Phil Lollar', 'NULL', ['Reparto', 'Guion', 'Sonido'], ['su papel en 3-2-1 Penguins!', 'The Jungle Book', '3-2-1 Penguins: The Doom Funnel Rescue!'], 'NULL'), ('Steve Schleicher', 'NULL', ['Reparto', 'Reparto'], ['su papel en Jake Muller Adventures'], 'NULL'), ('Aimee Lilly', 'NULL', ['Reparto', 'Reparto'], ['su papel en Jake Muller Adventures'], 'NULL')], [('Arturo Adame', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Neighborhood Groove', 'Run3', 'Requiem Ondevilla'], 'NULL'), ('Mitch Alcala', 'NULL', ['Reparto', 'Guion', 'Casting'], ['su papel en Mission: Irreparable', 'Mission: Imposter', 'Unbelievable!!!!!'], 'NULL'), ('Brian Allen', 'NULL', ['Maquillaje', 'Reparto', 'Maquillaje'], ['su papel en Proximity', 'Boom Bar', 'Smoking Hot Vixens'], 'NULL'), ('Andrew Anderson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Boom Bar'], 'NULL'), ('Grigoriy Batiyenko', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en Chosen Warriors', 'Boom Bar', 'The Assassin Exit'], 'NULL'), ('April Bay', 'NULL', ['Producción', 'Dirección', 'Guion'], ['su papel en Mission: Imposter', 'Four Square', 'Love Bots'], 'NULL'), ('Angela Dela Cruz', 'NULL', ['Reparto', 'Arte', 'Reparto'], ['su papel en Chosen Warriors', 'Boom Bar', 'The Virtual'], 'NULL'), ('Stephen Dela Cruz', 'NULL', ['Reparto', 'Cámara y electricidad', 'Reparto'], ['su papel en The Hat: Viral Spots', 'Game Night', 'Boom Bar'], 'NULL'), ('Reuben Duran', 'NULL', ['Reparto', 'Reparto'], ['su papel en Boom Bar', 'Gregory Way TV', 'UFC on Fox'], 'NULL'), ('John Freeman', 'NULL', ['Dirección de fotografía y cámara', 'Cámara y electricidad', 'Dirección'], ['su papel en Capture', 'Immortal Game', 'A Chateau in the Loire'], ['San Diego Film Awards, US 2022 Nominado San Diego Film Award Best Visual Effects Immortal Game 2022 Nominado San Diego Film Award Best Cinematography Immortal Game 2022 Nominado San Diego Film Award Best Narrative Short Film Immortal Game 2022 Nominado San Diego Film Award Best Musical Composition Immortal Game 2019 Nominado San Diego Film Award Best Cinematography: Narrative Short Film Capture', 'Idyllwild International Festival of Cinema 2021 Nominado The Juan Ruiz Anchia Award Best Cinematography A Castle in Umbria', "The 48 Hour Film Project, San Diego 2017 Nominado Jury Prize Best Directing That's Final", 'Horrible Imaginings Film Festival 2020 Ganador HIFF Award Best Home Grown Local Film Capture Compartido con: Karenssa LeGear', 'The Monkey Bread Tree Film Awards 2019 Nominado Summer Award Best Cinematography Dollhead'])], [('Matt Damon', '1970', ['Producción', 'Reparto', 'Guion'], ['El indomable Will Hunting', 'Marte (The Martian', 'Infiltrados'], ['Academy Awards, USA 2017 Nominado Oscar Best Motion Picture of the Year Manchester frente al mar Compartido con: Lauren Beck · Kimberly Steward · Chris Moore · Kevin J. Walsh 2016 Nominado Oscar Best Performance by an Actor in a Leading Role Marte (The Martian) 2010 Nominado Oscar Best Performance by an Actor in a Supporting Role Invictus 1998 Nominado Oscar Best Actor in a Leading Role El indomable Will Hunting 1998 Ganador Oscar Best Writing, Screenplay Written Directly for the Screen El indomable Will Hunting Compartido con: Ben Affleck', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2016 Nominado Saturn Award Best Actor Marte (The Martian) 2005 Nominado Saturn Award Best Actor El mito de Bourne', "Berlin International Film Festival 2023 Nominado Berlinale Documentary Award Kiss the Future Compartido con: Nenad Cicin-Sain · Ben Affleck · Sarah Anthony · Ned O'Hanlon 2007 Ganador Silver Berlin Bear Outstanding Artistic Contribution El buen pastor Compartido con: John Sessions · Oleg Stefan · Martina Gedeck · Michael Gambon · John Turturro · Keir Dullea · · · · · · · · · · · · 1998 Ganador Silver Berlin Bear Outstanding Single Achievement El indomable Will Hunting For his screenwriting and acting in the film.", 'Blockbuster Entertainment Awards 2001 Nominado Blockbuster Entertainment Award Favorite Actor - Drama/Romance Todos los caballos bellos 2000 Nominado Blockbuster Entertainment Award Favorite Actor - Suspense El talento de Mr. Ripley 1999 Ganador Blockbuster Entertainment Award Favorite Actor - Video El indomable Will Hunting 1998 Nominado Blockbuster Entertainment Award Favorite Actor - Drama Legítima defensa, de John Grisham', 'Boston Society of Film Critics Awards 2006 Nominado BSFC Award Best Ensemble Cast Infiltrados Compartido con: Mark Wahlberg · Vera Farmiga · Jack Nicholson · Anthony Anderson · Leonardo DiCaprio · Alec Baldwin · · 1999 Nominado BSFC Award Best Actor El talento de Mr. Ripley Tied with Russell Crowe for El dilema (The Insider) (1999) and Kevin Spacey for American Beauty (1999). 1997 Nominado BSFC Award Best Screenplay El indomable Will Hunting Compartido con: Ben Affleck']), ('Tian Jing', '1988', ['Reparto', 'Reparto'], ['Kong: La Isla Calavera', 'La Gran Muralla', 'Pacific Rim: Insurrección'], ['Huading Award 2022 Nominado Huading Award Best Performance by an Actress in Contemporary Television Series Liu guang zhi cheng', 'Domestic TV series Ceremony, China 2017 Ganador Jury Award Popular Young Actress Da Tang rong yao 2015 Nominado Jury Award Best Female Character The Legend of Xiao Zhuang', 'Tencent Video TV And Movie Award 2023 Ganador Jury Award Glamorous Artist of the Year 2023 Ganador Jury Award VIP Star of the Year', 'Weibo Awards Ceremony 2024 Ganador Weibo Award Shining Actor of the Year Le You Yuan 2023 Ganador Weibo TV & Internet Video Summit Award Best Charming Actor of the Year Le You YuanZhuo zhuo feng liu 2018 Ganador Weibo Award Popular Artist of the Year', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Willem Dafoe', '1955', ['Reparto', 'Producción', 'Guion'], ['Spider-Man', 'Van Gogh, a las puertas de la eternidad', 'The Florida Project'], ['Academy Awards, USA 2019 Nominado Oscar Best Performance by an Actor in a Leading Role Van Gogh, a las puertas de la eternidad 2018 Nominado Oscar Best Performance by an Actor in a Supporting Role The Florida Project 2001 Nominado Oscar Best Actor in a Supporting Role La sombra del vampiro 1987 Nominado Oscar Best Actor in a Supporting Role Platoon', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2001 Ganador Saturn Award Best Supporting Actor La sombra del vampiro', 'Berlin International Film Festival 2018 Ganador Honorary Golden Berlin Bear', 'Bodil Awards 2010 Ganador Bodil Best Actor (Bedste mandlige hovedrolle) Anticristo', 'Boston Society of Film Critics Awards 2017 Ganador BSFC Award Best Supporting Actor The Florida Project 2004 Nominado BSFC Award Best Ensemble Cast Life Aquatic Compartido con: Cate Blanchett · Bud Cort · Michael Gambon · Jeff Goldblum · Anjelica Huston · Bill Murray · ·']), ('Andy Lau', '1961', ['Reparto', 'Producción', 'Arte'], ['Juego sucio', 'Juego sucio 3', 'La casa de las dagas voladoras'], ['Asian American International Film Festival 2000 Ganador Asian Media Award', 'Sitges - Catalonian International Film Festival 2013 Ganador Maria Best Actor Maang taam', 'Golden Bauhinia Awards 2007 Nominado Golden Bauhinia Best Actor Mo gong 2006 Nominado Golden Bauhinia Best Actor Tung mung kei yuen 2005 Nominado Golden Bauhinia Best Actor Tian xia wu zei 2004 Nominado Golden Bauhinia Best Actor Juego sucio 3 2004 Nominado Golden Bauhinia Best Actor Dai zek lo 4 más', 'Golden Horse Film Festival 2011 Ganador Golden Horse Award Best Leading Actor Una vida sencilla 2004 Ganador Golden Horse Award Best Leading Actor Juego sucio 3 2003 Nominado Golden Horse Award Best Leading Actor Juego sucio 2001 Nominado Golden Horse Award Best Leading Actor Sau sun nam lui 1999 Nominado Golden Horse Award Best Original Film Song Hui nin yin fa dak bit doh For the song "The Longest Summer" Compartido con: Wah-Chuen Lam', 'Golden Rooster Awards 2015 Nominado Golden Rooster Best Actor Shi gu']), ('Pedro Pascal', '1975', ['Reparto', 'Producción', 'Banda sonora'], ['The Last of Us', 'The Mandalorian', 'Juego de tronos'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2024 Nominado Saturn Award Best Actor in a Television Series The Last of Us', 'Critics Choice Awards 2024 Nominado Critics Choice Award Best Actor in a Drama Series The Last of Us', 'Primetime Emmy Awards 2023 Nominado Primetime Emmy Outstanding Lead Actor in a Drama Series The Last of Us 2023 Nominado Primetime Emmy Outstanding Guest Actor in a Comedy Series Saturday Night Live For "Host: Pedro Pascal" 2023 Nominado Primetime Emmy Outstanding Narrator Patagonia: Life on the Edge of the World For "Mountains"', 'Golden Globes, USA 2024 Nominado Golden Globe Best Performance by a Male Actor in a Television Series, Drama The Last of Us', 'Satellite Awards 2024 Nominado Satellite Award Best Actor in a Series, Drama or Genre The Last of Us']), ('Hanyu Zhang', '1964', ['Reparto', 'Reparto'], ['Mei Gong he xing dong', 'The Captain', 'Héroes de guerra'], ['Golden Horse Film Festival 2009 Nominado Golden Horse Award Best Supporting Actor La ecuación del amor y la muerte 2008 Ganador Golden Horse Award Best Leading Actor Héroes de guerra', 'Golden Rooster Awards 2015 Ganador Golden Rooster Best Actor La conquista de la Montaña del Tigre 2009 Nominado Golden Rooster Best Actor Héroes de guerra', 'Hong Kong Film Critics Society Awards 2016 Nominado HKFCS Award Best Actor La conquista de la Montaña del Tigre', 'Hundred Flowers Awards 2020 Nominado Hundred Flowers Award Best Actor The Captain 2018 Nominado Hundred Flowers Award Best Actor Mei Gong he xing dong 2018 Nominado Hundred Flowers Award Best Supporting Actor Hong hai xing dong 2008 Ganador Hundred Flowers Award Best Actor Héroes de guerra', 'Huabiao Film Awards 2020 Nominado Huabiao Film Award Outstanding Actor The Captain 2016 Nominado Huabiao Film Award Outstanding Actor La conquista de la Montaña del Tigre 2009 Ganador Huabiao Film Award Outstanding Actor Héroes de guerra']), ('Han Lu', '1990', ['Reparto', 'Banda sonora', 'Reparto'], ['Chong fan 20 sui', 'La Gran Muralla', 'Shang hai bao lei'], ['Shanghai International Film Festival 2015 Nominado China Movie Channel Media Award Best Male Newcomer Chong fan 20 sui', "Beijing Student Film Festival 2015 Nominado Jury Award Best Newcomer Chong fan 20 sui 2015 Ganador Students' Choice Award Favorite Actor Chong fan 20 sui", 'Chinese Film Media Awards 2016 Nominado Audience Award Favorite Performance Chong fan 20 sui 2016 Nominado Audience Award Favorite Actor Chong fan 20 sui', 'Huading Award 2016 Nominado Huading Award Best New Performer Chong fan 20 sui', "Guangzhou Student Film Festival 2015 Ganador Students' Choice Award Favorite Actor Chong fan 20 sui"]), ('Kenny Lin', '1988', ['Reparto', 'Reparto'], ['Tong zhuo de ni', 'La Gran Muralla', 'Princess Agents'], ['Hong Kong Film Awards 2014 Nominado Hong Kong Film Award Best New Performer El joven detective Dee. El poder del dragón marino', 'Macau International Movie Festival 2018 Nominado Golden Lotus Awards Best Supporting Actor Detective Dee: The Four Heavenly Kings', 'Chinese Film Media Awards 2015 Nominado Audience Award Favorite Actor Tong zhuo de ni 2014 Nominado Audience Award Favorite Performance El joven detective Dee. El poder del dragón marino', 'Huading Award 2015 Nominado Huading Award Best Performance by an Actor in a Supporting Role in a Motion Picture La conquista de la Montaña del Tigre 2014 Nominado Huading Award Best Supporting Actor in a Motion Picuture El joven detective Dee. El poder del dragón marino', "Guangzhou Student Film Festival 2018 Nominado Students' Choice Award Favorite Supporting Actor Detective Dee: The Four Heavenly Kings 2014 Ganador Students' Choice Award Favorite Actor Tong zhuo de ni"]), ('Eddie Peng', '1982', ['Reparto', 'Guion', 'Banda sonora'], ['Mei Gong he xing dong', 'Cim hang', 'Xie bu ya zheng'], ['Golden Horse Film Festival 2013 Nominado Golden Horse Award Best Supporting Actor Ji zhan 2011 Nominado Golden Horse Award Best Leading Actor Fan gun ba! A Xin 2007 Nominado Golden Horse Award Best New Performer Ji yin jue ding wo ai ni', 'Hong Kong Film Awards 2015 Nominado Hong Kong Film Award Best Actor El renacer de una leyenda 2014 Nominado Hong Kong Film Award Best Supporting Actor Ji zhan', 'Hundred Flowers Awards 2014 Nominado Hundred Flowers Award Best New Performer Cold War', 'Changchun Film Festival 2018 Nominado Golden Deer Best Actor Ming yue ji shi you 2014 Nominado Golden Deer Best Supporting Actor Ji zhan 2012 Nominado Golden Deer Best Actor Fan gun ba! A Xin', 'Asian Film Awards 2018 Nominado Asian Film Award Best Supporting Actor Ming yue ji shi you']), ('Xuan Huang', '1985', ['Reparto', 'Producción', 'Reparto'], ['Tui na', 'Fang hua', 'Jiu zhou: Hai shang mu yun ji'], ['Hong Kong International Film Festival 2021 Ganador Golden Firebird Award Best Actor-Chinese Wu Hai', 'Las Palmas Film Festival 2015 Ganador Best Actor Tui na', 'Shanghai International TV Festival 2021 Nominado Magnolia Award Best Performance by an Actor in a Television Series Shan hai qing 2015 Nominado Magnolia Award Best Performance by an Actor in a Supporting Role in a Television Series Hong gao liang', 'Golden Eagle Awards, China 2022 Nominado Golden Eagle Best Actor in a Leading Role Shan hai qing', 'Beijing Student Film Festival 2015 Nominado Jury Award Best Actor Tui na'])], [('Stylist B.', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dirty Movie', 'NCIS: Nueva Orleans', 'Silence Is Compliance'], 'NULL'), ('John Charles II', 'NULL', 'NULL', ['su papel en Black and Blue', 'Homicide City: Charlotte', 'Silence Is Compliance'], ['Imágenes 14', '', '', '', ''])], [('Menetie T. Ejeye', 'NULL', ['Producción', 'Reparto', 'Guion'], ['su papel en In the Cage', 'Dead Power', 'Relentless Love'], 'NULL'), ('Tamara Goodwin', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Finding Kismet', 'In the Cage', 'Tosh'], 'NULL'), ('Andrew J McGuinness', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Captain Battle: Legacy War', 'In the Design of Madness y In the Cage'], 'NULL'), ('Elsi Eng', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en True Blood', 'The 10-50', 'Sea to Sky'], 'NULL'), ('Maryl De Milo', 'NULL', ['Reparto', 'Reparto'], ['su papel en In the Cage'], 'NULL'), ('Lap T. Nguyen', 'NULL', ['Reparto', 'Reparto'], ['su papel en In the Cage'], 'NULL'), ('Ninja N. Devoe', 'NULL', ['Reparto', 'Reparto'], ['su papel en Green Book', 'Luke Cage', 'El Cuartel Secreto'], 'NULL'), ('Blair Sharné', 'NULL', ['Reparto', 'Reparto'], ['su papel en In the Cage', 'Crackula Goes to Hollywood', 'Women of Heart Baptist Church'], 'NULL'), ('Eiji Inoue', 'NULL', ['Reparto', 'Dirección de casting', 'Especialistas'], ['su papel en The Terror', 'Click', 'El último samurái'], 'NULL'), ('Mantha Balourdou', 'NULL', ['Reparto', 'Producción', 'Equipos adicionales'], ['su papel en Slink', 'In the Cage', 'Midnight Sex Run'], 'NULL')], [('Kate Beckinsale', '1973', ['Reparto', 'Producción', 'Equipos adicionales'], ['Underworld: Evolution', 'Underworld', 'Van Helsing'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2004 Nominado Saturn Award Best Actress Underworld 2002 Nominado Saturn Award Best Actress Serendipity', 'Critics Choice Awards 2016 Nominado Critics Choice Award Best Actress in a Comedy Amor y amistad 2009 Nominado Critics Choice Award Best Actress Nada más que la verdad', 'Sitges - Catalonian International Film Festival 1997 Ganador Gran Angular Award Best Actress Como pez en el agua', 'Costume Designers Guild Awards 2012 Ganador Spotlight Award', 'Evening Standard British Film Awards 2017 Ganador Evening Standard British Film Award Best Actress Amor y amistad']), ('Theo James', '1984', ['Reparto', 'Producción', 'Equipo ejecutivo'], ['Divergente', 'La serie Divergente: Insurgente', 'La serie Divergente: Leal'], ['Primetime Emmy Awards 2023 Nominado Primetime Emmy Outstanding Supporting Actor in a Drama Series The White Lotus', 'Film Independent Spirit Awards 2023 Nominado Independent Spirit Award Best New Non-Scripted or Documentary Series Pepsi, ¿dónde está mi avión? Compartido con: Nick Boak · Andrew Renzi · Andrew Corkin · Andrew Fried · Jordan Wynn · Dane Lillegard · Sarina Roma ·', "People's Choice Awards, USA 2015 Ganador People's Choice Award Favorite Movie Duo Divergente Compartido con: Shailene Woodley", 'Screen Actors Guild Awards 2023 Ganador Actor Outstanding Performance by an Ensemble in a Drama Series The White Lotus Compartido con: F. Murray Abraham · Paolo Camilli · Jennifer Coolidge · Adam DiMarco · Meghann Fahy · Federico Ferrante · · · · · · · · · · · · · ·', 'Teen Choice Awards 2016 Nominado Teen Choice Award Choice Movie Actor: Action La serie Divergente: Leal 2016 Nominado Teen Choice Award Choice Movie: Chemistry La serie Divergente: Leal Compartido con: Shailene Woodley 2016 Nominado Teen Choice Award Choice Movie: Liplock La serie Divergente: Leal Compartido con: Shailene Woodley 2015 Nominado Teen Choice Award Choice Movie Actor: Action La serie Divergente: Insurgente 2015 Ganador Teen Choice Award Choice Movie: Liplock La serie Divergente: Insurgente Compartido con: Shailene Woodley 2 más']), ('Tobias Menzies', '1974', ['Reparto', 'Guion', 'Banda sonora'], ['Manhunt', 'You Hurt My Feelings', 'The Crown'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2015 Nominado Saturn Award Best Actor on Television Outlander', 'BAFTA Awards 2021 Nominado BAFTA TV Award Supporting Actor The Crown', 'Critics Choice Awards 2021 Nominado Critics Choice Award Best Supporting Actor in a Drama Series The Crown 2020 Nominado Critics Choice Award Best Actor in a Drama Series The Crown', 'Primetime Emmy Awards 2021 Ganador Primetime Emmy Outstanding Supporting Actor in a Drama Series The Crown For playing: "Prince Philip, Duke of Edinburgh".', 'Golden Globes, USA 2020 Nominado Golden Globe Best Performance by an Actor in a Television Series - Drama The Crown 2016 Nominado Golden Globe Best Performance by an Actor in a Supporting Role in a Series, Limited Series or Motion Picture Made for Television Outlander']), ('Lara Pulver', '1980', ['Reparto', 'Banda sonora', 'Reparto'], ['Maternal', 'Sherlock', 'The Split'], ["Critics Choice Television Awards 2012 Nominado Critics' Choice TV Award Best Actress in a Movie/Miniseries Sherlock", 'Online Film & Television Association 2012 Nominado OFTA Television Award Best Supporting Actress in a Motion Picture or Miniseries Sherlock', 'Gold Derby Awards 2012 Nominado Gold Derby TV Award TV Movie/Mini Supporting Actress A Scandal in Belgravia for "Sherlock: A Scandal in Belgravia"', 'National Film Awards, UK 2016 Nominado National Film Award Best Supporting Actress A Patch of Fog', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Charles Dance', '1946', ['Reparto', 'Producción', 'Dirección'], ['Gosford Park', 'The Imitation Game (Descifrando Enigma', 'Juego de tronos'], ['BAFTA Awards 1985 Nominado BAFTA TV Award Best Actor La joya de la corona', "International Cinematographers' Film Festival Manaki Brothers 2006 Ganador Special Golden Camera 300", 'Critics Choice Awards 2002 Ganador Critics Choice Award Best Acting Ensemble Gosford Park Compartido con: Eileen Atkins · Bob Balaban · Alan Bates · Stephen Fry · Michael Gambon · Richard E. Grant · · · · · · · · · · · · ·', 'Broadcasting Press Guild Awards 2006 Ganador Broadcasting Press Guild Award Best Actor Casa desolada', 'Sitges - Catalonian International Film Festival 2013 Ganador Time-Machine Honorary Award']), ('James Faulkner', '1948', ['Reparto', 'Producción', 'Reparto'], ['Pablo, el apóstol de Cristo', 'Downton Abbey', 'Juego de tronos'], 'NULL'), ('Peter Andersson', '1953', ['Reparto', 'Reparto'], ['Millennium 2: La chica que soñaba con una cerilla y un bidón de gasolina', 'Millennium 1: Los hombres que no amaban a las mujeres', 'Underworld: Guerras de sangre'], 'NULL'), ('Clementine Nicholson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Underworld: Guerras de sangre', 'Knightfall'], 'NULL'), ('Bradley James', '1983', ['Reparto', 'Reparto'], ['The Liberator', 'Damien', 'Merlín'], 'NULL'), ('Daisy Head', '1991', ['Reparto', 'Reparto'], ['Dungeons & Dragons: Honor entre ladrones', 'Underworld: Guerras de sangre', 'Fallen'], 'NULL')], [('Tamara Casellas', 'NULL', ['Reparto', 'Casting', 'Reparto'], ['su papel en Ama', 'El hijo zurdo', 'El perro de un torero'], ['Cinema Writers Circle Awards, Spain 2022 Nominado CEC Award Best New Actress (Mejor Actriz Revelación) Ama', 'Málaga Spanish Film Festival 2021 Ganador Silver Biznaga Best Actress (Mejor Actriz) Ama', 'Sant Jordi Awards 2022 Ganador Sant Jordi Best Actress in a Spanish Film (Millor actriu en pel·lícula espanyola) Ama', 'Almería International Film Festival 2021 Ganador Torrecárdenas Award Best Actress Ama', 'Feroz Awards, ES 2024 Nominado Feroz Award Best Supporting Actress in a Series El hijo zurdo 2022 Nominado Feroz Award Best Actress in a Leading Role Ama']), ('Leire Marín Vara', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ama', 'Machos Alfa'], 'NULL'), ('Estefanía de los Santos', '1975', ['Reparto', 'Reparto'], ['La mesita del comedor', 'Jaulas', 'Grupo 7'], ['Cinema Writers Circle Awards, Spain 2013 Nominado CEC Award Best New Actress (Mejor Actriz Revelación) Grupo 7', 'Goya Awards 2013 Nominado Goya Best New Actress (Mejor Actriz Revelación) Grupo 7', 'A Night of Horror International Film Festival 2024 Ganador Festival Prize Best Performance La mesita del comedor', 'ASECAN 2021 Nominado ASECAN Award Best Female Performance Sevillanas de Brooklyn 2019 Ganador ASECAN Award Best Lead Actress Jaulas 2013 Nominado Antoñita Colomé Award Best Female Performance Grupo 7', 'Premios Pávez - Festival Nacional de Cortometrajes Talavera de la Reina 2023 Ganador Premio Pávez Best Supporting Actress Madreselva']), ('Manuel de Blas', '1941', ['Reparto', 'Reparto'], ['Uncharted', 'Los fantasmas de Goya', 'Cinzento e Negro'], 'NULL'), ('Ana Turpin', '1978', ['Reparto', 'Reparto'], ['Uma', 'Todos los nombres de Dios', 'Amar es para siempre'], 'NULL'), ('Pablo Gómez-Pando', '1983', ['Reparto', 'Reparto'], ['Adiós', 'Ama', 'Feria: La luz más oscura'], 'NULL'), ('Chema del Barco', 'NULL', ['Reparto', 'Reparto'], ['su papel en El plan', 'El Niño', 'Cuéntame cómo pasó'], ['Cinema Writers Circle Awards, Spain 2021 Ganador CEC Award Best New Actor (Mejor Actor Revelación) El plan', 'Goya Awards 2021 Nominado Goya Best New Actor (Mejor Actor Revelación) El plan', 'Feroz Awards, ES 2021 Nominado Feroz Award Best Actor in a Supporting Role El plan', 'ASECAN 2021 Ganador ASECAN Award Best Lead Actor El plan', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Marina Marrero', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ama'], 'NULL'), ('Carmen Ibeas', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ama', 'Estándar', 'Viejos'], 'NULL'), ('Erin Gallego', 'NULL', ['Reparto', 'Reparto'], ['su papel en La Odisea de Vasi', 'Ama'], 'NULL')], [('Yura Sugiura', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gekijo-ban Porisu × Senshi rabupatorina! ~ Kaito kara no chosen! Rabu de papatto taiho seyo!~', 'Police x Heroine Lovepatrina!'], 'NULL'), ('Miyu Watanabe', '2007', ['Reparto', 'Reparto'], ['Gekijo-ban Porisu × Senshi rabupatorina! ~ Kaito kara no chosen! Rabu de papatto taiho seyo!~', 'Police x Heroine Lovepatrina!'], 'NULL'), ('Rina Yamaguchi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gekijo-ban Porisu × Senshi rabupatorina! ~ Kaito kara no chosen! Rabu de papatto taiho seyo!~', 'Secret × Heroine Phantomirage!', 'Police x Heroine Lovepatrina!'], 'NULL'), ('Yui Yamashita', 'NULL', ['Reparto', 'Reparto'], ['su papel en Gekijo-ban Porisu × Senshi rabupatorina! ~ Kaito kara no chosen! Rabu de papatto taiho seyo!~', 'Police x Heroine Lovepatrina!'], 'NULL')], [('Upendra', '1967', ['Reparto', 'Música', 'Guion'], ['Upendra', 'Uppi 2', 'Super'], 'NULL'), ('Pranitha Subhash', '1992', ['Reparto', 'Reparto'], ['Brahma', 'Massu Engira Masilamani', 'Bhuj: The Pride of India'], ['Filmfare Awards South 2014 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actress Attarintiki Daredi 2013 Nominado Filmfare Award - Kannada Film Industry Best Actress Bheema Theeradalli', 'South Indian International Movie Awards 2015 Nominado SIIMA - Kannada Best Actress in a Leading Role Brahma 2014 Nominado SIIMA - Kannada Best Actress in a Leading Role Whistle', 'Edison Awards India 2016 Ganador Merit Award Best Supporting Actress Massu Engira Masilamani', 'Indian Film and Television Awards 2022 Nominado Film Award Best Female Star Debut Bhuj: The Pride of India', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Nassar', '1958', ['Reparto', 'Dirección', 'Guion'], ['Baahubali 2, la conclusión', 'Baahubali: The Beginning', 'Caza a la espía'], ['Filmfare Awards South 2007 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Pokiri', 'South Indian International Movie Awards 2016 Nominado SIIMA - Tamil Best Actor in a Supporting Role Uttama Villain 2015 Nominado SIIMA - Tamil Best Actor in a Supporting Role Saivam 2013 Nominado SIIMA - Telugu Best Actor in a Supporting Role Business Man 2013 Nominado SIIMA - Telugu Best Performance by an Actor in a Negative Role Dhammu', 'Vijay Awards 2009 Nominado Jury Award Best Villain Poi Solla Porom', 'Tamil Nadu State Film Awards, India 1995 Nominado Creative Award Best Film Avathaaram 1992 Ganador Special Award Special Prize Aavarampoo', "Nandi Awards 1991 Ganador Nandi Award Best Villain Chanti For the the role 'Pedababu"]), ('Sayaji Shinde', '1959', ['Reparto', 'Producción', 'Reparto'], ['Shool', '1 - Nenokkadine', 'Antim: The Final Truth'], ['Filmfare Awards 2001 Nominado Filmfare Award Best Supporting Actor Kurukshetra 2000 Nominado Filmfare Award Best Performance by an Actor in a Negative Role Shool', 'Zee Cine Awards 2001 Nominado Popular Award Best Actor in a Supporting Role Kurukshetra', "Sansui Viewer's Choice Awards 2001 Nominado Sansui Viewer's Choice Award Best Supporting Actor Kurukshetra", 'Filmfare Awards South 2010 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Arundhati 2006 Nominado Filmfare Award - Telugu Film Industry Best Villain Andhrudu', "People's Choice Awards, India 2000 Nominado People's Choice Award Best Comic Performance Kurukshetra 1999 Nominado People's Choice Award Best Negative Performance Shool"]), ('Rahul Dev', '1968', ['Reparto', 'Reparto'], ['Champion', 'Asoka', 'Footpath'], ['Filmfare Awards 2001 Nominado Filmfare Award Best Performance by an Actor in a Negative Role Champion', 'Awards of the International Indian Film Academy 2001 Ganador Popular Award Best Male Debut Champion', 'Zee Cine Awards 2004 Ganador Popular Award Best Performance by an Actor in a Villainous Role Footpath 2001 Nominado Popular Award Best Debut Actor Champion', "Bollywood Awards 2003 Nominado Bollywood Movie Award Best Actor in a Villain's Role Awara Paagal Deewana 2002 Nominado Bollywood Movie Award Best Actor in a Villain's Role Aashiq", "Sansui Viewer's Choice Awards 2001 Nominado Sansui Viewer's Choice Award Most Promising Debut Actor Champion"]), ('Rangayana Raghu', 'NULL', ['Reparto', 'Música', 'Banda sonora'], ['su papel en Johny Mera Naam Preethi Mera Kaam', 'Hudugaru', 'Badava Rascal'], ['Filmfare Awards South 2014 Nominado Filmfare Award - Kannada Film Industry Best Supporting Actor Jayammana Maga 2012 Nominado Filmfare Award - Kannada Film Industry Best Supporting Actor 2011 Nominado Filmfare Award - Kannada Film Industry Best Supporting Actor For the movie Modalaasala (2010)', 'South Indian International Movie Awards 2022 Nominado SIIMA - Kannada Best Comic Actor Badava Rascal 2015 Nominado SIIMA - Kannada Best Comedian For the film "Power Star (2014)".', 'Suvarna Film Awards 2012 Nominado Jury Award Best Actor In A Negative Role Jarasandha 2012 Ganador Jury Award Best Comedian Johny Mera Naam Preethi Mera Kaam 2012 Nominado Jury Award Best Comedian Hudugaru 2012 Nominado Jury Award Best Comedian Paramathma', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Sadhu Kokila', '1966', ['Reparto', 'Composición', 'Música'], ['Mr'], ['Filmfare Awards South 2017 Nominado Filmfare Award - Kannada Film Industry Best Supporting Actor Zoom', 'South Indian International Movie Awards 2022 Nominado SIIMA - Kannada Best Comic Actor Sakath 2020 Ganador SIIMA - Kannada Best Comedian Yajamana 2017 Nominado SIIMA - Kannada Best Comic Actor Mungaru Male 2 2015 Nominado SIIMA - Kannada Best Comedian Mr. And Mrs. Ramchari', 'Suvarna Film Awards 2011 Nominado Jury Award Best Director Mr. Theertha', 'IIFA Utsavam 2016 Ganador Kannada Cinema Best Performance by an Actor in a Comic Role Mr. And Mrs. Ramchari', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Suchendra Prasad', 'NULL', ['Reparto', 'Reparto'], ['su papel en Brahma', 'Kempe Gowda 2', 'Manasmita'], 'NULL'), ('Bullet Prakash', 'NULL', ['Reparto', 'Reparto'], ['Bachchan', 'Bhajarangi', 'Brahma'], 'NULL'), ('Sumithra', 'NULL', ['Reparto', 'Reparto'], ['su papel en Nizhal Nijamakirathu', 'Valimai', 'Brahma'], 'NULL')], [], [('Alessandro Ingrà', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Uno Strano Weekend al Mare', 'Storia di un inganno', "Sbagliando s'impara"], 'NULL'), ('Alessandro Paci', '1964', ['Reparto', 'Dirección', 'Guion'], ['Andata e ritorno', 'Gli infami - Episodi di vita quotidiana', 'Non ci resta che ridere'], 'NULL'), ('Graziano Salvadori', '1964', ['Reparto', 'Dirección', 'Guion'], ['Sarebbe stato facile', 'Storia di un inganno', 'Locas de alegría'], 'NULL'), ('Giovanni Cacioppo', '1965', ['Reparto', 'Música', 'Reparto'], ["Sbagliando s'impara", 'Così è la vita', 'Lucignolo'], 'NULL'), ('Sossio Aruta', 'NULL', ['Reparto', 'Reparto'], ["su papel en Sbagliando s'impara", 'Uno Strano Weekend al Mare'], 'NULL'), ('Maura Leone', 'NULL', ['Reparto', 'Reparto'], ["su papel en Sbagliando s'impara", 'Eternal', 'Laguna'], 'NULL'), ('Barbara Kal', 'NULL', ['Reparto', 'Reparto'], ["su papel en Sbagliando s'impara", 'Se mi ascolti e mi credi'], 'NULL'), ('Luca Ferrante', '1979', ['Reparto', 'Reparto'], ["Sbagliando s'impara", "Alex l'ariete", 'El paraíso de las señoras'], 'NULL'), ('Pietro Fornaciari', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cybernetic Genesis - La guerra tra i due mondi', 'Capri', "Sbagliando s'impara"], 'NULL'), ('Gloria Tonini', 'NULL', ['Reparto', 'Reparto'], ["su papel en Sbagliando s'impara", 'Quando una storia è solo una storia', 'Non Solo Barzellette'], 'NULL')], [], [('Jordan Charles Walker', 'NULL', ['Reparto', 'Reparto'], ['su papel en Underneath: Children of the Sun', 'Temporary Suspicion', 'Bad Business'], 'NULL'), ('Marshida B. Harris', 'NULL', ['Reparto', 'Reparto'], ['su papel en Icon'], 'NULL'), ('Bruce Carlton Cunningham Jr.', 'NULL', ['Reparto', 'Producción', 'Guion'], 'NULL', 'NULL'), ('Joel Antony', 'NULL', ['Reparto', 'Producción', 'Reparto'], ['su papel en The Gift of Grace', 'Rumble', 'Absolution'], 'NULL'), ('Adam L. Flowers', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en Heat', 'The Kingdom y B'], 'NULL'), ('Mycol Vynn', 'NULL', ['Reparto', 'Reparto'], ['su papel en Icon'], 'NULL'), ('Rob Wood', 'NULL', ['Reparto', 'Reparto'], ['su papel en Chicago Med', 'Secretos, mentiras, pasiones y jazz', 'Respect'], 'NULL'), ('Ashley Santana', 'NULL', ['Reparto', 'Reparto'], ['su papel en Underneath: Children of the Sun', 'Hardware', 'Icon'], 'NULL'), ('James Hendricks', 'NULL', ['Reparto', 'Reparto'], ['su papel en Icon'], 'NULL'), ('Rashawn Ewing', 'NULL', ['Reparto', 'Reparto'], ['su papel en Icon', 'No More Mister Nice Guy y Static'], 'NULL')], [('Sarah Langrish-Smith', 'NULL', ['Equipos adicionales', 'Reparto', 'Localización'], ['su papel en Textual Relationship', 'The Deal', 'The Rizen: Possession'], ['NewFilmmakers Los Angeles 2017 Nominado NewFilmmakers Los Angeles Best of 2017 Award Best Foreign Short Film (Comedy) Textual Relationship Compartido con: Syd Heather · Dan Eycott · David Frias-Robles · Tom Glover', 'Limelight Film Awards 2017 Ganador Best Comedy Best Comedy Textual Relationship Compartido con: Syd Heather · Dan Eycott · David Frias-Robles 2017 Ganador Film of the Night Textual Relationship Compartido con: Syd Heather · Dan Eycott · Tom Glover · David Frias-Robles', 'New Renaissance Film Festival 2020 Nominado Festival Award Best Comedy Short The Deal Compartido con: Syd Heather', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Hayley Thompson', 'NULL', ['Reparto', 'Diseño de producción', 'Equipos adicionales'], ['su papel en Quentin Quarantine', 'The One With the Condom', 'Dot'], 'NULL')], [('Draper Wynston', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en Walker', 'Bid for Love 2 y Secrets Beneath the Floorboards'], 'NULL'), ('Manni Adriano', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Hopelessly Awkward', 'Gravity', 'Proof Sheet'], 'NULL'), ('Breezelle Fox', 'NULL', ['Reparto', 'Reparto'], ['su papel en Trinity'], 'NULL'), ('Rhonda Jackson', 'NULL', ['Reparto', 'Reparto'], ['su papel en Woman Scorned', 'No Weapon Formed Against Us', 'Bullshit'], ['San Diego Black Film Festival 2015 Ganador Festival Prize Best Dramatic Film No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Sherdon Lavan · Mimi Hojnik · Kam Ayeni · Marlene Vanegas · · · 2015 Nominado Festival Prize Best Religious Film No Weapon Formed Against Us Compartido con: Eriah S. Howard · Mimi Hojnik · Sherdon Lavan · Marlene Vanegas · Mahari Blanks · Kam Ayeni · · 2015 Nominado Festival Prize Best Film No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Sherdon Lavan · Tiana Grant · Mimi Hojnik · Marlene Vanegas · · 2015 Ganador Festival Prize Best Supporting Actor No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Tiana Grant · Mahari Blanks · Kam Ayeni · Marlene Vanegas · ·', "Fort Worth Indie Film Showcase, US 2016 Nominado FWIFS Award What's Love Got To Do With It? Award Woman Scorned Compartido con: Eriah S. Howard · Dreka Shevon · Sam Sarpong · Mimi Hojnik · Brent Marriott · Rekey Jackson · 2016 Nominado FWIFS Award Best Drama Short Woman Scorned Compartido con: Eriah S. Howard · Dreka Shevon · Crystal Singleton · Mimi Hojnik · Brent Marriott · Rekey Jackson · · 2016 Nominado FWIFS Award Every Woman Woman Scorned Compartido con: Eriah S. Howard · Dreka Shevon · Sam Sarpong · Rekey Jackson · Kam Ayeni · Brent Marriott · · 2016 Ganador Director's Choice Award Director's Choice Award Woman Scorned Compartido con: Eriah S. Howard · Sam Sarpong · Dreka Shevon · Crystal Singleton · Kam Ayeni · Brent Marriott · ·", "The People's Film Festival, New York 2016 Ganador Honorable Mention Best Short Film Woman Scorned Compartido con: Eriah S. Howard · Dreka Shevon · Sam Sarpong · Brent Marriott · Rekey Jackson · Mimi Hojnik · ·", "Las Vegas Black Film Festival 2016 Nominado Festival Prize Best Short Film Woman Scorned Compartido con: Eriah S. Howard · Mimi Hojnik · Crystal Singleton · Sam Sarpong · Dreka Shevon · Kam Ayeni · · 2015 Ganador Festival Prize People's Choice Award No Weapon Formed Against Us Compartido con: Eriah S. Howard · Mahari Blanks · Sam Sarpong · Mimi Hojnik · Marlene Vanegas · Sherdon Lavan · · 2015 Nominado Festival Prize Best Religious Film No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Kam Ayeni · Sherdon Lavan · Mahari Blanks · Tiana Grant · Marlene Vanegas · 2015 Ganador Festival Prize Best Actress in a Feature No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Mimi Hojnik · Mahari Blanks · Marlene Vanegas · Kam Ayeni · ·", 'IndieFEST Film Awards 2014 Ganador Award of Merit Best Film (African American) No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Mimi Hojnik · Mahari Blanks · Tiana Grant · Sherdon Lavan · · 2014 Ganador Award of Merit Christian No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Sherdon Lavan · Tiana Grant · Kam Ayeni · Marlene Vanegas · · 2014 Ganador Award of Merit Dramatic Impact No Weapon Formed Against Us Compartido con: Eriah S. Howard · Sam Sarpong · Mimi Hojnik · Marlene Vanegas · Kam Ayeni · Mahari Blanks · ·']), ('Ty Lathan', 'NULL', ['Reparto', 'Reparto'], ['su papel en Die Fighting', 'All That I Need', 'Trinity'], 'NULL'), ('Louis', 'NULL', ['Reparto', 'Reparto'], ['su papel en Trinity'], 'NULL'), ('David Pitt', 'NULL', ['Reparto', 'Edición', 'Reparto'], ['su papel en Trinity'], 'NULL')], [('Biff Yeager', '1942', ['Reparto', 'Producción', 'Dirección'], ['Eduardo Manostijeras', 'Sid y Nancy', 'Batman vuelve'], 'NULL'), ('Noora Albright', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en I Am Good', 'Life Mission', 'You'], ['Cinequest San Jose Film Festival 2017 Nominado Best Short Narrative Film Best Film You 2016 Nominado Best Short Narrative Film Best Film I Am Good', 'Columbus International Film & Video Festival 2015 Nominado Chris Award Narrative I Am Good', 'Ft. Lauderdale International Film Festival 2015 Nominado Jury Award Best Short I Am Good', 'WorldFest Houston 2015 Ganador Gold Award Best Short Film I Am Good 2012 Ganador Bronze Award Best Short Film Life Mission', 'Accolade Competition 2015 Ganador Award of Merit Women Filmmakers I Am Good']), ('Christopher Fairbanks', 'NULL', ['Reparto', 'Reparto'], ['su papel en Saints & Sinners', 'Alias', 'Watch Over Me'], 'NULL'), ('Xingu Del Rosario', 'NULL', ['Reparto', 'Guion', 'Especialistas'], ['su papel en DMZ', 'Leverage: Redemption', 'NCIS: Los Ángeles'], 'NULL'), ('M.J. Timothy', 'NULL', ['Reparto', 'Reparto'], ['su papel en Life Mission'], 'NULL'), ('Tony Haig', '1948', ['Reparto', 'Producción', 'Cámara y electricidad'], ["Shirley Temple's Storybook", 'Words & Actions', 'Six Gun Savior'], 'NULL'), ('Lance Johnson', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en Life Mission', 'Six Gun'], 'NULL'), ('Annzella Victoria', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mentes criminales', 'Dead Drop', 'H+'], 'NULL'), ('Pete Brown', '1952', ['Reparto', 'Reparto'], ["That's Life", 'Jimmy Kimmel Live!', 'Bloodshed'], 'NULL'), ('Youlanda Davis', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en American Auto', 'Your Honor', 'I Think You Should Leave with Tim Robinson'], 'NULL')], [('Secun de la Rosa', '1969', ['Reparto', 'Guion', 'Dirección'], ['El bar', 'La mula', 'Aída'], ['Málaga Spanish Film Festival 2021 Nominado Golden Biznaga Best Spanish Film (Mejor Película Española) El Cover 2021 Ganador Young Jury Award Best Film El Cover', 'Spanish Actors Union 2022 Nominado Award of the Spanish Actors Union Television: Performance in a Minor Role, Male (Reparto TV - Categoría Masculina) 30 Monedas 2018 Nominado Award of the Spanish Actors Union Film: Performance in a Minor Role, Male (Reparto Cine - Categoría Masculina) La Llamada 2011 Ganador Award of the Spanish Actors Union Television: Performance in a Minor Role, Male (Reparto TV - Categoría Masculina) Aída 2004 Ganador Award of the Spanish Actors Union Film: Performance in a Minor Role, Male (Reparto Cine - Categoría Masculina) Días de fútbol 2003 Nominado Award of the Spanish Actors Union Film: Performance in a Minor Role, Male (Reparto Cine - Categoría Masculina) El otro lado de la cama 1 más', 'Almería International Film Festival 2021 Nominado FICAL Best First Film El Cover', 'CinEuphoria Awards 2018 Nominado CinEuphoria Best Ensemble - International Competition El bar Compartido con: Alejandro Awada · Mario Casas · Joaquín Climent · Carmen Machi · Jaime Ordóñez · Terele Pávez · 2014 Nominado CinEuphoria Best Ensemble - International Competition Las brujas de Zugarramurdi Compartido con: Hugo Silva · Mario Casas · Carmen Maura · Terele Pávez · Pepón Nieto · Carolina Bang · Jaime Ordóñez · · · · · · ·', 'Top Shorts Film Festival 2019 Ganador December Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Roberto Millán · Pablo Escamilla · Diego Gurpegui · Ana Soro · María José Moreno 2018 Ganador February Award Best Ensemble Blown Away. Enchufados Compartido con: Arévalo · Roberto Millán · Pablo Escamilla · María Espinosa · Manuel Durán · Rodrigo Moreno · · ·']), ('Pablo Escamilla', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Dmus', 'Blown Away'], ['Top Shorts Film Festival 2019 Ganador December Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Diego Gurpegui · Ana Soro · María José Moreno 2018 Ganador February Award Best Ensemble Blown Away. Enchufados Compartido con: Secun de la Rosa · Arévalo · Roberto Millán · María Espinosa · Manuel Durán · Rodrigo Moreno · · ·', 'Los Angeles Film Awards 2018 Nominado Festival Award Best Ensemble Blown Away. Enchufados Compartido con: Secun de la Rosa · Arévalo · Roberto Millán · María Espinosa · Manuel Durán · Rodrigo Moreno · · · 2017 Ganador Honorable Mention Ensemble Dmus Compartido con: Tiffany Daniels · Elia Cantu · Bahia Haifi · Alexa Blanks · Aneiszka Sea · Rafa Higuera · Lara Baladrón · · · · · · · · · ·', 'Actors Awards, Los Angeles 2017 Ganador Actors Award Best Ensemble Dmus Compartido con: Bahia Haifi · Tiffany Daniels · Elia Cantu · Aneiszka Sea · Michelle Talley · Alexa Blanks · Adriana Gil · · · · · · · · · ·', 'Oniros Film Awards 2019 Nominado Best of Year Best Ensemble Blown Away. Enchufados Compartido con: Arévalo · Secun de la Rosa · María Espinosa · Roberto Millán · Rodrigo Moreno · Manuel Durán · · · 2018 Ganador August Monthly Edition Best Ensemble Blown Away. Enchufados Compartido con: Arévalo · Secun de la Rosa · Roberto Millán · María Espinosa · Rodrigo Moreno · Jorge Aparicio · · ·', 'New York Film Awards 2017 Ganador Honorable Mention Ensemble Blown Away. Enchufados Compartido con: Secun de la Rosa · Arévalo · Roberto Millán · María Espinosa · Manuel Durán · Rodrigo Moreno · · ·']), ('Diego Gurpegui', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cuando la Música Deja de Sonar', 'Armugán'], ['Top Shorts Film Festival 2019 Ganador December Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · Ana Soro · María José Moreno', 'Sunny Side Up Film Festival 2019 Ganador Sunny Side Up Film Festival Award Best Child Actor Cuando la Música Deja de Sonar (3rd place award)', 'Independent Shorts Awards 2019 Ganador Honorable Mention Best Acting Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Ana Soro · Pablo Escamilla · María José Moreno', 'Best Actor Award 2019 Ganador Gold Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · Ana Soro · María José Moreno', "Cowpokes Int'l Film Festival 2021 Ganador Jury Prize Best Young Actor Cuando la Música Deja de Sonar"]), ('Roberto Millán', 'NULL', ['Reparto', 'Dirección de producción', 'Reparto'], ['su papel en Cuando la Música Deja de Sonar', 'Blown Away'], ['Top Indie Film Awards 2020 Nominado Winter Award Best Actor Cuando la Música Deja de Sonar', 'Top Shorts Film Festival 2019 Ganador December Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Pablo Escamilla · Diego Gurpegui · Ana Soro · María José Moreno 2018 Ganador February Award Best Ensemble Blown Away. Enchufados Compartido con: Secun de la Rosa · Arévalo · Pablo Escamilla · María Espinosa · Manuel Durán · Rodrigo Moreno · · ·', 'Festigious International Film Festival 2018 Ganador Festival Award Best Supporting Actor Blown Away. Enchufados', 'Los Angeles Film Awards 2018 Nominado Festival Award Best Ensemble Blown Away. Enchufados Compartido con: Secun de la Rosa · Arévalo · Pablo Escamilla · María Espinosa · Manuel Durán · Rodrigo Moreno · · ·', 'Actors Awards, Los Angeles 2017 Nominado Actors Award Best Supporting Actor Blown Away. Enchufados']), ('María José Moreno', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cuando la Música Deja de Sonar', 'Una de zombis', 'Nuestros amantes'], ['Top Shorts Film Festival 2019 Ganador December Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · Diego Gurpegui · Ana Soro', 'Independent Shorts Awards 2019 Ganador Honorable Mention Best Acting Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Ana Soro · Pablo Escamilla · Diego Gurpegui', 'Best Actor Award 2019 Ganador Gold Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · Ana Soro · Diego Gurpegui', "Cowpokes Int'l Film Festival 2021 Nominado Jury Prize Best Supporting Actress (Short) Cuando la Música Deja de Sonar", 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Ana Soro', 'NULL', ['Reparto', 'Reparto'], ['su papel en Cuando la Música Deja de Sonar', 'Las Luces del Amanecer', 'El vuelo de Lena'], ['Top Shorts Film Festival 2019 Ganador December Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · Diego Gurpegui · María José Moreno', 'New York Film Awards 2022 Ganador January Award Best Acting Ensemble Las Luces del Amanecer Compartido con: Cotton Filgaira · Jesús Marco Murillo · Pablo Gimeno · Neil Santolaya', 'Independent Shorts Awards 2019 Ganador Honorable Mention Best Acting Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · María José Moreno · Diego Gurpegui', 'Best Actor Award 2019 Ganador Gold Award Best Ensemble Cuando la Música Deja de Sonar Compartido con: Secun de la Rosa · Roberto Millán · Pablo Escamilla · María José Moreno · Diego Gurpegui', "Cowpokes Int'l Film Festival 2021 Nominado Jury Prize Best Actress (Short) Cuando la Música Deja de Sonar"])], [('Darin Murphy', 'NULL', ['Reparto', 'Composición', 'Guion'], ["su papel en Empty Nester's Handbook", 'My Ranger', 'Inhumanity'], 'NULL'), ('Beth Pina', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Your First Day in Hell', 'Sign Here', 'Salvando a Salvador'], 'NULL'), ('Christopher Proutt', '1991', ['Reparto', 'Guion', 'Producción'], ['A Casual Affair', 'Revolution', 'Growing Apart'], 'NULL'), ('Charlotte Delaney Riggs', 'NULL', ['Reparto', 'Guion', 'Dirección de la segunda unidad o asistencia a dirección'], ['su papel en Alita: Ángel de combate', 'Forever and a Day', 'Thunder Road'], 'NULL'), ('Alyssia Rivera', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Tamale Season', 'Losing Lalo', 'Leverage'], ['Twister Alley International Film Festival 2018 Ganador Festival Prize Best Ensamble - Feature Compartido con: James Christopher · Mike Donis · Rachel Clark-Spear · Mindy Fay Parks · Christopher James Raynor · ·', "Austin Revolution Film Festival 2023 Ganador Festival Award Best Texas Comedy Short Looking Back Compartido con: Michael Charron 2019 Ganador Hall Of Fame Award Hall Of Fame 2016 Ganador Director's Choice Directors Choice Actress Unimaginable 2016 Nominado Festival Award Best Texas Actress Unimaginable", 'Reel East Texas Film Festival 2023 Ganador Reel East Texas Film Festival Reel Award Best Actress - Short Leverage 2023 Ganador Reel East Texas Film Festival Reel Award Best Texas Short Leverage Compartido con: Michael Charron', 'MidWest WeirdFest 2023 Ganador MidWest WeirdFest Award Best Short Film Leverage Compartido con: Michael Charron', "Prison City Film Festival 2023 Ganador The Warden's Vanguard Award Best Overall Submission Looking Back Compartido con: Michael Charron"]), ('Katusha Robert', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Public Displays of Insanity', 'Central Standard', 'Aurora'], ['Indie Short Fest 2019 Nominado Certificate of Achievement Best Dark Comedy Short Headphones Compartido con: Kanan Capshaw · Danny Trevino', 'IndieX Film Festival 2019 Nominado Major Award Best Acting Duo Headphones Compartido con: Danny Trevino', 'Hollywood Blood Horror Festival 2019 Ganador July Award Best Acting Duo Headphones Compartido con: Danny Trevino', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Tiana Stuart', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Love & Death', 'Merrily We Roll Along y The Flood'], 'NULL'), ('Danny Trevino', 'NULL', ['Reparto', 'Equipos adicionales', 'Producción'], ['su papel en Headphones', 'From Dusk Till Dawn: The Series', 'Trap Plane'], ["Idyllwild International Festival of Cinema 2017 Nominado IIFC Award Best Ensemble - Feature Trippin' to the Altar Compartido con: Oryan Landa · Lara Shah · Jordan LeuVoy · Tiana Stuart · Kathleen Brown · Jordan Michael Brinkman", 'Indie Short Fest 2019 Nominado Certificate of Achievement Best Dark Comedy Short Headphones Compartido con: Kanan Capshaw · Katusha Robert', 'IndieX Film Festival 2019 Nominado Major Award Best Acting Duo Headphones Compartido con: Katusha Robert', 'Hollywood Blood Horror Festival 2019 Ganador July Award Best Acting Duo Headphones Compartido con: Katusha Robert', 'Austin Indie Fest 2019 Ganador Best Actor Short Film Headphones'])], [('Nora Navas', '1975', ['Reparto', 'Banda sonora', 'Reparto'], ['Dolor y gloria', 'Todos queremos lo mejor para ella', 'Pa negre'], ['Angers European First Film Festival 2014 Ganador Best Actress Todos queremos lo mejor para ella', 'Cinema Writers Circle Awards, Spain 2022 Ganador CEC Award Best Supporting Actress (Mejor Actriz Secundaria) Libertad 2016 Nominado CEC Award Best Actress (Mejor Actriz) La adopción 2014 Nominado CEC Award Best Actress (Mejor Actriz) Todos queremos lo mejor para ella', 'Goya Awards 2022 Ganador Goya Best Supporting Actress (Mejor Actriz de Reparto) Libertad 2016 Nominado Goya Best Supporting Actress (Mejor Actriz de Reparto) Felices 140 2014 Nominado Goya Best Lead Actress (Mejor Actriz Protagonista) Todos queremos lo mejor para ella 2011 Ganador Goya Best Lead Actress (Mejor Actriz Protagonista) Pa negre', 'Miami Film Festival 2014 Ganador Grand Jury Prize Best Performance Todos queremos lo mejor para ella', 'San Sebastián International Film Festival 2010 Ganador Silver Seashell Best Actress Pa negre']), ('Halima Ilter', 'NULL', ['Reparto', 'Reparto'], ['Kafkanistan', 'Zagros', 'Carneval - Der Clown bringt den Tod'], 'NULL'), ('Eman Eido', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sinjar'], 'NULL'), ('Guim Puig', 'NULL', ['Reparto', 'Banda sonora', 'Reparto'], ['su papel en Bienvenidos a Edén', 'Paraíso', 'Cucut'], 'NULL'), ('Mouafaq Rushdie', 'NULL', ['Reparto', 'Reparto'], ["su papel en Soeurs d'armes", 'Das Milan-Protokoll', 'Sinjar'], 'NULL'), ('Luisa Gavasa', '1951', ['Reparto', 'Reparto'], ['La novia', 'Paris 70', 'Una vida asegurada'], ['Cinema Writers Circle Awards, Spain 2024 Nominado CEC Award Best Supporting Actress (Mejor Actriz Secundaria) El maestro que prometió el mar 2016 Ganador CEC Award Best Supporting Actress (Mejor Actriz Secundaria) La novia', 'Goya Awards 2024 Nominado Goya Best Supporting Actress (Mejor Actriz de Reparto) El maestro que prometió el mar 2016 Ganador Goya Best Supporting Actress (Mejor Actriz de Reparto) La novia', 'Spanish Actors Union 2018 Nominado Award of the Spanish Actors Union Television: Performance in a Minor Role, Female (Reparto TV - Categoría Femenina) El Ministerio del Tiempo 2016 Ganador Award of the Spanish Actors Union Film: Supporting Performance, Female (Secundario Cine - Categoría Femenina) La novia', 'Feroz Awards, ES 2024 Nominado Feroz Award Best Actress in a Supporting Role El maestro que prometió el mar 2016 Ganador Feroz Award Best Actress in a Supporting Role La novia', 'CinEuphoria Awards 2024 Nominado CinEuphoria Best Actress in a Short Film - International Competition Paris 70 2019 Nominado CinEuphoria Best Supporting Actress - International Competition La novia']), ('Franz Harram', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sinjar'], 'NULL'), ('Àlex Casanovas', '1964', ['Reparto', 'Dirección', 'Guion'], ['Manila', 'Barcelona 1714', 'Refree graba Nones'], ['Gaudí Awards 2013 Nominado Gaudí Award Best Male Lead (Millor Protagonista Masculí) Fènix 11·23', 'Catalonian Actors and Directors Association 1992 Ganador AADC Award Best Movie Actor Manila', 'Premis de Cinematografia de la Generalitat de Catalunya 1992 Ganador Premi de Cinematografia de la Generalitat de Catalunya Best Actor Manila', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Hennan Bereket Qasso', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sinjar'], 'NULL'), ('Samia Naif', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sinjar'], 'NULL')], [('Emily Tremaine Fernandez', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en El lobo de Wall Street', 'El estornino', 'Vinyl'], 'NULL'), ('Arnold Chun', '1976', ['Reparto', 'Dirección', 'Producción'], ['El hombre en el castillo', 'Kidding', 'Kingdom'], ['Valley Film Festival 2017 Ganador Audience Award Best Short Drama Fractured', 'Moondance International Film Festival 2017 Nominado Short Film Award Best Short Film Fractured', 'Honolulu International Festival (II) 2010 Ganador Golden Kahuna Award Excellence in Filmmaking Invincible Scripture', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Shana Dowdeswell', '1989', ['Reparto', 'Reparto'], ['La gran boda', 'Mistress America', 'Mercy'], 'NULL'), ('Thomas R. Baker', 'NULL', ['Reparto', 'Decoración', 'Equipos adicionales'], 'NULL', ['Dreamachine International Film Festival 2023 Ganador Spring Award Best Action Film Halo: The Rebellion Compartido con: Jamon Holmes · Delaney Vance · Martin Elias · Elena Ghenoiu · Maryam Roberts · Curtis Beard Jr. · · · · · · · · · · · · · · · · · · · · · · · · ·', 'Red Movie Award 2022 Nominado December Award Best Fan Film Halo: The Rebellion Compartido con: Jamon Holmes · Delaney Vance · Daniel Furuya · Peter L. Johnson · Maryam Roberts · Arlene Newman-Van Asperen · · ·', 'Paradise Film Festival 2022 Ganador July Award Best Action/Adventure Halo: The Rebellion Compartido con: Jamon Holmes · Delaney Vance · Daniel Furuya · Peter L. Johnson · Maryam Roberts · Elena Ghenoiu · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Heidi Kristoffer', '1982', ['Reparto', 'Reparto'], ['Confesiones de una compradora compulsiva', 'Todos mis hijos', 'La hija de mi mejor amigo'], 'NULL'), ('Steve Beauchamp', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Cómo deshacerte de tu jefe', 'El irlandés', 'Ley y orden: Unidad de Víctimas Especiales'], 'NULL'), ('Kevin Henderson', 'NULL', ['Reparto', 'Equipos adicionales', 'Guion'], ['su papel en Alita: Ángel de combate', 'Avatar: El sentido del agua', 'Mercy'], 'NULL'), ('April Rawlings', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Producción'], ['su papel en Mercy', 'CSI: Miami', 'Dahlias: Wild Card'], 'NULL'), ('Nate Lombardi', 'NULL', ['Reparto', 'Producción', 'Música'], ['su papel en Mercy', 'Blue Bloods', 'See You in September'], 'NULL'), ('Rick Pantera', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mercy', 'The letter', 'Argyle'], 'NULL')], [('Julie Corgill', 'NULL', ['Reparto', 'Transporte', 'Reparto'], 'NULL', 'NULL'), ('Glen Jensen', 'NULL', ['Reparto', 'Reparto'], ['su papel en Denizen', 'Salvation', 'Huella dactilar'], 'NULL'), ('Ben Bayless', '1972', ['Reparto', 'Cámara y electricidad', 'Equipos adicionales'], ['Salvation', 'Denizen', 'Arañas devoradoras'], 'NULL'), ('Jody Mullins', '1972', ['Reparto', 'Especialistas', 'Equipos adicionales'], ['Los siete magníficos', 'Barry Seal: El traficante', 'NCIS: Nueva Orleans'], 'NULL'), ('Dianne Sullivan', 'NULL', ['Reparto', 'Equipos adicionales', 'Producción'], ['su papel en N-Secure', 'My Generation', 'Last Kung Fu Monk'], 'NULL'), ('J.A. Steel', '1969', ['Guion', 'Dirección', 'Producción'], ['Salvation', 'Denizen', 'Death Key: Origins'], ['B-Movie Film Festival 2002 Nominado B-Movie Award Best Action Sequences The Third Society', "Bare Bones International Film & Music Festival 2010 Nominado Bonehead Award Best Oklahoma Soil Film Denizen 2010 Nominado Bonehead Award Best Feature Horror/Sci-Fi Denizen 2008 Nominado Bonehead Award Best Artists Soundtrack Salvation Compartido con: Dan Radlauer · Cody McCallister · Emily O'Neary 2008 Nominado Bonehead Award Best Oklahoma Soil Film Salvation 2008 Nominado Bonehead Certificate Best Actor Indie Auteur Salvation 4 más", 'WorldFest Houston 2016 Ganador Gold Remi Award Work in Progress - Shorts Death Key: Origins Compartido con: Christian K. Koch · Frederick W. Mercer · John A. Ruffner III · Diana L. Ruffner · Rodney Ridenour · · ·', "Park City Film Music Festival 2008 Ganador Bronze Medal for Excellence Best Impact of Music in a Feature Film Salvation Compartido con: Dan Radlauer · Emily O'Neary · Cody McCallister 2005 Ganador Gold Medal for Excellence Feature Film: Action Film Score The Third Society Compartido con: James Henriques · Emily O'Neary", 'Shriekfest 2012 Nominado Shriekfest Award Best Song Blood Fare Compartido con: Caleb S. Gray']), ('Jennifer Lea Turner', '1980', ['Equipos adicionales', 'Reparto', 'Cámara y electricidad'], ['Tracker', 'Denizen', 'Lost Suburbia'], 'NULL'), ('Janet Mayson', '1969', ['Reparto', 'Producción', 'Dirección'], ['Libby Girl: Bored Games', 'Libby Girl: Curiosity Driven', 'Libby Girl: Something New!'], 'NULL'), ('Hilary Delgado', 'NULL', ['Reparto', 'Reparto'], ['su papel en Denizen', 'Denizen: Descent'], 'NULL'), ('Dominique Storni', 'NULL', ['Reparto', 'Reparto'], ['su papel en Denizen', 'Denizen: Descent', 'Out of the Box: Inside Out'], 'NULL')], [('Shahid Kapoor', '1981', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Música'], ['Haider', 'Jab We Met', 'Udta Punjab'], ["Filmfare Awards 2023 Nominado Critics' Awards Best Actor Jersey 2020 Nominado Filmfare Award Best Actor Kabir Singh 2017 Ganador Critics' Awards Best Actor Udta Punjab Tied with Manoj Bajpayee for Aligarh (2015) 2017 Nominado Filmfare Award Best Actor Udta Punjab 2015 Ganador Filmfare Award Best Actor Haider 3 más", 'Awards of the International Indian Film Academy 2020 Ganador Popular Award Best Actor in a Leading Role Kabir Singh 2017 Ganador Popular Award Best Actor in a Leading Role Udta Punjab 2015 Ganador Popular Award Best Actor in a Leading Role Haider 2010 Nominado Popular Award Best Actor in a Leading Role Kaminey 2008 Nominado Popular Award Best Actor in a Leading Role Jab We Met 2 más', "Zee Cine Awards 2023 Nominado Jury's Choice Award Best Actor Jersey 2020 Nominado Viewers' Choice Award Best Actor Kabir Singh 2020 Nominado Jury's Choice Award Best Actor Kabir Singh 2017 Nominado Critics Choice Award Best Actor Udta Punjab 2008 Nominado Popular Award Best Actor Jab We Met 1 más", 'Apsara Film Producers Guild Awards 2015 Ganador Apsara Award Best Actor - Movie Haider 2010 Nominado Apsara Award Best Actor Kaminey 2008 Nominado Apsara Award Best Actor Jab We Met', "Sansui Viewer's Choice Awards 2004 Ganador Sansui Viewer's Choice Award Most Promising Debut Actor Ishq Vishk"]), ('Sonakshi Sinha', '1987', ['Reparto', 'Música', 'Vestuario'], ['Holiday', 'Dabangg', 'Lootera'], ['Filmfare Awards 2014 Nominado Filmfare Award Best Actress Lootera 2011 Ganador Filmfare Award Best Debut Actress Dabangg', 'Awards of the International Indian Film Academy 2014 Nominado Popular Award Best Actress in a Leading Role Lootera 2011 Ganador Popular Award Best Debut Actress Dabangg', 'Zee Cine Awards 2014 Ganador Critics Choice Award Best Actress Lootera 2011 Ganador Popular Award Best Female Debut Dabangg 2011 Nominado Popular Award International Icon - Female', 'Apsara Film Producers Guild Awards 2014 Nominado Apsara Award Best Actress Lootera 2011 Ganador Apsara Award Best Debut Actress Dabangg', 'South Indian International Movie Awards 2015 Nominado SIIMA - Tamil Best Female Debutant Lingaa']), ('Sonu Sood', '1973', ['Reparto', 'Producción', 'Dirección'], ['Dabangg', 'Jodhaa Akbar', 'Dookudu'], ['Filmfare Awards 2009 Nominado Filmfare Award Best Supporting Actor Jodhaa Akbar', 'Awards of the International Indian Film Academy 2011 Ganador Popular Award Best Performance by an Actor in a Negative Role Dabangg 2009 Nominado Popular Award Best Actor in a Supporting Role Jodhaa Akbar', "Zee Cine Awards 2019 Nominado Jury's Choice Award Best Performance by an Actor Negative Role Simmba 2011 Nominado Popular Award Best Performance by an Actor in a Negative Role Dabangg", 'Apsara Film Producers Guild Awards 2011 Ganador Apsara Award Best Performance by an Actor in a Negative Role Dabangg 2009 Nominado Apsara Award Best Actor In A Supporting Role Jodhaa Akbar', 'Filmfare Awards South 2012 Nominado Filmfare Award - Telugu Film Industry Best Supporting Actor Kandireega 2010 Ganador Filmfare Award - Telugu Film Industry Best Supporting Actor Arundhati']), ('Govardhan Asrani', '1941', ['Reparto', 'Dirección', 'Producción'], ['Bodyguard', 'Sholay', 'Chala Murari Hero Banne'], ['Filmfare Awards 1982 Nominado Filmfare Award Best Comic Actor Ek Duuje Ke Liye 1980 Nominado Filmfare Award Best Comedian/Comedienne Sargam 1979 Nominado Filmfare Award Best Comic Actor Pati Patni Aur Woh 1977 Ganador Filmfare Award Best Comedian/Comedienne Balika Badhu 1975 Nominado Filmfare Award Best Comic Actor Bidaai 4 más', "People's Choice Awards, India 2012 Nominado Movie Award Favourite Ensemble Cast Bol Bachchan Compartido con: Ajay Devgn · Abhishek Bachchan · Asin Thottumkal · Prachi Desai · Archana Puran Singh · Neeraj Vora · · ·", 'Screen Awards, IN 2013 Nominado Screen Award Best Ensemble Cast Bol Bachchan Compartido con: Ajay Devgn · Abhishek Bachchan · Asin Thottumkal · Prachi Desai · Archana Puran Singh · Neeraj Vora · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Ashish Vidyarthi', '1962', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Haider', '24', 'Teenkahon'], ['Filmfare Awards 1996 Nominado Filmfare Award Best Performance by an Actor in a Negative Role Droh Kaal', 'Zee Cine Awards 1998 Nominado Zee Cine Award Best Performance by an Actor in a Negative Role (Male/Female) Ziddi', 'National Film Awards, India 1995 Ganador Silver Lotus Award Best Supporting Actor Droh Kaal Tied with Nagesh for Nammavar (1994). (For bringing credibility to his role with strength and total conviction.)', 'Filmfare Awards South 2006 Ganador Filmfare Award - Telugu Film Industry Best Villain Athanokkade', 'South Indian International Movie Awards 2015 Nominado SIIMA - Kannada Best Performance by an Actor in a Negative Role Shivajinagara']), ('Mukul Dev', '1970', ['Reparto', 'Guion', 'Reparto'], ['21 Sarfarosh - Saragarhi 1897', 'Omerta', 'Daud Pakad'], 'NULL'), ('Srihari', '1964', ['Reparto', 'Producción', 'Edición'], ['Sri Ramulayya', 'Nuvvostanante Nenoddantana', 'Magadheera'], 'NULL'), ('Bharat Dabholkar', '1969', ['Reparto', 'Dirección', 'Guion'], ['God Only Knows!', 'Company', 'Blame It on Yashraj'], 'NULL'), ('Vijay Patkar', '1963', ['Reparto', 'Dirección', 'Producción'], ['Chashme Bahaddar', 'Life in Dark', 'Mohar'], 'NULL'), ('Ashok Samarth', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mareparyant Phaashi', 'Dhamak', 'Right Yaaa Wrong'], 'NULL')], [('Jack Black', '1969', ['Reparto', 'Producción', 'Guion'], ['Escuela de rock', 'Alta fidelidad', 'Super Nacho'], ['American Comedy Awards, USA 2001 Nominado American Comedy Award Funniest Supporting Actor in a Motion Picture Alta fidelidad', 'Annie Awards 2024 Nominado Annie Outstanding Achievement for Voice Acting in an Animated Feature Super Mario Bros.: La película', 'Blockbuster Entertainment Awards 2001 Ganador Blockbuster Entertainment Award Favorite Supporting Actor - Comedy/Romance Alta fidelidad', 'Boston Society of Film Critics Awards 2000 Nominado BSFC Award Best Supporting Actor Alta fidelidad', 'Critics Choice Awards 2013 Nominado Critics Choice Award Best Actor in a Comedy Bernie']), ('Bryan Cranston', '1956', ['Reparto', 'Producción', 'Guion'], ['Breaking Bad', 'Malcolm', 'Argo'], ['Academy Awards, USA 2016 Nominado Oscar Best Performance by an Actor in a Leading Role Trumbo: La lista negra de Hollywood', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2018 Nominado Saturn Award Best Guest Performance in a Television Series Electric Dreams 2014 Nominado Saturn Award Best Actor on Television Breaking Bad 2013 Ganador Saturn Award Best Actor on Television Breaking Bad Tied with Kevin Bacon for The Following (2013). 2012 Ganador Saturn Award Best Actor on Television Breaking Bad 2011 Nominado Saturn Award Best Actor on Television Breaking Bad 2 más', 'Annie Awards 2019 Ganador Annie Outstanding Achievement for Voice Acting in an Animated Feature Production Isla de perros as Chief', 'Breckenridge Festival of Film 1999 Ganador Best of the Fest Drama Last Chance', 'BAFTA Awards 2016 Nominado BAFTA Film Award Best Leading Actor Trumbo: La lista negra de Hollywood']), ('Dustin Hoffman', '1937', ['Reparto', 'Producción', 'Dirección'], ['Tootsie', 'Rain Man', 'Cowboy de medianoche'], ['Academy Awards, USA 1998 Nominado Oscar Best Actor in a Leading Role La cortina de humo 1989 Ganador Oscar Best Actor in a Leading Role Rain Man 1983 Nominado Oscar Best Actor in a Leading Role Tootsie 1980 Ganador Oscar Best Actor in a Leading Role Kramer contra Kramer 1975 Nominado Oscar Best Actor in a Leading Role Lenny 2 más', 'American Film Institute, USA 1999 Ganador Life Achievement Award', 'Annie Awards 2009 Ganador Annie Best Voice Acting in an Animated Feature Production Kung Fu Panda', 'Berlin International Film Festival 1989 Ganador Honorary Golden Berlin Bear', 'Boston Society of Film Critics Awards 1983 Ganador BSFC Award Best Actor Tootsie']), ('Angelina Jolie', '1975', ['Reparto', 'Producción', 'Dirección'], ['Maléfica', 'Inocencia interrumpida', 'El intercambio'], ['Academy Awards, USA 2009 Nominado Oscar Best Performance by an Actress in a Leading Role El intercambio 2000 Ganador Oscar Best Actress in a Supporting Role Inocencia interrumpida 2014 Ganador Jean Hersholt Humanitarian Award', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2015 Nominado Saturn Award Best Actress Maléfica 2011 Nominado Saturn Award Best Actress Salt 2009 Ganador Saturn Award Best Actress El intercambio 2005 Nominado Saturn Award Best Supporting Actress Sky Captain y el mundo del mañana 2002 Nominado Saturn Award Best Actress Lara Croft: Tomb Raider', 'American Society of Cinematographers, USA 2018 Ganador Board of the Governors Award', 'Berlin International Film Festival 2007 Ganador Silver Berlin Bear Outstanding Artistic Contribution El buen pastor Compartido con: John Sessions · Oleg Stefan · Martina Gedeck · Michael Gambon · John Turturro · Keir Dullea · · · · · · · · · · · ·', 'Blockbuster Entertainment Awards 2001 Ganador Blockbuster Entertainment Award Favorite Actress - Action 60 segundos 2000 Nominado Blockbuster Entertainment Award Favorite Actress - Suspense El coleccionista de huesos 2000 Ganador Blockbuster Entertainment Award Favorite Supporting Actress - Drama Inocencia interrumpida']), ('J.K. Simmons', '1955', ['Reparto', 'Banda sonora', 'Reparto'], ['Whiplash', 'Juno', 'La ciudad de las estrellas'], ['Academy Awards, USA 2022 Nominado Oscar Best Performance by an Actor in a Supporting Role Ser los Ricardo 2015 Ganador Oscar Best Performance by an Actor in a Supporting Role Whiplash', 'Academy of Science Fiction, Fantasy & Horror Films, USA 2015 Nominado Saturn Award Best Supporting Actor Whiplash', 'Boston Society of Film Critics Awards 2014 Ganador BSFC Award Best Supporting Actor Whiplash', 'BAFTA Awards 2015 Ganador BAFTA Film Award Best Supporting Actor Whiplash', 'Critics Choice Awards 2022 Nominado Critics Choice Award Best Supporting Actor Ser los Ricardo 2015 Ganador Critics Choice Award Best Supporting Actor Whiplash 2008 Nominado Critics Choice Award Best Acting Ensemble Juno Compartido con: Allison Janney · Elliot Page · Jennifer Garner · Jason Bateman · Michael Cera · Olivia Thirlby']), ('Jackie Chan', '1954', ['Producción', 'Reparto', 'Especialistas'], ['Who Am I? (¿Quién soy?', 'Hora punta', 'Armas invencibles'], ['Academy Awards, USA 2017 Ganador Honorary Award An international film star who has captivated millions with his wit, boundless energy and unparalleled athletic artistry.', "Asia-Pacific Film Festival 2010 Ganador Outstanding Achievement Award 2005 Ganador Special Jury Award 1993 Ganador Lifetime Achievement Award This honor was (likely) given to the youngest person ever at any world wide 'lifetime achievement awards'.", 'Blockbuster Entertainment Awards 2001 Nominado Blockbuster Entertainment Award Favorite Action Team (Internet Only) Shanghai Kid: Del Este al Oeste Compartido con: Owen Wilson 1999 Ganador Blockbuster Entertainment Award Favorite Duo - Action/Adventure Hora punta Compartido con: Chris Tucker', 'Cinequest San Jose Film Festival 1998 Ganador Maverick Tribute Award', 'Daytime Emmy Awards 2002 Nominado Daytime Emmy Outstanding Performer in an Animated Program Las aventuras de Jackie Chan For playing himself.']), ('Seth Rogen', '1982', ['Producción', 'Reparto', 'Guion'], ['Juerga hasta el fin', 'Supersalidos', 'Superfumados'], ['American Comedy Awards, USA 2014 Nominado American Comedy Award Best Comedy Actor - Film Juerga hasta el fin 2014 Ganador American Comedy Award Best Comedy Director - Film Juerga hasta el fin Compartido con: Evan Goldberg 2014 Nominado American Comedy Award Best Comedy Screenplay Juerga hasta el fin Compartido con: Evan Goldberg', 'Annie Awards 2024 Nominado Annie Outstanding Achievement in Writing for an Animated Feature Ninja Turtles: Caos mutante Compartido con: Evan Goldberg · Jeff Rowe · Dan Hernandez · Benji Samit', 'Canadian Comedy Awards 2009 Ganador Canadian Comedy Award Canadian Comedy Person of the Year - All Media 2008 Ganador Canadian Comedy Award Canadian Comedy Person of the Year - All Media 2008 Ganador Canadian Comedy Award Best Writing - Film Supersalidos Compartido con: Evan Goldberg', 'Primetime Emmy Awards 2022 Nominado Primetime Emmy Outstanding Limited or Anthology Series Pam & Tommy Compartido con: Megan Ellison · Sue Naegle · Ali Krug · Evan Goldberg · James Weaver · Alex McAtee · Robert Siegel · · · · · · 2022 Nominado Primetime Emmy Outstanding Supporting Actor in a Limited or Anthology Series or Movie Pam & Tommy For playing: "Rand Gautier". 2022 Nominado Primetime Emmy Outstanding Short Form Animated Program The Boys: Diabolical For "John and Sun-Hee (2022)" Compartido con: Simon Racioppa · Eric Kripke · Evan Goldberg · James Weaver · Neal H. Moritz · Pavun Shetty · · · · · · · · · · · · · · 2021 Nominado Primetime Emmy Outstanding Drama Series The Boys Compartido con: Eric Kripke · Evan Goldberg · James Weaver · Neal H. Moritz · Pavun Shetty · Craig Rosenberg · · · · · · · · · · 2005 Nominado Primetime Emmy Outstanding Writing for a Variety, Music or Comedy Program Da Ali G Show Compartido con: Sacha Baron Cohen · Anthony Hines · Dan Mazer · James Bobin · Jeff Stilson · Rich Dahm · Jamie Glassman ·', 'Golden Globes, USA 2023 Nominado Golden Globe Best Performance by an Actor in a Supporting Role in a Limited Series, Anthology Series or Motion Picture Made for Television Pam & Tommy']), ('Lucy Liu', '1968', ['Reparto', 'Dirección', 'Producción'], ['Kill Bill: Volumen 1', 'Los ángeles de Charlie', 'Chicago'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 2004 Nominado Saturn Award Best Supporting Actress Kill Bill: Volumen 1 2001 Nominado Saturn Award Best Supporting Actress Los ángeles de Charlie', 'American Comedy Awards, USA 2000 Nominado American Comedy Award Funniest Supporting Female Performer in a TV Series Ally McBeal', 'Blockbuster Entertainment Awards 2001 Ganador Blockbuster Entertainment Award Favorite Action Team (Internet Only) Los ángeles de Charlie Compartido con: Drew Barrymore · Cameron Diaz 2001 Ganador Blockbuster Entertainment Award Favorite Supporting Actress - Action Shanghai Kid: Del Este al Oeste', 'Critics Choice Awards 2003 Ganador Critics Choice Award Best Acting Ensemble Chicago Compartido con: Christine Baranski · Ekaterina Chtchelkanova · Taye Diggs · Denise Faye · Colm Feore · Richard Gere · · · · · · · ·', 'Primetime Emmy Awards 1999 Nominado Primetime Emmy Outstanding Supporting Actress in a Comedy Series Ally McBeal For playing: "Ling Woo".']), ('David Cross', '1964', ['Reparto', 'Guion', 'Producción'], ['Arrested Development', '¡Olvídate de mí!', 'Megamind'], ['CableACE Awards 1997 Ganador CableACE Original Song Mr. Show For episode: "The Biggest Failure in Broadway History" Compartido con: Bob Odenkirk · Eban Schletter · Dino Stamatopoulos', 'Primetime Emmy Awards 1999 Nominado Primetime Emmy Outstanding Writing for a Variety or Music Program Mr. Show Compartido con: Bob Odenkirk · Scott Aukerman · Jerry Collins · Jay Johnston · Bill Odenkirk · BJ Porter · Brian Posehn · 1998 Nominado Primetime Emmy Outstanding Music and Lyrics Mr. Show For the song "How High The Mountain". Compartido con: Dino Stamatopoulos · Eban Schletter · Bill Odenkirk · Bob Odenkirk 1998 Nominado Primetime Emmy Outstanding Writing for a Variety or Music Program Mr. Show Compartido con: Bob Odenkirk · Jay Johnston · Bill Odenkirk · Brian Posehn · Dino Stamatopoulos · Michael Stoyanov · · 1993 Ganador Primetime Emmy Outstanding Writing in a Variety or Music Program The Ben Stiller Show Compartido con: Judd Apatow · Robert Cohen · Brent Forrester · Jeff Kahn · Bruce Kirschbaum · Bob Odenkirk · · ·', 'Satellite Awards 2004 Nominado Golden Satellite Award Best Actor in a Supporting Role in a Series, Comedy or Musical Arrested Development', 'Grammy Awards 2017 Nominado Grammy Best Comedy Album Album: "...America...Great..." 2004 Nominado Grammy Best Comedy Album For "Shut Up, You F***ing Baby!"', 'Phoenix Film Festival 2003 Ganador Copper Wing Award Best Ensemble Acting Melvin Goes to Dinner Compartido con: Michael Blieden · Stephanie Courtney · Matt Price · Annabelle Gurwitch · Maura Tierney · Melora Walters · · ·']), ('Kate Hudson', '1979', ['Reparto', 'Producción', 'Dirección'], ['Casi famosos', 'Cómo perder a un chico en 10 días', 'Guerra de novias'], ['Academy Awards, USA 2001 Nominado Oscar Best Actress in a Supporting Role Casi famosos', 'American Comedy Awards, USA 2001 Nominado American Comedy Award Funniest Supporting Actress in a Motion Picture Casi famosos', 'Blockbuster Entertainment Awards 2001 Ganador Blockbuster Entertainment Award Favorite Female - Newcomer Casi famosos', 'BAFTA Awards 2001 Nominado BAFTA Film Award Best Performance by an Actress in a Leading Role Casi famosos', 'Critics Choice Awards 2001 Ganador Critics Choice Award Breakthrough Artist Casi famosos 2001 Nominado Critics Choice Award Best Supporting Actress Casi famosos'])], [('Sam Gittins', '1993', ['Reparto', 'Reparto'], ['The Liberator', 'Obediencia', 'Los amos del aire'], ['Southampton International Film Festival 2020 Nominado Best Supporting Actor Sorority 2013 Ganador Festival Prize Leading Actor in a Short Driftwood', 'British Urban Film Festival 2020 Nominado Festival Prize Best Actor Sorority', 'British Independent Film Festival 2022 Nominado British Independent Film Festival Award Best Supporting Actor The Last Heist', "FrightFest 2018 Nominado Screen International's Rising Star Award Rising Star Await Further Instructions", 'Unrestricted View Film Festival 2021 Ganador Festival Prize Best Actor (Feature) Sorority']), ('George Appleby', 'NULL', ['Reparto', 'Especialistas', 'Reparto'], ['su papel en Juego de tronos', 'Puppet Master: Axis Termination', 'Ravenwolf Towers'], 'NULL'), ('Leslie Ash', '1960', ['Reparto', 'Producción', 'Reparto'], ['Men Behaving Badly', 'Quadrophenia', 'La maldición de la Pantera Rosa'], 'NULL'), ('Amelia Bath', 'NULL', ['Reparto', 'Reparto'], ['su papel en To Be Someone', 'Broke', 'Ibiza Weekender'], 'NULL'), ('Kim Benson', '1965', ['Reparto', 'Maquillaje', 'Reparto'], ['In Fabric', '2point4 Children', 'Miss the Kiss'], 'NULL'), ('Perry Benson', '1961', ['Reparto', 'Guion', 'Producción'], ['Sid y Nancy', 'This Is England', "You Rang, M'Lord?"], 'NULL'), ('Kerrieanne Booker', 'NULL', ['Reparto', 'Localización', 'Dirección de casting'], ['su papel en Accident Man', 'Street Blood', 'Dead Ringer'], 'NULL'), ('Ray Burdis', '1958', ['Reparto', 'Guion', 'Producción'], ['Operation Good Guys', 'The Wee Man', 'La toma final'], 'NULL'), ('Julian Clapton', 'NULL', ['Reparto', 'Reparto'], ['su papel en Best Geezer', 'Casting Kill', 'Daggers Inn'], 'NULL')], [('Sirin Giobbi', 'NULL', ['Reparto', 'Reparto'], ['su papel en Sevdim Seni Bir Kere', 'Beni Affet', 'Rafadan Tayfa Göbeklitepe'], 'NULL'), ('Levent Kol', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rüzgarli Sokak', 'Rafadan Tayfa Göbeklitepe', 'Rafadan Tayfa Dehliz Macerasi'], 'NULL'), ('Emine Sergen', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rafadan Tayfa Göbeklitepe', 'Rafadan Tayfa Dehliz Macerasi', 'Rafadan Tayfa Galaktik Tayfa'], 'NULL'), ('Yagmur Sergen', 'NULL', ['Reparto', 'Reparto'], ['su papel en Rafadan Tayfa Göbeklitepe', 'Rafadan Tayfa Dehliz Macerasi', 'Rafadan Tayfa Galaktik Tayfa'], 'NULL')], [('Voreas Faethon Aherusia', 'NULL', ['Reparto', 'Guion', 'Composición'], ['su papel en Aherusia: Prometheus', 'Papanisher', 'Santapocalypse'], 'NULL'), ('Thoth Amon', 'NULL', ['Reparto', 'Reparto'], ["su papel en Santa's Hunger"], 'NULL'), ('Stelios Antaras', 'NULL', ['Reparto', 'Reparto'], ["su papel en Santa's Hunger"], 'NULL'), ('Nick Chanakoulas', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Dr'], 'NULL'), ('Diamantis Dourdoumas', 'NULL', ['Reparto', 'Reparto'], ['su papel en Papanisher', "Santa's Hunger"], 'NULL'), ('Anna Fragkaki', 'NULL', ['Reparto', 'Maquillaje', 'Reparto'], ['su papel en With a Kiss I Die', "Santa's Hunger"], 'NULL'), ('Alexandros Gkiziotis', 'NULL', ['Reparto', 'Reparto'], ["su papel en Santa's Hunger"], 'NULL'), ('Christos Grammenos', 'NULL', ['Reparto', 'Reparto'], ['su papel en Santerminator', "Santa's Hunger"], 'NULL'), ('Chara Gri', 'NULL', ['Reparto', 'Reparto'], ["su papel en Santa's Hunger"], 'NULL'), ('George Karagiannis', 'NULL', ['Reparto', 'Reparto'], ['su papel en Devastated', 'Papanisher', 'Santapocalypse'], 'NULL')], [], [('Joey Rasdien', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en The Umbrella Men', 'Blitzpatrollie', 'Bunny Chow: Know Thyself'], 'NULL'), ('David Kau', 'NULL', ['Reparto', 'Dirección', 'Producción'], ['su papel en The First Last Tour', 'Taxi Ride', "Shampoo's Retirement Village"], 'NULL'), ('David Kibuuka', 'NULL', ['Guion', 'Producción', 'Reparto'], ['su papel en Las noticias de Jon Stewart', 'Bunny Chow: Know Thyself', 'Font'], ['Primetime Emmy Awards 2023 Ganador Primetime Emmy Outstanding Talk Series Las noticias de Jon Stewart Compartido con: Jennifer Flanz · Max Browning · Dan Amira · Jeff Gussow · Beth Shorr · Ramin Hedayati · Jill Katz · · · · · · · · · · 2023 Nominado Primetime Emmy Outstanding Writing for a Variety Series Las noticias de Jon Stewart Compartido con: Dan Amira · Lauren Sarver Means · Daniel Radosh · David Angelo · Nicole Conlan · Devin Delliquanti · · · · · · · · · · · · 2022 Nominado Primetime Emmy Outstanding Writing for a Variety Series Las noticias de Jon Stewart Compartido con: Dan Amira · Lauren Sarver Means · Daniel Radosh · David Angelo · Devin Delliquanti · Zach DiLanzo · · · · · · · · · · · · 2022 Nominado Primetime Emmy Outstanding Variety Talk Series Las noticias de Jon Stewart Compartido con: Trevor Noah · Jennifer Flanz · Jill Katz · Justin Melkmann · Ian Berger · Max Browning · Pamela DePace · · · · · · · · · · 2022 Nominado Primetime Emmy Outstanding Short Form Nonfiction or Reality Series The Daily Show: Between the Scenes Compartido con: Trevor Noah · Jennifer Flanz · Ramin Hedayati · Folake Ayiloge · Jocelyn Conn 5 más', 'Image Awards (NAACP) 2019 Ganador Image Award Outstanding Writing in a Comedy Series Las noticias de Jon Stewart For Alex Wagner (2018). Compartido con: Trevor Noah · Steve Bodow · Zhubin Parang · Dan Amira · Lauren Sarver Means · Daniel Radosh · · ·', 'PGA Awards 2023 Nominado PGA Award Outstanding Producer of Live Entertainment, Variety, Sketch, Standup & Talk Television Las noticias de Jon Stewart Compartido con: Juliet Werner · Matt Negrin · Aaron Perlman-Price · Ryan Middleton · Allison MacDonald · Kira Klang Hopf · · · · · · · · · · · · · · · · · · · · · 2021 Nominado PGA Award Outstanding Producer of Live Entertainment & Talk Television Las noticias de Jon Stewart Season 26 Compartido con: Trevor Noah · Jennifer Flanz · Jill Katz · Justin Melkmann · Max Browning · Eric Davies · Pamela DePace · · · · · · · · · · · · · · · · · · 2020 Nominado PGA Award Outstanding Producer of Live Entertainment & Talk Television Las noticias de Jon Stewart Season 25 Compartido con: Trevor Noah · Jennifer Flanz · Jill Katz · Justin Melkmann · Zhubin Parang · Jocelyn Conn · Max Browning · · · · · · · · · · · 2019 Nominado PGA Award Outstanding Producer of Live Entertainment & Talk Television Las noticias de Jon Stewart Season 24 Compartido con: Trevor Noah · Steve Bodow · Jennifer Flanz · Jill Katz · Justin Melkmann · Zhubin Parang · Max Browning · · · · · · · · · · ·', 'Writers Guild of America, USA 2024 Nominado WGA Award (TV) Comedy/Variety Talk Series Las noticias de Jon Stewart Compartido con: Dan Amira · Daniel Radosh · Lauren Sarver Means · David Angelo · Nicole Conlan · Devin Delliquanti · · · · · · · · · · · · · · · · · 2018 Nominado WGA Award (TV) Comedy/Variety Series Las noticias de Jon Stewart Compartido con: Dan Amira · David Angelo · Steve Bodow · Kashana Cauley · Devin Delliquanti · Zach DiLanzo · · · · · · · · · · · 2017 Nominado WGA Award (TV) Comedy/Variety Series Las noticias de Jon Stewart Compartido con: Dan Amira · Steve Bodow · Travon Free · Hallie Haglund · Matt Koff · Adam Lowitt · Dan McCoy · · · · · · · · · · · ·', 'South African Film and Television Awards 2009 Nominado SAFTA Golden Horn Best Achievement in Scriptwriting - TV Comedy Font Compartido con: Kagiso Lediga 2007 Nominado SAFTA Golden Horn Best Achievement in Script Writing - Feature Film Bunny Chow: Know Thyself']), ('Chris Forrest', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Blitzpatrollie', 'Footskating 101', 'ROSA: Republic of South Ah Sh**t!'], 'NULL'), ('Mel Miller', 'NULL', ['Reparto', 'Reparto'], ['su papel en Blitzpatrollie', 'Material', 'Kniediep'], 'NULL'), ('Kagiso Lediga', 'NULL', ['Producción', 'Reparto', 'Guion'], ['su papel en Cupido en acción', 'Queen Sono', 'The Pure Monate Show'], 'NULL'), ('Craig Urbani', 'NULL', ['Reparto', 'Reparto'], ['su papel en Los últimos días del crimen', 'Killsite y The Construct'], 'NULL'), ('Sans Moonsamy', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Red Room', "Who's My Daddy?", 'Blitzpatrollie'], 'NULL'), ('Quentin Krog', '1983', ['Equipos adicionales', 'Reparto', 'Dirección'], ['G'], 'NULL'), ('Kaseran Pillay', 'NULL', ['Reparto', 'Reparto'], ['su papel en Trackers', 'Blitzpatrollie', 'Inkabi'], 'NULL')], [('Steven Briggs', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Larry David (Curb Your Enthusiasm', 'Creedme', "Laff Mobb's Laff Tracks"], 'NULL'), ('Jasmine Wattar', 'NULL', ['Reparto', 'Producción', 'Guion'], ['su papel en Foil', 'Nelk Boys', 'The Things That Haunt Us'], 'NULL')], [('Laura Burns', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Headless Hipster', 'Peter and Jane Know Some of the Same People'], 'NULL'), ('Izad Etemadi', 'NULL', ['Reparto', 'Guion', 'Casting'], ['su papel en Orphan Black: Echoes', 'Ghosts', 'Revenge of the Black Best Friend'], 'NULL'), ('John Goodrich', 'NULL', ['Reparto', 'Reparto'], ['su papel en Pros & Cons', 'Bandidos', 'New Waterford Girl'], 'NULL')], [], [('Sonalii Castillo', 'NULL', ['Reparto', 'Producción', 'Guion'], ['The Outpost', 'Variant', 'Héroes'], ['One-Reeler Short Film Competition 2018 Ganador Special Mention June Voodoo Vanessa', 'Global Shorts, Los Angeles 2021 Ganador Honorable Mention September Kid', 'Independent Shorts Awards 2021 Nominado September Award Best Women Short Kid 2021 Nominado September Award Best Women Short Kid 2018 Ganador Platinum Award Best Thriller Short MAMBA 2018 Ganador Gold Award Best Actress MAMBA 2018 Ganador Platinum Award Best Thriller Short MAMBA Compartido con: Sam Puefua', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('April Rawlings', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Producción'], ['su papel en Mercy', 'CSI: Miami', 'Dahlias: Wild Card'], 'NULL'), ('Alyssa M. Jones', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dahlias: Wild Card'], 'NULL'), ('Audrey Lotspeich', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dahlias: Wild Card'], 'NULL'), ('Cain Vincent', 'NULL', ['Reparto', 'Reparto'], ['su papel en MAMBA', 'Dahlias: Wild Card'], 'NULL'), ('Jakob Bokulich', 'NULL', ['Reparto', 'Arte', 'Dirección'], ['su papel en Nine, Chinatown', 'Entangled'], 'NULL'), ('Joe Chacon', '1981', ['Reparto', 'Especialistas', 'Producción'], ['S'], ['Global Shorts, Los Angeles 2021 Ganador Award of Merit March The Black League of Superheroes Compartido con: Marcus Nel-Jamal Hamm · Steven Alan Davis', 'IndieX Film Festival 2021 Ganador Honorable Mention Best Fantasy Short The Black League of Superheroes Compartido con: Marcus Nel-Jamal Hamm · Steven Alan Davis', 'Los Angeles Cinematography AWARDS (LACA) 2021 Nominado June Award Best Cinematography of a Short Film The Black League of Superheroes Compartido con: Marcus Nel-Jamal Hamm · Steven Alan Davis', 'Hollywood Gold Awards 2021 Ganador June Award Best Action Short The Black League of Superheroes Compartido con: Marcus Nel-Jamal Hamm · Steven Alan Davis', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Christopher M. King', 'NULL', ['Reparto', 'Reparto'], ['su papel en Dahlias: Wild Card'], 'NULL'), ('Raphael Mihopoulos', 'NULL', ['Reparto', 'Especialistas', 'Reparto'], ['su papel en Dahlias: Wild Card', 'Lost Hills, California'], 'NULL')], [], [('Ry Barrett', 'NULL', ['Reparto', 'Guion', 'Producción'], ['su papel en Cult Hero', 'Still the Water', 'De naturaleza violenta'], ['Bare Bones International Film & Music Festival 2016 Nominado Bonehead Award Best Actor Save Yourself', 'Fantastic Planet Film Festival 2015 Ganador Festival Prize Best Male Performance The Demolisher', 'Buffalo Dreams Fantastic Film Festival 2017 Nominado Festival Prize Best Actor The Heretics 2017 Nominado Festival Prize Best Actor Art of Obsession', 'Toronto International Film and Video Awards 2011 Ganador TIFVA Award Best Actor Neverlost', 'Blood in the Snow Canadian Film Festival 2017 Ganador Bloodie Vanguard Award']), ('Tianna Nori', 'NULL', ['Reparto', 'Producción', 'Especialistas'], ['su papel en Letterkenny', 'Noel Next Door', 'Christmas Coach'], 'NULL'), ('Jessica Vano', 'NULL', ['Reparto', 'Producción', 'Especialistas'], ['su papel en The Hyperborean', 'Art of Obsession', 'The Demolisher'], 'NULL'), ('Duncan McLellan', 'NULL', ['Reparto', 'Especialistas', 'Reparto'], ['su papel en The Demolisher', 'Arrow', 'El hombre en el castillo'], 'NULL'), ('Gerrit Sepers', 'NULL', ['Reparto', 'Equipos adicionales', 'Sonido'], ['su papel en If a Tree Falls 2: Exist to Exit, If a Tree Falls', 'The Interview'], 'NULL'), ('Bruce Turner', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en The Wolf of Waubamik Woods', 'The RODE We Choose', 'Forbidden Science'], 'NULL'), ('Duane Frey', 'NULL', ['Reparto', 'Guion', 'Efectos especiales'], ['su papel en The Demolisher', 'Death on Scenic Drive', 'Haresapien'], 'NULL'), ('Andrew Bussey', 'NULL', ['Guion', 'Dirección', 'Reparto'], ['su papel en The Demolisher', 'Haresapien'], 'NULL'), ('Owen Fawcett', 'NULL', ['Reparto', 'Equipos adicionales', 'Reparto'], ['su papel en The Demolisher', 'The Umbrella Project', 'The Way Home'], 'NULL'), ('Rich Piatkowski', 'NULL', ['Reparto', 'Dirección', 'Guion'], ['su papel en Dead Rush', 'Carrera hacia la Casa Blanca', 'Capture Kill Release'], 'NULL')], [('Dwight Angelito', 'NULL', ['Efectos visuales', 'Reparto', 'Animación'], ['su papel en Mad Max: Furia en la carretera', 'Godzilla', 'X-Men: Días del futuro pasado'], 'NULL'), ('Steve Boshear', 'NULL', ['Edición', 'Guion', 'Dirección'], ['su papel en Weird City: Street Food', 'Weird City: Power Gamers', 'Weird City: Beast Mode'], 'NULL'), ('Reggie Johnson II', 'NULL', ['Dirección', 'Guion', 'Composición'], ['su papel en Algor, The Hill (Proof of ConceptPractice'], 'NULL')], [('Reidar Palmgren', '1966', ['Reparto', 'Reparto'], ['Jäähyväiset ilman kyyneleitä', 'Onnellisia ihmisiä', 'Rajavartija'], 'NULL'), ('Leila Väisänen', 'NULL', ['Reparto', 'Reparto'], ['su papel en Miesten juoma', 'Uusi päivä'], 'NULL'), ('Jukka Nieminen', 'NULL', ['Reparto', 'Guion', 'Dirección'], ['su papel en Prioni', 'Prioni II', '1964'], 'NULL')], [], [('Johnny Damon', '1973', ['Reparto', 'Equipos adicionales', 'Reparto'], ['Amor en juego', 'El club de medianoche', 'TNA iMPACT! Wrestling'], 'NULL'), ('Vivica A. Fox', '1964', ['Reparto', 'Producción', 'Dirección'], ['Independence Day', 'Kill Bill: Volumen 1', 'Missing'], ['Academy of Science Fiction, Fantasy & Horror Films, USA 1997 Nominado Saturn Award Best Supporting Actress Independence Day', 'Acapulco Black Film Festival 1998 Ganador Black Film Award Best Actress Líos de familia', 'BET Awards 2004 Nominado BET Award Best Actress Kill Bill: Volumen 1 2002 Nominado BET Award Best Actress Boat Trip: Este barco es un peligroMejor en el cieloDos para jugar', 'Image Awards (NAACP) 2024 Nominado Image Award Outstanding Directing in a Television Movie or Special First Lady of BMF: The Tonesa Welch Story 2008 Nominado Image Award Outstanding Supporting Actress in a Comedy Series Larry David (Curb Your Enthusiasm) 2006 Ganador Image Award Outstanding Actress in a Drama Series Missing 2005 Nominado Image Award Outstanding Actress in a Drama Series Missing 2003 Nominado Image Award Outstanding Actress in a Motion Picture Juwanna Mann 4 más', 'MTV Movie + TV Awards 1998 Nominado MTV Movie Award Best Female Performance Líos de familia 1997 Nominado MTV Movie Award Best Breakthrough Performance Independence Day 1997 Ganador MTV Movie Award Best Kiss Independence Day Compartido con: Will Smith']), ('Leeza Gibbons', '1957', ['Producción', 'Reparto', 'Producción'], ['The Apprentice', 'Leeza', 'Entertainment Tonight'], ['CableACE Awards 1993 Nominado CableACE Magazine Host Growing Up Together', 'Daytime Emmy Awards 2014 Nominado Daytime Emmy Outstanding Host in a Lifestyle/Travel Program America Now Compartido con: Bill Rancic 2013 Ganador Daytime Emmy Outstanding Host in a Lifestyle/Travel Program My Generation 1999 Nominado Daytime Emmy Outstanding Talk Show Leeza Compartido con: Jose Pretlow · Donna Harrison · Jill Mullikin-Bates · Rudy Guido · Tracy Mazuer · Marilyn Zielinski · · · · · · · 1999 Nominado Daytime Emmy Outstanding Talk Show Host Leeza 1998 Nominado Daytime Emmy Outstanding Talk Show Leeza Compartido con: Donna Harrison · Rudy Guido · Jill Mullikin-Bates · Kathy Giaconia · Shantel Klinger · Julie Laughlin-Jones · · · 7 más', 'Walk of Fame 1998 Ganador Star on the Walk of Fame Television On 7 May 1998. At 7021 Hollywood Blvd.', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Brandi Glanville', '1972', ['Reparto', 'Reparto'], ['Los juegos del resacón', 'Sharknado: Que la 4ª te acompañe', 'Desaparecida a los 17'], 'NULL'), ('Kate Gosselin', '1975', ['Producción', 'Producción'], ['Jon & Kate Plus 8', 'Surviving Sextuplets and Twins', 'Kate Plus 8'], 'NULL'), ('Gilbert Gottfried', '1955', ['Producción', 'Reparto', 'Guion'], ['Aladdín', 'Las aventuras de Ford Fairlane', 'Mira quién habla también'], ['CableACE Awards 1988 Nominado ACE Writing a Comedy Special Gilbert Gottfried... Naturally', "Daytime Emmy Awards 2009 Nominado Daytime Emmy Outstanding New Approaches - Daytime Children's Cyberchase Compartido con: Sandra Sheppard · Frances Nankin · Ellen Doherty · David Hirmes · Tanner Vea · Denyse Ouellette · ·", 'Razzie Awards 1991 Nominado Razzie Award Worst Supporting Actor Las aventuras de Ford FairlaneMira quién habla tambiénEste chico es un demonio', "Rondo Hatton Classic Horror Awards 2021 Ganador Rondo Statuette Best Podcast Gilbert Gottfried's Amazing Colossal Podcast! With Frank Santopadre Compartido con: Frank Santopadre 2019 Ganador Rondo Statuette Best Multi-Media Website Gilbert Gottfried's Amazing Colossal Podcast! With Frank Santopadre", 'Horror Host Hall Of Fame 2015 Ganador Horror Host Hall Of Fame Plaque Horror Host Hall Of Fame USA Up All Night']), ('Sig Hansen', '1966', ['Equipos adicionales', 'Producción', 'Reparto'], ['Cars 2', 'Deadliest Catch: The Viking Returns', 'Deadliest Catch: The Bait'], 'NULL'), ('Shawn Johnson', '1992', ['Reparto', 'Reparto'], ['Vida secreta de una adolescente', 'Beijing 2008: Games of the XXIX Olympiad', 'Dancing with the Stars'], ['Teen Choice Awards 2009 Nominado Teen Choice Award Choice TV: Female Reality/Variety Star Dancing with the Stars 2009 Nominado Teen Choice Award Choice Celebrity Dancer', 'Young Hollywood Awards 2009 Ganador Young Hollywood Award Athlete', 'ESPY Awards 2009 Ganador ESPY Best U.S. Female Olympian Beijing 2008: Games of the XXIX Olympiad Gymnastic', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Kevin Jonas', '1987', ['Reparto', 'Música', 'Producción'], ['Camp Rock', 'Jonas', 'Aquamarine'], ['Teen Choice Awards 2013 Ganador Teen Choice Award Choice TV: Male Reality/Variety Star Married to Jonas 2010 Ganador Teen Choice Award Choice TV Actor: Comedy Jonas Compartido con: Joe Jonas · Nick Jonas 2009 Ganador Teen Choice Award Choice TV Actor: Comedy Jonas Compartido con: Nick Jonas · Joe Jonas 2008 Nominado Teen Choice Award Choice Fanatic Fans Compartido con: Nick Jonas · Joe Jonas 2008 Ganador Teen Choice Award Choice Male Hottie Compartido con: The Jonas Brothers · Joe Jonas · Nick Jonas 1 más', 'Astra Awards 2009 Nominado Astra Award Favourite International Personality or Actor Camp Rock Compartido con: Joe Jonas · Nick Jonas', 'Nickelodeon Kids Choice Awards, Mexico 2010 Nominado Nickelodeon Kids Choice Award Favorite International TV Star Jonas Compartido con: Nick Jonas · Joe Jonas', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir', 'Más de esta persona']), ('Keshia Knight Pulliam', '1979', ['Reparto', 'Dirección', 'Producción'], ['La hora de Bill Cosby', 'House of Payne (Serie de TV', 'Madea Goes to Jail'], ['Primetime Emmy Awards 1986 Nominado Primetime Emmy Outstanding Supporting Actress in a Comedy Series La hora de Bill Cosby For playing "Rudy Huxtable".', "Image Awards (NAACP) 2012 Ganador Image Award Outstanding Supporting Actress in a Comedy Series House of Payne (Serie de TV) 2011 Nominado Image Award Outstanding Supporting Actress in a Comedy Series House of Payne (Serie de TV) 2010 Ganador Image Award Outstanding Supporting Actress in a Comedy Series House of Payne (Serie de TV) 2009 Ganador Image Award Outstanding Supporting Actress in a Comedy Series House of Payne (Serie de TV) 2003 Nominado Image Award Outstanding Performance in a Youth/Children's Series or Special What About Your Friends: Weekend Getaway 2 más", "Kids' Choice Awards, USA 1991 Ganador Blimp Award Favorite TV Actress La hora de Bill Cosby", 'NAMIC Vision Awards 2009 Nominado Vision Award Best Performance - Comedy House of Payne (Serie de TV)', "People's Choice Awards, USA 1988 Ganador People's Choice Award Favorite Young TV Performer Tied with Kirk Cameron."])], [('Karine Vanasse', '1983', ['Reparto', 'Producción', 'Banda sonora'], ['Politécnico', 'Cardinal', 'Séraphin: un homme et son péché'], ['Genie Awards 2010 Ganador Genie Best Performance by an Actress in a Leading Role Politécnico 2004 Nominado Genie Best Performance by an Actress in a Leading Role Séraphin: un homme et son péché', 'Jutra Awards 2008 Nominado Jutra Best Actress (Meilleure Actrice) Ma fille, mon ange 2003 Ganador Jutra Best Actress (Meilleure Actrice) Séraphin: un homme et son péché 2000 Ganador Jutra Best Actress (Meilleure Actrice) Emporte-moi', 'Namur International Festival of French-Speaking Film 1999 Ganador Golden Bayard Best Actress (Meilleure Comédienne) Emporte-moi', 'Toronto International Film Festival 1999 Ganador Best Canadian First Feature Film - Special Jury Congratulation Emporte-moi', 'Valladolid International Film Festival 1999 Ganador Youth Jury Award Competition Emporte-moi Compartido con: Léa Pool']), ('Eric Cantona', '1966', ['Reparto', 'Dirección', 'Producción'], ['Elizabeth', 'Venganza fatal', 'Buscando a Eric'], 'NULL'), ('Mehdi Nebbou', '1971', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Edición'], ['Red de mentiras', 'Múnich', 'English Vinglish'], 'NULL'), ('Aurélien Recoing', '1958', ['Reparto', 'Dirección', 'Sonido'], ['La vida de Adèle', 'El empleo del tiempo', '¡Que te calles!'], 'NULL'), ('Karina Testa', '1981', ['Reparto', 'Reparto'], ['Frontera(s', 'Muñecas y ángeles', 'Odysseus'], 'NULL'), ('Bruno Todeschini', '1962', ['Reparto', 'Reparto'], ['Su hermano', 'Vete a saber', '1 Journée'], ['César Awards, France 2004 Nominado César Best Actor (Meilleur acteur) Su hermano', 'European Film Awards 2003 Nominado European Film Award European Actor Su hermano', 'Swiss Film Prize 2008 Nominado Swiss Film Prize Best Actor (Bester Darsteller) 1 Journée', 'Lumiere Awards, France 2004 Ganador Lumiere Award Best Actor (Meilleur comédien) Su hermano', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ('Maxim Roy', '1972', ['Reparto', 'Producción', 'Dirección'], ['La Facción Octubre', 'Bad Blood', 'Shadowhunters: The Mortal Instruments'], ["Jutra Awards 2009 Nominado Jutra Best Supporting Actress (Meilleure Actrice de Soutien) Adam's Wall", 'Catalina Film Festival 2021 Ganador Best Actor, Female Best Actress Wichita', 'Canadian Screen Awards, CA 2018 Nominado Canadian Screen Award Best Lead Actress, Drama Program or Limited Series Bad Blood 2015 Nominado Canadian Screen Award Best Performance by an Actress in a Featured Supporting Role in a Dramatic Program or Series 19-2 Episodes: "Islands (2014)" and "Winter (2014)"', 'Top Shorts Film Festival 2021 Ganador Honorable Mention Actress Wichita', 'Global Shorts, Los Angeles 2020 Ganador Global Shorts Award Best Actress Wichita']), ('Niseema Theillaud', 'NULL', ['Reparto', 'Reparto'], ['su papel en Pequeñas mentiras sin importancia', 'Switch', 'No soy un hombre fácil'], 'NULL'), ('Sophie Faucher', '1958', ['Reparto', 'Guion', 'Reparto'], ['Laurence Anyways', 'Bethune: La forja de un héroe', 'Switch'], 'NULL'), ('Stéphan Guérin-Tillié', '1972', ['Reparto', 'Guion', 'Dirección'], ["Juste une question d'amour", 'Edy', 'Sam'], 'NULL')], [('Juan Javier Aguirre', 'NULL', ['Edición', 'Dirección', 'Guion'], ['su papel en Unsub', 'Hey Ellie', 'Saint Micheal! Avenge Me!'], 'NULL'), ('Keylis Guirguis', 'NULL', ['Reparto', 'Reparto'], ['su papel en Saint Micheal! Avenge Me!', 'A Phone Call from Paradise'], 'NULL'), ('Chad Hackworth', 'NULL', ['Reparto', 'Reparto'], ['su papel en Saint Micheal! Avenge Me!'], 'NULL'), ('Cat Van Rae', 'NULL', ['Reparto', 'Reparto'], ['su papel en Acting Out', '-', 'Saint Micheal! Avenge Me!', 'Thought and Prayers with Hope and Grace'], 'NULL'), ('Juan Manuel Villegas', 'NULL', ['Reparto', 'Cámara y electricidad', 'Dirección'], ['su papel en We Missed You, Error', 'Project Re: Fate'], 'NULL')], [('Vikram', '1966', ['Reparto', 'Equipos adicionales', 'Música'], ['I', 'Anniyan', 'Raavanan'], ['National Film Awards, India 2004 Ganador Silver Lotus Award Best Actor Pithamagan For his powerful performance in a difficult and demanding role.', 'Asianet Film Awards 2016 Ganador Asianet Film Award Most Popular Tamil Actor I 2005 Ganador Asianet Film Award Special Jury Award Anniyan', 'Filmfare Awards South 2017 Nominado Filmfare Award - Tamil Film Industry Best Actor Iru Mugan 2016 Ganador Filmfare Award - Tamil Film Industry Best Actor I 2013 Nominado Filmfare Award - Tamil Film Industry Best Actor Thaandavam 2012 Nominado Filmfare Award - Tamil Film Industry Best Actor Deiva Thirumagal 2012 Ganador Filmfare Award - Tamil Film Industry Critics Award for Best Actor Deiva Thirumagal 7 más', 'South Indian International Movie Awards 2023 Nominado SIIMA - Tamil Best Actor in a Leading Role Mahaan 2023 Nominado SIIMA - Tamil Best Actor in a Leading Role Ponniyin Selvan: Part One 2017 Nominado SIIMA - Tamil Best Actor in a Leading Role Iru Mugan 2016 Ganador SIIMA - Tamil Best Actor in a Leading Role I 2012 Nominado SIIMA - Tamil Best Actor in a Leading Role Deiva Thirumagal 1 más', "Stardust Awards, India 2011 Nominado Jury's Choice Superstar of Tomorrow - Male Raavan"]), ('Amy Jackson', '1992', ['Reparto', 'Banda sonora', 'Reparto'], ['2'], ['Filmfare Awards South 2013 Nominado Filmfare Award - Tamil Film Industry Best Supporting Actress Thaandavam', 'South Indian International Movie Awards 2016 Nominado SIIMA - Tamil Best Actress in a Leading Role I 2015 Ganador Generation Next Award Stylish Youth Icon of South Indian Cinema (Female)', 'Vijay Awards 2011 Nominado Jury Award Best Debut Actress Madrasapattinam', 'London Asian Film Festival 2018 Ganador Festival Award Bright Star Making Waves Globally Boogie Man', 'Ananda Vikatan Cinema Awards 2011 Ganador Ananda Vikatan Cinema Award Best Debut Actress Madrasapattinam']), ('Suresh Gopi', '1958', ['Reparto', 'Música', 'Edición'], ['Kaliyattam', 'I', 'Commissioner'], 'NULL'), ('Upen Patel', '1980', ['Reparto', 'Reparto'], ['36 China Town', 'Ajab Prem Ki Ghazab Kahani', 'I'], ['Awards of the International Indian Film Academy 2007 Ganador Popular Award Best Debut Actor 36 China Town', 'Zee Cine Awards 2007 Ganador Popular Award Best Debut - Male 36 China Town', 'Bollywood Awards 2007 Ganador Bollywood Movie Award Best Debutant-Male 36 China Town', "People's Choice Awards, India 2006 Nominado People's Choice Award Best Male Debut 36 China Town", 'Annual Central European Bollywood Awards, India 2007 Nominado Popular Award Breakthrough Role (Actor) 36 China Town']), ('Santhanam', '1980', ['Reparto', 'Producción', 'Guion'], ['Enthiran', 'Boss Engira Baskaran', 'Velayudham'], ['Filmfare Awards South 2013 Nominado Filmfare Award - Tamil Film Industry Best Supporting Actor Oru Kal Oru Kannadi 2012 Nominado Filmfare Award - Tamil Film Industry Best Supporting Actor Siruthai 2012 Nominado Filmfare Award - Tamil Film Industry Best Supporting Actor Deiva Thirumagal 2011 Nominado Filmfare Award - Tamil Film Industry Best Supporting Actor Boss Engira Baskaran', 'South Indian International Movie Awards 2016 Nominado SIIMA - Tamil Best Comic Actor Vasuvum Saravananum Onna Padichavanga 2014 Nominado SIIMA - Tamil Best Comedian Theeya Velai Seiyyanum Kumaru 2013 Nominado SIIMA - Tamil Best Comedian Oru Kal Oru Kannadi 2012 Ganador SIIMA - Tamil Best Comedian Vaanam', 'Vijay Awards 2012 Ganador Jury Award Best Comedian Siruthai 2012 Nominado Jury Award Best Comedian Deiva Thirumagal 2011 Ganador Jury Award Best Comedian Boss Engira Baskaran 2010 Ganador Jury Award Best Comedian Siva Manasula Sakthi', 'Edison Awards India 2011 Ganador Merit Award Best Comedian Boss Engira Baskaran', 'Ananda Vikatan Cinema Awards 2013 Ganador Ananda Vikatan Cinema Award Best Comedian - Male Oru Kal Oru Kannadi 2012 Ganador Ananda Vikatan Cinema Award Best Comedian - Male Deiva ThirumagalVelayudham 2011 Ganador Ananda Vikatan Cinema Award Best Comedian - Male Boss Engira Baskaran']), ('Ramkumar Ganesan', 'NULL', ['Reparto', 'Reparto'], ['su papel en I', 'Chandramukhi', 'Boomerang'], 'NULL'), ('Ojas Rajani', 'NULL', ['Reparto', 'Maquillaje', 'Reparto'], ['su papel en I', 'Fashion', 'Beyond Bollywood'], 'NULL'), ('Kamaraj', 'NULL', ['Reparto', 'Reparto'], ['su papel en Mersal', 'I', 'Yutha Satham'], 'NULL'), ('Mohan Kapur', '1965', ['Reparto', 'Reparto'], ['The Marvels', 'Ms'], 'NULL'), ('Powerstar Srinivasan', 'NULL', ['Reparto', 'Dirección', 'Reparto'], ['su papel en I', 'Kanna Laddu Thinna Aasaiya', 'Vizhi Moodi Yosithaal'], 'NULL')], [], [('C.J. Dawson', 'NULL', ['Reparto', 'Sonido', 'Dirección'], ['su papel en Jack Walks Down a Hilly Road', "I'm Asking the Questions Here!", 'End of the Line'], 'NULL'), ('Cal Gunderson', 'NULL', ['Reparto', 'Dirección de la segunda unidad o asistencia a dirección', 'Composición'], ['su papel en Cigarettes', 'Virtually in Love', 'American Ninja Warrior'], 'NULL')], [('Yoson An', 'NULL', ['Reparto', 'Producción', 'Dirección'], ['su papel en Mulán', 'Las luminarias', 'Fresh Eggs'], 'NULL'), ('Awhi Dansey-Smith', 'NULL', ['Reparto', 'Reparto'], ['su papel en Weirdoes'], 'NULL'), ('Angella Dravid', 'NULL', ['Reparto', 'Guion', 'Reparto'], ['su papel en Golden Boy', 'Weirdoes', 'Under the Vines'], 'NULL'), ('Rebekkah Farrell', 'NULL', ['Reparto', 'Reparto'], ['su papel en Ash vs'], 'NULL'), ('Lara Fischel-Chisholm', 'NULL', ['Reparto', 'Equipos adicionales', 'Dirección'], ['su papel en Dynamotion: The Rainbow Dance', 'Dynamotion: Eurotrash', 'Movement Study 1'], 'NULL'), ('Lucinda Hare', 'NULL', ['Reparto', 'Maquillaje', 'Guion'], ['su papel en Auckward Love', 'La cura', 'This Giant Papier-Mâché Boulder Is Actually Really Heavy'], 'NULL'), ('Renee Lyons', 'NULL', ['Reparto', 'Guion', 'Casting'], ['su papel en M3GAN', 'Power Rangers Ninja Steel', "The Great Maiden's Blush"], 'NULL'), ('Roberto Nascimento', '1981', ['Producción', 'Dirección', 'Reparto'], ['Sui Generis', 'The Adventures of Suzy Boon', 'Sky Tower'], ["Show Me Shorts Film Festival 2022 Ganador People's Choice Atali'i O Le Crezent (Sons of the Crezent) Compartido con: Valentine Taylor · Perry Bradley · Belinda Bradley", 'Melbourne Web Fest 2022 Nominado Festival Award Best International Drama Sui Generis 2019 Nominado Best of Spotlight Spotlight Sui Generis 2018 Ganador Best of Spotlight Spotlight Sui Generis', '48 Hours, New Zealand 2022 Nominado NZ Award Incredibly Strange Ruthless Compartido con: Louis Mendiola 2009 Nominado NZ Award V Best Actor Gruff', 'Iris Prize Festival 2023 Nominado Iris Prize Iris Prize Boyfriend Compartido con: Josh McKenzie', 'DC Web Fest 2022 Ganador Bronze Web Series Boyfriend Compartido con: Josh McKenzie · Márcia Sasano 2019 Ganador Gold Web Series Sui Generis Compartido con: Gabriel Lunte · Jonathan Potton · Milon Tesiram 2019 Ganador Social Impact Award Web Series Sui Generis Compartido con: Jonathan Potton · Gabriel Lunte · Milon Tesiram']), ('Chris Parker', 'NULL', ['Guion', 'Reparto', 'Producción'], ['su papel en Double Parked', 'Nude Tuesday', 'Funny Girls'], 'NULL'), ('Jodie Rimmer', '1974', ['Reparto', 'Equipos adicionales', 'Reparto'], ['El refugio de mi padre', 'Turbo Kid', 'The Strip'], 'NULL')], [('Ibbi El Hani', 'NULL', ['Reparto', 'Reparto'], ['su papel en Kidnapping', 'Que viene el lobo', 'Shrouded Destiny: A Star Wars Long Tale'], 'NULL'), ('Christian Bogetoft', 'NULL', ['Producción', 'Guion', 'Dirección'], ['su papel en Portræt', 'Kervorum', 'Fravær af Kærlighed'], 'NULL')], [('Pooky Quesnel', '1966', ['Reparto', 'Guion', 'Reparto'], ['Orgullo + Prejuicio + Zombis', 'Doctors', 'Grandes esperanzas de Mike Newell'], 'NULL'), ('Jonnie Hurn', '1971', ['Reparto', 'Producción', 'Guion'], ['Love, Death & Robots', 'In Circles', 'Umbrage'], ['Santa Cruz Film Festival 2010 Ganador Jury Award Best Feature Film Do Elephants Pray? Compartido con: Paul Hills', 'Raindance Film Festival 2011 Ganador Jury Prize Best Micro Budget Feature Monk3ys Compartido con: Drew Cullingham · James Fisher', 'Mexico International Film Festival 2010 Ganador Silver Palm Award Feature Film Do Elephants Pray? Compartido con: Paul Hills', "Irina Palm d'Or 2013 Nominado Irina Palm Worst British Actor Do Elephants Pray?", 'International Film Festival ASTERFEST 2013 Ganador Special Jury Mention Best Short King John Act IV Scene I Compartido con: John Last']), ('Sebastian Street', 'NULL', ['Reparto', 'Producción', 'Equipo ejecutivo'], ['su papel en Tomorrow', 'I Am Henry', 'Tiempo de héroes'], ['Savannah Film Festival 2018 Ganador Best Narrative Feature Best Film Tomorrow Compartido con: Stuart Brennan · Martha Pinson · Dean M. Woodford', 'Napa Valley Film Festival 2018 Ganador Audience Award Favorite Actor Tomorrow 2018 Ganador Jury Award Best Narrative Feature Tomorrow Compartido con: Stuart Brennan · Dean M. Woodford 2018 Ganador Audience Award Favorite Narrative Feature Tomorrow Compartido con: Stuart Brennan · Dean M. Woodford', 'International Film Festival of Wales 2011 Ganador Judges Award Best Actor Stag Night of the Dead', 'Los Angeles Cinema Festival of Hollywood, US 2016 Ganador Best Actor Best Male Actor I Am Henry', 'LA Indie Film Fest 2018 Ganador LAIFF Award Best Actor Yellow Jacket']), ('Ian Manson', '1969', ['Reparto', 'Producción', 'Guion'], ['The Final 45, In Circles', 'United in Lockdown'], 'NULL'), ('Louis Holland', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Crown', 'Riding Liberty'], 'NULL'), ('Joshua Marshall', 'NULL', ['Reparto', 'Reparto'], ['su papel en Riding Liberty'], 'NULL'), ('Dale Curran', 'NULL', ['Reparto', 'Reparto'], ['su papel en Riding Liberty'], 'NULL'), ('David Fensom', 'NULL', ['Reparto', 'Reparto'], ['su papel en Riding Liberty'], 'NULL'), ('Charlie Hancock', 'NULL', ['Reparto', 'Reparto'], ['su papel en Riding Liberty'], 'NULL'), ('Pam Coningsby', 'NULL', ['Reparto', 'Reparto'], ['su papel en Riding Liberty'], 'NULL')], [('Robert Follin', '1971', ['Especialistas', 'Reparto', 'Equipos adicionales'], ['Vikingos', 'El último rey', 'Viking Destiny'], 'NULL'), ('Kenneth Wolf Andersen Haugen', '1989', ['Reparto', 'Especialistas', 'Producción'], ['El último superviviente', 'De Gjenvendte', 'Kuppel 16'], 'NULL'), ('Arben Bala', 'NULL', ['Reparto', 'Reparto'], ['su papel en Uno tras otro', 'La noche del lobo', 'Reprise'], 'NULL'), ('Veera Seppälä', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en El último superviviente', 'Koskinen', 'Piiritys'], 'NULL'), ('Mikael Aron', 'NULL', ['Especialistas', 'Reparto', 'Producción'], ['su papel en Christmas Cookies', 'El último superviviente', 'Våre Små Liv'], 'NULL'), ('Silje Landrø Botten', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en Trol', "Brother's Revenge", 'El abismo de Kiruna'], 'NULL'), ('Stian Buland Lie', 'NULL', ['Reparto', 'Especialistas', 'Efectos especiales'], ['su papel en El último superviviente', "Brother's Revenge", '17'], 'NULL'), ('Simen Øverås Andersen', '1995', ['Reparto', 'Reparto'], ["Brother's Revenge"], 'NULL'), ('Oskar Lunde', 'NULL', ['Especialistas', 'Reparto', 'Especialistas'], ['su papel en El último superviviente', "Brother's Revenge", 'Stuntfarmen'], 'NULL'), ('Preben Finsås', 'NULL', ['Reparto', 'Reparto'], ["su papel en Brother's Revenge"], 'NULL')], [('Calyx Passailaigue', 'NULL', ['Dirección', 'Producción', 'Guion'], ['su papel en Zeen?', 'Speak', 'Scarlet'], ['Alternative Film Festival 2018 Nominado Best Comedy Short Film Scarlet Compartido con: Delaney Novinka', 'Couch Film Festival 2023 Nominado Best Long Short Film 20:01 to 45:00 Minutes The Injustices 2022 Nominado Best Comedy 20:01 to 45:00 minutes Zeen? Compartido con: Richard Walters 2022 Nominado Best Actor 20:01 - 45:00 Minutes Zeen? 2022 Nominado Best of the Festival Best of The Festival (Short Film) Men at War', 'RUFF: Ryerson University Film Festival 2017 Nominado RUFF Award Best Fiction Film Scarlet Compartido con: Delaney Novinka', 'Toronto Arthouse Film Festival 2020 Ganador Best Toronto Based Filmmaker Zeen?', 'Trinidad and Tobago Film Festival 2020 Ganador Best Film Best Medium Length Narrative Film Zeen? Compartido con: Richard Walters']), ('Andon Campbell', 'NULL', ['Reparto', 'Efectos visuales', 'Equipos adicionales'], ['su papel en Descension Part II', 'Descension', 'The Injustices'], 'NULL'), ('Tarick Glancy', 'NULL', ['Reparto', 'Reparto'], ['su papel en Righteous Sinner', 'Because We Are', 'The Transformation of Ferlin Rusty'], 'NULL'), ('Angela Kaiser', 'NULL', ['Reparto', 'Reparto'], ['su papel en The Ghost Is a Lie, Natanael: The Crossing', 'No More Stolen Afternoons'], 'NULL'), ('Yannick Francis', 'NULL', ['Reparto', 'Reparto'], ['su papel en Zeen?'], 'NULL'), ('Jacob Currie', 'NULL', ['Guion', 'Reparto', 'Guion y continuidad'], ['su papel en Wayne', 'What Other People Say', 'Scarlet'], 'NULL'), ('Gary Scott', 'NULL', ['Reparto', 'Reparto'], ["su papel en She's Still", 'Zeen?'], 'NULL'), ('Danny Roberts', 'NULL', ['Reparto', 'Reparto'], ['su papel en Zeen?'], 'NULL'), ('Astrid Harrison', 'NULL', ['Arte', 'Reparto', 'Producción'], ['su papel en Ariana', 'Wayne', 'Scarlet'], 'NULL')], [('Cynthia Rothrock', '1957', ['Reparto', 'Producción', 'Dirección'], ['The Martial Arts Kid', 'In Search of the Last Action Heroes', 'Diary of a Lunatic'], 'NULL'), ('Costas Mandylor', '1965', ['Reparto', 'Reparto'], ['Saw V', 'Saw VI', 'Saw IV'], ['MTV Movie + TV Awards 2009 Nominado MTV Movie Award Best Fight Saw V Compartido con: Scott Patterson', 'Screen Actors Guild Awards 1996 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Picket Fences Compartido con: Amy Aquino · Kathy Baker · Don Cheadle · Kelly Connell · Fyvush Finkel · Lauren Holly · Marlee Matlin · · · · 1995 Nominado Actor Outstanding Performance by an Ensemble in a Drama Series Picket Fences Compartido con: Kathy Baker · Don Cheadle · Holly Marie Combs · Kelly Connell · Robert Cornthwaite · Fyvush Finkel · · · · · ·', 'Maverick Movie Awards 2009 Nominado Maverick Movie Award Best Actor: Feature Emma Blue', "Buffalo Dreams Fantastic Film Festival 2020 Nominado Dreamer Award Outstanding Actor in a Feature Daddy's Girl", 'Festival of Serbian Fantastic Film (FSFF) 2020 Ganador Oscull Best Supporting Actor C.L.E.A.N.']), ('Nina Bergman', 'NULL', ['Reparto', 'Música', 'Producción'], ['su papel en Hell Hath No Fury', 'Doom: Aniquilación', 'Call of Duty: Black Ops 4'], 'NULL'), ('Al Sapienza', 'NULL', ['Reparto', 'Producción', 'Reparto'], 'NULL', 'NULL'), ('Patrick Kilpatrick', 'NULL', ['Reparto', 'Producción', 'Guion'], ['Minority Report', 'Libertad para morir', 'El último hombre'], ['Downtown Film Festival Los Angeles 2022 Ganador Festival Award Best Ensemble Cast Catalyst Compartido con: Michael Roark · David Bianchi · Melanie Liburd · Noel Gugliemi · Jermaine Love · Chris Michael Wood ·', 'Actors Awards, Los Angeles 2022 Ganador Actors Award Best ensemble Borrowed Time III Compartido con: Alan Delabie · Eric Roberts · Costas Mandylor · Matthias Hues · Christina Okolo · Mark Stas · · · 2022 Ganador Actors Award Best Villain Borrowed Time III 2022 Nominado Annual Actors Award Best Ensemble of the Year Borrowed Time III borrowed time III Compartido con: Alan Delabie · Eric Roberts · Costas Mandylor · Matthias Hues', 'Dreamachine International Film Festival 2019 Ganador Annual Event Award Lifetime Achievement Award', 'Vegas Movie Awards 2022 Ganador Award of Excellence Best Ensemble Borrowed Time III Compartido con: Alan Delabie · Eric Roberts · Christina Okolo · Theresa Kassa · Costas Mandylor · Louis Mandylor · · ·', 'New York International Film Awards 2022 Ganador March Monthly Award Best Ensemble Cast Borrowed Time III Compartido con: Alan Delabie · Eric Roberts · Louis Mandylor · Costas Mandylor · Matthias Hues · Christina Okolo · · ·']), ('Olivier Gruner', '1960', ['Reparto', 'Producción', 'Especialistas'], ['Sector 4', 'Fuerza de choque', 'Re-Generator'], 'NULL'), ('Mike Ferguson', '1971', ['Reparto', 'Especialistas', 'Producción'], ['Showdown at the Grand', 'Amityville Uprising', 'Static Codes'], ['Accolade Competition 2022 Ganador Accolade Global Film Competiton Ensemble Cast Amber Road Compartido con: Rachel Riley · Elissa Dowling · William McNamara · Janet Wang · Tom Sizemore · Robert LaSardo · · · · · · · · · · · · ·', 'IndieX Film Festival 2019 Ganador August Award Best Supporting Actor Nowhere to Run', 'Hollywood Blood Horror Festival 2022 Ganador October Award Best Acting Ensemble Amber Road Compartido con: Tom Sizemore · William McNamara · Vincent M. Ward · BJ Mezek · Vernon Wells · Rachel Riley · · · · · · · ·', 'Vegas Movie Awards 2022 Ganador Award of Excellence Best Ensemble Amber Road Compartido con: William McNamara · Janet Wang · Rachel Riley · Elissa Dowling · Jed Rowen · Tom Sizemore · · · · · · · ·', 'Contribuir a esta página Sugerir un cambio o añadir el contenido que falta Consulta nuestra guía para actualizar los premios Más información acerca de cómo contribuir']), ("Quinton 'Rampage' Jackson", '1978', ['Reparto', 'Producción', 'Reparto'], ['El equipo A', 'El vagón de la muerte', 'Muere otra vez'], 'NULL'), ('Robert Miano', '1942', ['Reparto', 'Producción', 'Guion'], ['Fast & Furious: Aún más rápido', 'Un perdedor con suerte', 'Donnie Brasco'], ['ITN Distribution Film Festival 2014 Ganador Festival Award Life Time Achievement Award', 'Golden Door International Film Festival of Jersey City 2014 Nominado Festival Award Best Supporting Actor in a Feature Broken Blood', "Anatomy Crime and Horror Film Festival 2020 Ganador Festival Award People's Choice Best Actor The Deep Ones Compartido con: Johann Urb", 'Wildsound Feedback Writing and Film Festival 2022 Ganador Best Performance Award Best Performances The Samaritan Compartido con: Andrew Onochie · Devon Stewart · John Tague · Brett Wagner · Antonio D. Charity · Darren Mangler · · · · · · ·', 'Hollywood Horrorfest 2019 Ganador Festival Award Best Actor Exorcism at 60,000 Feet']), ('Tatiana Neva', 'NULL', ['Reparto', 'Producción', 'Especialistas'], ['su papel en Rush Hour', 'The Mindy Project', 'Prey of Wrath'], 'NULL')]]}


In [3]:
df_todo=pd.DataFrame(dic_peli)
df_todo
#convierto en cvs 
df_todo.to_csv('actores_pelis_700_800.csv', index=False)